In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

In [ ]:
import pandas as pd

In [ ]:
data = pd.read_csv('/content/drive/My Drive/Teste_A3Data/id_desc.csv')

In [ ]:
data.head()

,id,desc
0,004c744a-2b82-49b3-a536-90dbb68f368e,Vectra is a cybersecurity platform that uses A...
1,0062d2eb-8210-480a-adef-ec91d0e74855,Roadzen is a global leader in the P&C insuranc...
2,00657562-4dd3-41f9-aceb-df114606bc9e,Restream is a multi-streaming solution that al...
3,00b16654-5d01-4f2f-b6b3-ee2ef13c0d9e,DLP works for 1% of companies; fortunately the...
4,00b4e919-4d10-4ce8-b013-b6cf73ebd780,Shift Technology delivers AI-native decision a...


In [ ]:
data.shape

(3617, 2)

In [ ]:
data[data["desc"].isna()]

,id,desc
28,0b24ca09-7a47-4513-91d2-dc0e245c9827,NaN
95,2ea3e420-25ff-43ae-9362-40d0122ceede,NaN
116,37573898-6cf3-4946-9467-0af84fdc7823,NaN
133,4182b4cb-fe1e-4f7f-b090-5455eb84f226,NaN
161,50b3f33a-a7e8-481c-927b-a6fc9c24cde5,NaN
...,...,...
3167,d763163f-c397-4a9b-b0ef-15682ada5c6f,NaN
3192,d9546de6-a6de-436c-b1e0-212091463b13,NaN
3227,dcdf8d62-0f15-4218-bbfc-d5714ca861d0,NaN
3254,debd5d6e-b1af-4a07-82ca-4ec5de31a85c,NaN


In [ ]:
data = data.dropna()

In [ ]:
dict_data = data.to_dict('records')

In [ ]:
# function to convert to superscript
def get_super(x):
    normal = "ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz0123456789+-=()"
    super_s = "ᴬᴮᶜᴰᴱᶠᴳᴴᴵᴶᴷᴸᴹᴺᴼᴾQᴿˢᵀᵁⱽᵂˣʸᶻᵃᵇᶜᵈᵉᶠᵍʰᶦʲᵏˡᵐⁿᵒᵖ۹ʳˢᵗᵘᵛʷˣʸᶻ⁰¹²³⁴⁵⁶⁷⁸⁹⁺⁻⁼⁽⁾"
    res = x.maketrans(''.join(normal), ''.join(super_s))
    return x.translate(res)
  
# display superscipt
print(get_super('GeeksforGeeks')) #ᴳᵉᵉᵏˢᶠᵒʳᴳᵉᵉᵏˢ

ᴳᵉᵉᵏˢᶠᵒʳᴳᵉᵉᵏˢ


In [ ]:
#NER: company
ner_company_dict_list = {}
#Question: what is the company?
question_answer_dict_list = {}
#labeles samples 
n_labeled_samples = {'n':0}

labels_dict_list = {}
labels_dict_list['samples'] = {}

In [ ]:
labels_dict_list

{'samples': {}}

In [ ]:
def sample_id_generator(data_sample_ids):
  for id in data_sample_ids:
    yield id

In [ ]:
def insert_new_labels(sample_id, skip_ner = False, skip_answer = False):
  if sample_id in labels_dict_list['samples']:
    print("Already inserted!")
  else:
    labels_dict_list['samples'][sample_id] = {}
    if not skip_ner:
      labels_dict_list['samples'][sample_id]['ner_company'] = {'start':start_ner_company, 'end':end_ner_company}
      print('NER:', labels_dict_list['samples'][sample_id]['ner_company'])
    if not skip_answer:
      labels_dict_list['samples'][sample_id]['question_answer'] = {'start':start_answer, 'end':end_answer}
      print('Answer:', labels_dict_list['samples'][sample_id]['question_answer'])
    print('Total Labeling Count:', len(labels_dict_list['samples']))

In [ ]:
import random
random.seed(0)

In [ ]:
data_sample_ids = sample_id_generator(random.sample(range(len(dict_data)), 100))

In [ ]:
sample_id = next(data_sample_ids)
print(sample_id)
encoded_input = tokenizer(dict_data[sample_id]['desc'])

3458


In [ ]:
dict_data[sample_id]['desc']

'LifeScore Labs is an insurtech company delivers data-driven risk-scoring model.'

In [ ]:
" ".join([tokenizer.decode(token) + get_super(str(i)) for i, token in enumerate(encoded_input["input_ids"])])

'[CLS]⁰ Life¹ ##S² ##core³ Labs⁴ is⁵ an⁶ ins⁷ ##urt⁸ ##ech⁹ company¹⁰ delivers¹¹ data¹² -¹³ driven¹⁴ risk¹⁵ -¹⁶ scoring¹⁷ model¹⁸ .¹⁹ [SEP]²⁰'

In [ ]:
start_ner_company = 1
end_ner_company = 4 + 1

#Q1
start_answer = 1
end_answer = 19 + 1

print(tokenizer.decode(encoded_input["input_ids"][start_ner_company:end_ner_company]))
print(tokenizer.decode(encoded_input["input_ids"][start_answer:end_answer]))

LifeScore Labs
LifeScore Labs is an insurtech company delivers data - driven risk - scoring model.


In [ ]:
insert_new_labels(sample_id)

NER: {'start': 1, 'end': 5}
Answer: {'start': 1, 'end': 20}
Total Labeling Count: 1


In [ ]:
sample_id = next(data_sample_ids)
print(sample_id)
encoded_input = tokenizer(dict_data[sample_id]['desc'])
dict_data[sample_id]['desc']

1577


'SHONAN Hospital specializes in health care and medical care services.'

In [ ]:
" ".join([tokenizer.decode(token) + get_super(str(i)) for i, token in enumerate(encoded_input["input_ids"])])

'[CLS]⁰ SH¹ ##ON² ##AN³ Hospital⁴ specializes⁵ in⁶ health⁷ care⁸ and⁹ medical¹⁰ care¹¹ services¹² .¹³ [SEP]¹⁴'

In [ ]:
start_ner_company = 1
end_ner_company = 3 + 1

#Q1
start_answer = 1
end_answer = 13 + 1

print(tokenizer.decode(encoded_input["input_ids"][start_ner_company:end_ner_company]))
print(tokenizer.decode(encoded_input["input_ids"][start_answer:end_answer]))

SHONAN
SHONAN Hospital specializes in health care and medical care services.


In [ ]:
insert_new_labels(sample_id)

NER: {'start': 1, 'end': 4}
Answer: {'start': 1, 'end': 14}
Total Labeling Count: 2


In [ ]:
sample_id = next(data_sample_ids)
print(sample_id)
encoded_input = tokenizer(dict_data[sample_id]['desc'])
dict_data[sample_id]['desc']

3104


'XREX is a neo-fintech leveling the playing field by partnering with banks, regulators, and verified individuals to redefine banking together. Our blockchain-driven solutions create a collective financial system that empowers all to participate and contribute to the global economy. Founded in 2018 and headquartered in Taipei, XREX comprises a teamof world-leading experts in cybersecurity, fintech, compliance, and cryptocurrency to offer a full suite of innovative products such as BitCheck, XREX Circles, Risk Level Detector to solve the dollar-liquidity shortage issues faced by cross-border merchants in emerging economies.Read More'

In [ ]:
" ".join([tokenizer.decode(token) + get_super(str(i)) for i, token in enumerate(encoded_input["input_ids"])])

'[CLS]⁰ X¹ ##RE² ##X³ is⁴ a⁵ neo⁶ -⁷ fin⁸ ##tech⁹ level¹⁰ ##ing¹¹ the¹² playing¹³ field¹⁴ by¹⁵ partner¹⁶ ##ing¹⁷ with¹⁸ banks¹⁹ ,²⁰ regulator²¹ ##s²² ,²³ and²⁴ verified²⁵ individuals²⁶ to²⁷ red²⁸ ##ef²⁹ ##ine³⁰ banking³¹ together³² .³³ Our³⁴ block³⁵ ##cha³⁶ ##in³⁷ -³⁸ driven³⁹ solutions⁴⁰ create⁴¹ a⁴² collective⁴³ financial⁴⁴ system⁴⁵ that⁴⁶ em⁴⁷ ##power⁴⁸ ##s⁴⁹ all⁵⁰ to⁵¹ participate⁵² and⁵³ contribute⁵⁴ to⁵⁵ the⁵⁶ global⁵⁷ economy⁵⁸ .⁵⁹ Founded⁶⁰ in⁶¹ 2018⁶² and⁶³ headquartered⁶⁴ in⁶⁵ Taipei⁶⁶ ,⁶⁷ X⁶⁸ ##RE⁶⁹ ##X⁷⁰ comprises⁷¹ a⁷² team⁷³ ##of⁷⁴ world⁷⁵ -⁷⁶ leading⁷⁷ experts⁷⁸ in⁷⁹ c⁸⁰ ##y⁸¹ ##bers⁸² ##ec⁸³ ##urity⁸⁴ ,⁸⁵ fin⁸⁶ ##tech⁸⁷ ,⁸⁸ compliance⁸⁹ ,⁹⁰ and⁹¹ cry⁹² ##pt⁹³ ##oc⁹⁴ ##ur⁹⁵ ##ren⁹⁶ ##cy⁹⁷ to⁹⁸ offer⁹⁹ a¹⁰⁰ full¹⁰¹ suite¹⁰² of¹⁰³ innovative¹⁰⁴ products¹⁰⁵ such¹⁰⁶ as¹⁰⁷ Bit¹⁰⁸ ##C¹⁰⁹ ##he¹¹⁰ ##ck¹¹¹ ,¹¹² X¹¹³ ##RE¹¹⁴ ##X¹¹⁵ Circle¹¹⁶ ##s¹¹⁷ ,¹¹⁸ Risk¹¹⁹ Level¹²⁰ Det¹²¹ ##ector¹²² to¹²³ solve¹²⁴ the¹²⁵ dollar¹²⁶ -¹²⁷ liquid¹²⁸ ##ity¹²⁹ shortage¹³⁰ issues¹³¹ faced¹³² by¹³³ c

In [ ]:
start_ner_company = 1
end_ner_company = 3 + 1

#Q1
start_answer = 1
end_answer = 33 + 1

print(tokenizer.decode(encoded_input["input_ids"][start_ner_company:end_ner_company]))
print(tokenizer.decode(encoded_input["input_ids"][start_answer:end_answer]))

XREX
XREX is a neo - fintech leveling the playing field by partnering with banks, regulators, and verified individuals to redefine banking together.


In [ ]:
insert_new_labels(sample_id)

NER: {'start': 1, 'end': 4}
Answer: {'start': 1, 'end': 34}
Total Labeling Count: 3


In [ ]:
sample_id = next(data_sample_ids)
print(sample_id)
encoded_input = tokenizer(dict_data[sample_id]['desc'])
dict_data[sample_id]['desc']

1722


'Meru Health offers a digital program that helps improve the lives of people living with depression and saves on healthcare costs. It develops and delivers cost-effective, clinically proven, and engaging digital solutions to empower people living with depression.Founded by Kristian Ranta, Albert Nazander, and Riku Lindholm in 2015, Meru Health isheadquartered in San Mateo, California.Read More'

In [ ]:
" ".join([tokenizer.decode(token) + get_super(str(i)) for i, token in enumerate(encoded_input["input_ids"])])

'[CLS]⁰ Me¹ ##ru² Health³ offers⁴ a⁵ digital⁶ program⁷ that⁸ helps⁹ improve¹⁰ the¹¹ lives¹² of¹³ people¹⁴ living¹⁵ with¹⁶ depression¹⁷ and¹⁸ saves¹⁹ on²⁰ healthcare²¹ costs²² .²³ It²⁴ develops²⁵ and²⁶ delivers²⁷ cost²⁸ -²⁹ effective³⁰ ,³¹ clinical³² ##ly³³ proven³⁴ ,³⁵ and³⁶ engaging³⁷ digital³⁸ solutions³⁹ to⁴⁰ em⁴¹ ##power⁴² people⁴³ living⁴⁴ with⁴⁵ depression⁴⁶ .⁴⁷ Founded⁴⁸ by⁴⁹ Kris⁵⁰ ##tian⁵¹ Ra⁵² ##nta⁵³ ,⁵⁴ Albert⁵⁵ Na⁵⁶ ##zan⁵⁷ ##der⁵⁸ ,⁵⁹ and⁶⁰ R⁶¹ ##ik⁶² ##u⁶³ Lin⁶⁴ ##dh⁶⁵ ##olm⁶⁶ in⁶⁷ 2015⁶⁸ ,⁶⁹ Me⁷⁰ ##ru⁷¹ Health⁷² is⁷³ ##head⁷⁴ ##qua⁷⁵ ##rter⁷⁶ ##ed⁷⁷ in⁷⁸ San⁷⁹ Mateo⁸⁰ ,⁸¹ California⁸² .⁸³ Read⁸⁴ More⁸⁵ [SEP]⁸⁶'

In [ ]:
start_ner_company = 1
end_ner_company = 3 + 1

#Q1
start_answer = 1
end_answer = 23 + 1

print(tokenizer.decode(encoded_input["input_ids"][start_ner_company:end_ner_company]))
print(tokenizer.decode(encoded_input["input_ids"][start_answer:end_answer]))

Meru Health
Meru Health offers a digital program that helps improve the lives of people living with depression and saves on healthcare costs.


In [ ]:
insert_new_labels(sample_id)

NER: {'start': 1, 'end': 4}
Answer: {'start': 1, 'end': 24}
Total Labeling Count: 4


In [ ]:
sample_id = next(data_sample_ids)
print(sample_id)
encoded_input = tokenizer(dict_data[sample_id]['desc'])
dict_data[sample_id]['desc']

165


'Aqua Security is the largest pure-play cloud native security company, providing customers the freedom to innovate and accelerate their digital transformations. The Aqua Platform provides prevention, detection and response automation across the entire application lifecycle to secure the build, secure cloud infrastructure and secure running workloadswherever they are deployed. Aqua customers are among the world’s largest enterprises in financial services, software, media, manufacturing and retail, with implementations across a broad range of cloud providers and modern technology stacks spanning containers, serverless functions and cloud VMs.Read More'

In [ ]:
" ".join([tokenizer.decode(token) + get_super(str(i)) for i, token in enumerate(encoded_input["input_ids"])])

'[CLS]⁰ A¹ ##qua² Security³ is⁴ the⁵ largest⁶ pure⁷ -⁸ play⁹ cloud¹⁰ native¹¹ security¹² company¹³ ,¹⁴ providing¹⁵ customers¹⁶ the¹⁷ freedom¹⁸ to¹⁹ inn²⁰ ##ova²¹ ##te²² and²³ accelerate²⁴ their²⁵ digital²⁶ transformations²⁷ .²⁸ The²⁹ A³⁰ ##qua³¹ Platform³² provides³³ prevention³⁴ ,³⁵ detection³⁶ and³⁷ response³⁸ automation³⁹ across⁴⁰ the⁴¹ entire⁴² application⁴³ life⁴⁴ ##cycle⁴⁵ to⁴⁶ secure⁴⁷ the⁴⁸ build⁴⁹ ,⁵⁰ secure⁵¹ cloud⁵² infrastructure⁵³ and⁵⁴ secure⁵⁵ running⁵⁶ work⁵⁷ ##load⁵⁸ ##s⁵⁹ ##where⁶⁰ ##ver⁶¹ they⁶² are⁶³ deployed⁶⁴ .⁶⁵ A⁶⁶ ##qua⁶⁷ customers⁶⁸ are⁶⁹ among⁷⁰ the⁷¹ world⁷² ’⁷³ s⁷⁴ largest⁷⁵ enterprises⁷⁶ in⁷⁷ financial⁷⁸ services⁷⁹ ,⁸⁰ software⁸¹ ,⁸² media⁸³ ,⁸⁴ manufacturing⁸⁵ and⁸⁶ retail⁸⁷ ,⁸⁸ with⁸⁹ implementation⁹⁰ ##s⁹¹ across⁹² a⁹³ broad⁹⁴ range⁹⁵ of⁹⁶ cloud⁹⁷ providers⁹⁸ and⁹⁹ modern¹⁰⁰ technology¹⁰¹ stacks¹⁰² spanning¹⁰³ containers¹⁰⁴ ,¹⁰⁵ server¹⁰⁶ ##less¹⁰⁷ functions¹⁰⁸ and¹⁰⁹ cloud¹¹⁰ V¹¹¹ ##Ms¹¹² .¹¹³ Read¹¹⁴ More¹¹⁵ [SEP]¹¹⁶'

In [ ]:
start_ner_company = 1
end_ner_company = 3 + 1

#Q1
start_answer = 1
end_answer = 28 + 1

print(tokenizer.decode(encoded_input["input_ids"][start_ner_company:end_ner_company]))
print(tokenizer.decode(encoded_input["input_ids"][start_answer:end_answer]))

Aqua Security
Aqua Security is the largest pure - play cloud native security company, providing customers the freedom to innovate and accelerate their digital transformations.


In [ ]:
insert_new_labels(sample_id)

NER: {'start': 1, 'end': 4}
Answer: {'start': 1, 'end': 29}
Total Labeling Count: 5


In [ ]:
sample_id = next(data_sample_ids)
print(sample_id)
encoded_input = tokenizer(dict_data[sample_id]['desc'])
dict_data[sample_id]['desc']

1060


'Resourcify Recycling Platform\u2028. An operating system for recycling and waste management. We’re connecting companies and recyclers together to make recycling easy, fast and affordable - with one software for all of your locations, recyclers, containers, and waste streams.'

In [ ]:
" ".join([tokenizer.decode(token) + get_super(str(i)) for i, token in enumerate(encoded_input["input_ids"])])

'[CLS]⁰ Re¹ ##so² ##ur³ ##ci⁴ ##fy⁵ Re⁶ ##cy⁷ ##c⁸ ##ling⁹ Platform¹⁰ .¹¹ An¹² operating¹³ system¹⁴ for¹⁵ recycling¹⁶ and¹⁷ waste¹⁸ management¹⁹ .²⁰ We²¹ ’²² re²³ connecting²⁴ companies²⁵ and²⁶ re²⁷ ##cycle²⁸ ##rs²⁹ together³⁰ to³¹ make³² recycling³³ easy³⁴ ,³⁵ fast³⁶ and³⁷ affordable³⁸ -³⁹ with⁴⁰ one⁴¹ software⁴² for⁴³ all⁴⁴ of⁴⁵ your⁴⁶ locations⁴⁷ ,⁴⁸ re⁴⁹ ##cycle⁵⁰ ##rs⁵¹ ,⁵² containers⁵³ ,⁵⁴ and⁵⁵ waste⁵⁶ streams⁵⁷ .⁵⁸ [SEP]⁵⁹'

In [ ]:
start_ner_company = 1
end_ner_company = 10 + 1

#Q1
start_answer = 12
end_answer = 20 + 1

print(tokenizer.decode(encoded_input["input_ids"][start_ner_company:end_ner_company]))
print(tokenizer.decode(encoded_input["input_ids"][start_answer:end_answer]))

Resourcify Recycling Platform
An operating system for recycling and waste management.


In [ ]:
insert_new_labels(sample_id)

NER: {'start': 1, 'end': 11}
Answer: {'start': 12, 'end': 21}
Total Labeling Count: 6


In [ ]:
sample_id = next(data_sample_ids)
print(sample_id)
encoded_input = tokenizer(dict_data[sample_id]['desc'])
dict_data[sample_id]['desc']

2094


'Amelia renders banking, HR, banking, insurance, healthcare, telecommunication and IT services.'

In [ ]:
" ".join([tokenizer.decode(token) + get_super(str(i)) for i, token in enumerate(encoded_input["input_ids"])])

'[CLS]⁰ Amelia¹ render² ##s³ banking⁴ ,⁵ H⁶ ##R⁷ ,⁸ banking⁹ ,¹⁰ insurance¹¹ ,¹² healthcare¹³ ,¹⁴ te¹⁵ ##le¹⁶ ##com¹⁷ ##munication¹⁸ and¹⁹ IT²⁰ services²¹ .²² [SEP]²³'

In [ ]:
start_ner_company = 1
end_ner_company = 1 + 1

#Q1
start_answer = 1
end_answer = 22 + 1

print(tokenizer.decode(encoded_input["input_ids"][start_ner_company:end_ner_company]))
print(tokenizer.decode(encoded_input["input_ids"][start_answer:end_answer]))

Amelia
Amelia renders banking, HR, banking, insurance, healthcare, telecommunication and IT services.


In [ ]:
insert_new_labels(sample_id)

NER: {'start': 1, 'end': 2}
Answer: {'start': 1, 'end': 23}
Total Labeling Count: 7


In [ ]:
sample_id = next(data_sample_ids)
print(sample_id)
encoded_input = tokenizer(dict_data[sample_id]['desc'])
dict_data[sample_id]['desc']

1990


'Code42 provides data loss protection, visibility, and recovery solutions. Its data loss protection solution rapidly detects insider threats, satisfies regulatory compliance requirements and speeds incident response, all without lengthy deployments, complex policy management or blocks on user collaboration. Its security, IT and complianceprofessionals can protect endpoint and cloud data from loss, leak and theft while maintaining an open and collaborative culture for employees.Founded in 2001, more than 50,000 organizations worldwide, including the most recognized brands in business and education, rely on Code42 to safeguard their ideas. The company is headquartered in Minneapolis, Minnesota, and backed by Accel Partners, JMI Equity, NEA, and Split Rock Partners.Read More'

In [ ]:
" ".join([tokenizer.decode(token) + get_super(str(i)) for i, token in enumerate(encoded_input["input_ids"])])

'[CLS]⁰ Code¹ ##42² provides³ data⁴ loss⁵ protection⁶ ,⁷ visibility⁸ ,⁹ and¹⁰ recovery¹¹ solutions¹² .¹³ Its¹⁴ data¹⁵ loss¹⁶ protection¹⁷ solution¹⁸ rapidly¹⁹ detect²⁰ ##s²¹ inside²² ##r²³ threats²⁴ ,²⁵ sat²⁶ ##is²⁷ ##fies²⁸ regulatory²⁹ compliance³⁰ requirements³¹ and³² speeds³³ incident³⁴ response³⁵ ,³⁶ all³⁷ without³⁸ lengthy³⁹ deployment⁴⁰ ##s⁴¹ ,⁴² complex⁴³ policy⁴⁴ management⁴⁵ or⁴⁶ blocks⁴⁷ on⁴⁸ user⁴⁹ collaboration⁵⁰ .⁵¹ Its⁵² security⁵³ ,⁵⁴ IT⁵⁵ and⁵⁶ compliance⁵⁷ ##p⁵⁸ ##ro⁵⁹ ##fe⁶⁰ ##ssion⁶¹ ##als⁶² can⁶³ protect⁶⁴ end⁶⁵ ##point⁶⁶ and⁶⁷ cloud⁶⁸ data⁶⁹ from⁷⁰ loss⁷¹ ,⁷² leak⁷³ and⁷⁴ theft⁷⁵ while⁷⁶ maintaining⁷⁷ an⁷⁸ open⁷⁹ and⁸⁰ collaborative⁸¹ culture⁸² for⁸³ employees⁸⁴ .⁸⁵ Founded⁸⁶ in⁸⁷ 2001⁸⁸ ,⁸⁹ more⁹⁰ than⁹¹ 50⁹² ,⁹³ 000⁹⁴ organizations⁹⁵ worldwide⁹⁶ ,⁹⁷ including⁹⁸ the⁹⁹ most¹⁰⁰ recognized¹⁰¹ brands¹⁰² in¹⁰³ business¹⁰⁴ and¹⁰⁵ education¹⁰⁶ ,¹⁰⁷ rely¹⁰⁸ on¹⁰⁹ Code¹¹⁰ ##42¹¹¹ to¹¹² safe¹¹³ ##guard¹¹⁴ their¹¹⁵ ideas¹¹⁶ .¹¹⁷ The¹¹⁸ company¹¹⁹ is¹²⁰ headquartered¹²¹ in¹²

In [ ]:
start_ner_company = 1
end_ner_company = 2 + 1

#Q1
start_answer = 1
end_answer = 13 + 1

print(tokenizer.decode(encoded_input["input_ids"][start_ner_company:end_ner_company]))
print(tokenizer.decode(encoded_input["input_ids"][start_answer:end_answer]))

Code42
Code42 provides data loss protection, visibility, and recovery solutions.


In [ ]:
insert_new_labels(sample_id)

NER: {'start': 1, 'end': 3}
Answer: {'start': 1, 'end': 14}
Total Labeling Count: 8


In [ ]:
sample_id = next(data_sample_ids)
print(sample_id)
encoded_input = tokenizer(dict_data[sample_id]['desc'])
dict_data[sample_id]['desc']

1658


'Chromatox is a toxicology testing laboratory that specializes in hair drug analysis, testing, and calibration, and online report services.'

In [ ]:
" ".join([tokenizer.decode(token) + get_super(str(i)) for i, token in enumerate(encoded_input["input_ids"])])

'[CLS]⁰ Ch¹ ##roma² ##to³ ##x⁴ is⁵ a⁶ toxic⁷ ##ology⁸ testing⁹ laboratory¹⁰ that¹¹ specializes¹² in¹³ hair¹⁴ drug¹⁵ analysis¹⁶ ,¹⁷ testing¹⁸ ,¹⁹ and²⁰ ca²¹ ##li²² ##bra²³ ##tion²⁴ ,²⁵ and²⁶ online²⁷ report²⁸ services²⁹ .³⁰ [SEP]³¹'

In [ ]:
start_ner_company = 1
end_ner_company = 2 + 1

#Q1
start_answer = 1
end_answer = 30 + 1

print(tokenizer.decode(encoded_input["input_ids"][start_ner_company:end_ner_company]))
print(tokenizer.decode(encoded_input["input_ids"][start_answer:end_answer]))

Chroma
Chromatox is a toxicology testing laboratory that specializes in hair drug analysis, testing, and calibration, and online report services.


In [ ]:
insert_new_labels(sample_id)

NER: {'start': 1, 'end': 3}
Answer: {'start': 1, 'end': 31}
Total Labeling Count: 9


In [ ]:
sample_id = next(data_sample_ids)
print(sample_id)
encoded_input = tokenizer(dict_data[sample_id]['desc'])
dict_data[sample_id]['desc']

3210


'Fetch Robotics is a San Jose, Calif.-based industrial robotics startup that develops and manufactures collaborative, autonomous mobile robot solutions for the warehousing and logistical markets. It takes takes advantage of its deep robotics expertise to provide reliable and safe collaborative AMR (Autonomous Mobile Robot) solutions for twocommercial applications: material handling and data collection.Fetch Robotics is a venture-backed Silicon Valley startup founded in 2014.Read More'

In [ ]:
" ".join([tokenizer.decode(token) + get_super(str(i)) for i, token in enumerate(encoded_input["input_ids"])])

'[CLS]⁰ Fe¹ ##tch² Robot³ ##ics⁴ is⁵ a⁶ San⁷ Jose⁸ ,⁹ Cal¹⁰ ##if¹¹ .¹² -¹³ based¹⁴ industrial¹⁵ robotic¹⁶ ##s¹⁷ start¹⁸ ##up¹⁹ that²⁰ develops²¹ and²² manufactures²³ collaborative²⁴ ,²⁵ autonomous²⁶ mobile²⁷ robot²⁸ solutions²⁹ for³⁰ the³¹ war³² ##eh³³ ##ous³⁴ ##ing³⁵ and³⁶ log³⁷ ##istic³⁸ ##al³⁹ markets⁴⁰ .⁴¹ It⁴² takes⁴³ takes⁴⁴ advantage⁴⁵ of⁴⁶ its⁴⁷ deep⁴⁸ robotic⁴⁹ ##s⁵⁰ expertise⁵¹ to⁵² provide⁵³ reliable⁵⁴ and⁵⁵ safe⁵⁶ collaborative⁵⁷ AM⁵⁸ ##R⁵⁹ (⁶⁰ Autonomous⁶¹ Mobile⁶² Robot⁶³ )⁶⁴ solutions⁶⁵ for⁶⁶ two⁶⁷ ##com⁶⁸ ##mer⁶⁹ ##cial⁷⁰ applications⁷¹ :⁷² material⁷³ handling⁷⁴ and⁷⁵ data⁷⁶ collection⁷⁷ .⁷⁸ Fe⁷⁹ ##tch⁸⁰ Robot⁸¹ ##ics⁸² is⁸³ a⁸⁴ venture⁸⁵ -⁸⁶ backed⁸⁷ Silicon⁸⁸ Valley⁸⁹ start⁹⁰ ##up⁹¹ founded⁹² in⁹³ 2014⁹⁴ .⁹⁵ Read⁹⁶ More⁹⁷ [SEP]⁹⁸'

In [ ]:
start_ner_company = 1
end_ner_company = 4 + 1

#Q1
start_answer = 1
end_answer = 41 + 1

print(tokenizer.decode(encoded_input["input_ids"][start_ner_company:end_ner_company]))
print(tokenizer.decode(encoded_input["input_ids"][start_answer:end_answer]))

Fetch Robotics
Fetch Robotics is a San Jose, Calif. - based industrial robotics startup that develops and manufactures collaborative, autonomous mobile robot solutions for the warehousing and logistical markets.


In [ ]:
insert_new_labels(sample_id)

NER: {'start': 1, 'end': 5}
Answer: {'start': 1, 'end': 42}
Total Labeling Count: 10


In [ ]:
sample_id = next(data_sample_ids)
print(sample_id)
encoded_input = tokenizer(dict_data[sample_id]['desc'])
dict_data[sample_id]['desc']

3399


'We are a software and service company focused on the electrical sector. We offer solutions for telemetry and intelligence applications on energy data. We work side by side with companies in the industry with the objective of increasing their operational efficiency, ensuring their regulatory adequacy and improving their relationship with customersthrough the application of technologies.Read More'

In [ ]:
" ".join([tokenizer.decode(token) + get_super(str(i)) for i, token in enumerate(encoded_input["input_ids"])])

'[CLS]⁰ We¹ are² a³ software⁴ and⁵ service⁶ company⁷ focused⁸ on⁹ the¹⁰ electrical¹¹ sector¹² .¹³ We¹⁴ offer¹⁵ solutions¹⁶ for¹⁷ te¹⁸ ##lem¹⁹ ##et²⁰ ##ry²¹ and²² intelligence²³ applications²⁴ on²⁵ energy²⁶ data²⁷ .²⁸ We²⁹ work³⁰ side³¹ by³² side³³ with³⁴ companies³⁵ in³⁶ the³⁷ industry³⁸ with³⁹ the⁴⁰ objective⁴¹ of⁴² increasing⁴³ their⁴⁴ operational⁴⁵ efficiency⁴⁶ ,⁴⁷ ensuring⁴⁸ their⁴⁹ regulatory⁵⁰ ad⁵¹ ##e⁵² ##qua⁵³ ##cy⁵⁴ and⁵⁵ improving⁵⁶ their⁵⁷ relationship⁵⁸ with⁵⁹ customers⁶⁰ ##th⁶¹ ##rough⁶² the⁶³ application⁶⁴ of⁶⁵ technologies⁶⁶ .⁶⁷ Read⁶⁸ More⁶⁹ [SEP]⁷⁰'

In [ ]:
#Q1
start_answer = 1
end_answer = 13 + 1

print(tokenizer.decode(encoded_input["input_ids"][start_answer:end_answer]))

We are a software and service company focused on the electrical sector.


In [ ]:
insert_new_labels(sample_id, skip_ner = True)

Answer: {'start': 1, 'end': 14}
Total Labeling Count: 11


In [ ]:
sample_id = next(data_sample_ids)
print(sample_id)
encoded_input = tokenizer(dict_data[sample_id]['desc'])
dict_data[sample_id]['desc']

1242


"Fitzabout is a fast-growing online health and fitness publisher, providing expert information on yoga, exercise, nutrition and recipes to its users via smartphone or computer. Fitzabout's users unexpectedly get expert information about health, fitness, yoga, exercise, nutrition, recipes, and supplements and get a powerful insight that keepsthe user within their range of fitness.Read More"

In [ ]:
" ".join([tokenizer.decode(token) + get_super(str(i)) for i, token in enumerate(encoded_input["input_ids"])])

"[CLS]⁰ Fi¹ ##tz² ##about³ is⁴ a⁵ fast⁶ -⁷ growing⁸ online⁹ health¹⁰ and¹¹ fitness¹² publisher¹³ ,¹⁴ providing¹⁵ expert¹⁶ information¹⁷ on¹⁸ yoga¹⁹ ,²⁰ exercise²¹ ,²² nutrition²³ and²⁴ recipes²⁵ to²⁶ its²⁷ users²⁸ via²⁹ smart³⁰ ##phone³¹ or³² computer³³ .³⁴ Fi³⁵ ##tz³⁶ ##about³⁷ '³⁸ s³⁹ users⁴⁰ unexpectedly⁴¹ get⁴² expert⁴³ information⁴⁴ about⁴⁵ health⁴⁶ ,⁴⁷ fitness⁴⁸ ,⁴⁹ yoga⁵⁰ ,⁵¹ exercise⁵² ,⁵³ nutrition⁵⁴ ,⁵⁵ recipes⁵⁶ ,⁵⁷ and⁵⁸ supplement⁵⁹ ##s⁶⁰ and⁶¹ get⁶² a⁶³ powerful⁶⁴ insight⁶⁵ that⁶⁶ keeps⁶⁷ ##the⁶⁸ user⁶⁹ within⁷⁰ their⁷¹ range⁷² of⁷³ fitness⁷⁴ .⁷⁵ Read⁷⁶ More⁷⁷ [SEP]⁷⁸"

In [ ]:
#NER
start_ner_company = 1
end_ner_company = 3 + 1

#Q1
start_answer = 1
end_answer = 34 + 1

print(tokenizer.decode(encoded_input["input_ids"][start_ner_company:end_ner_company]))
print(tokenizer.decode(encoded_input["input_ids"][start_answer:end_answer]))

Fitzabout
Fitzabout is a fast - growing online health and fitness publisher, providing expert information on yoga, exercise, nutrition and recipes to its users via smartphone or computer.


In [ ]:
insert_new_labels(sample_id)

NER: {'start': 1, 'end': 4}
Answer: {'start': 1, 'end': 35}
Total Labeling Count: 12


In [ ]:
sample_id = next(data_sample_ids)
print(sample_id)
encoded_input = tokenizer(dict_data[sample_id]['desc'])
dict_data[sample_id]['desc']

1952


'6over6 offers a digital healthcare platform that enables users to perform their own eye vision test. They have reimagined all fundamental optometric tools and fitted them into smartphone, enabling a complete and accurate measurement of the refractive error of the eye, for eyeglasses and contact lens prescription, using software.GlassesOn isregistered as a Class 1 Exempt Medical Device on the FDA Medical Device Listings. However, GlassesOn does not provide medical advice, and does not replace a full eye health exam.Read More'

In [ ]:
" ".join([tokenizer.decode(token) + get_super(str(i)) for i, token in enumerate(encoded_input["input_ids"])])

'[CLS]⁰ 6¹ ##over² ##6³ offers⁴ a⁵ digital⁶ healthcare⁷ platform⁸ that⁹ enables¹⁰ users¹¹ to¹² perform¹³ their¹⁴ own¹⁵ eye¹⁶ vision¹⁷ test¹⁸ .¹⁹ They²⁰ have²¹ re²² ##ima²³ ##gin²⁴ ##ed²⁵ all²⁶ fundamental²⁷ op²⁸ ##tom²⁹ ##et³⁰ ##ric³¹ tools³² and³³ fitted³⁴ them³⁵ into³⁶ smart³⁷ ##phone³⁸ ,³⁹ enabling⁴⁰ a⁴¹ complete⁴² and⁴³ accurate⁴⁴ measurement⁴⁵ of⁴⁶ the⁴⁷ re⁴⁸ ##fra⁴⁹ ##ctive⁵⁰ error⁵¹ of⁵² the⁵³ eye⁵⁴ ,⁵⁵ for⁵⁶ eye⁵⁷ ##glass⁵⁸ ##es⁵⁹ and⁶⁰ contact⁶¹ lens⁶² prescription⁶³ ,⁶⁴ using⁶⁵ software⁶⁶ .⁶⁷ Glass⁶⁸ ##es⁶⁹ ##O⁷⁰ ##n⁷¹ is⁷² ##re⁷³ ##gis⁷⁴ ##tered⁷⁵ as⁷⁶ a⁷⁷ Class⁷⁸ 1⁷⁹ Exempt⁸⁰ Medical⁸¹ Devi⁸² ##ce⁸³ on⁸⁴ the⁸⁵ FDA⁸⁶ Medical⁸⁷ Devi⁸⁸ ##ce⁸⁹ List⁹⁰ ##ings⁹¹ .⁹² However⁹³ ,⁹⁴ Glass⁹⁵ ##es⁹⁶ ##O⁹⁷ ##n⁹⁸ does⁹⁹ not¹⁰⁰ provide¹⁰¹ medical¹⁰² advice¹⁰³ ,¹⁰⁴ and¹⁰⁵ does¹⁰⁶ not¹⁰⁷ replace¹⁰⁸ a¹⁰⁹ full¹¹⁰ eye¹¹¹ health¹¹² exam¹¹³ .¹¹⁴ Read¹¹⁵ More¹¹⁶ [SEP]¹¹⁷'

In [ ]:
#NER
start_ner_company = 1
end_ner_company = 3 + 1

#Q1
start_answer = 1
end_answer = 19 + 1

print(tokenizer.decode(encoded_input["input_ids"][start_ner_company:end_ner_company]))
print(tokenizer.decode(encoded_input["input_ids"][start_answer:end_answer]))

6over6
6over6 offers a digital healthcare platform that enables users to perform their own eye vision test.


In [ ]:
insert_new_labels(sample_id)

NER: {'start': 1, 'end': 4}
Answer: {'start': 1, 'end': 20}
Total Labeling Count: 13


In [ ]:
sample_id = next(data_sample_ids)
print(sample_id)
encoded_input = tokenizer(dict_data[sample_id]['desc'])
dict_data[sample_id]['desc']

1466


'Limitless enables businesses to connect customers with qualified experts to solve customer service enquiries on-demand, providing a faster and better service for significantly less cost. Expert responses and customer feedback then train our AI systems to automate low value queries overtime. The platform is live in over 20 countries and majorclients include Microsoft, Unilever and Sage. Current investors include Unilever Ventures, Downing Ventures and Albion Capital.Read More'

In [ ]:
" ".join([tokenizer.decode(token) + get_super(str(i)) for i, token in enumerate(encoded_input["input_ids"])])

'[CLS]⁰ Lim¹ ##it² ##less³ enables⁴ businesses⁵ to⁶ connect⁷ customers⁸ with⁹ qualified¹⁰ experts¹¹ to¹² solve¹³ customer¹⁴ service¹⁵ en¹⁶ ##quiries¹⁷ on¹⁸ -¹⁹ demand²⁰ ,²¹ providing²² a²³ faster²⁴ and²⁵ better²⁶ service²⁷ for²⁸ significantly²⁹ less³⁰ cost³¹ .³² Ex³³ ##pert³⁴ responses³⁵ and³⁶ customer³⁷ feedback³⁸ then³⁹ train⁴⁰ our⁴¹ AI⁴² systems⁴³ to⁴⁴ auto⁴⁵ ##mate⁴⁶ low⁴⁷ value⁴⁸ que⁴⁹ ##ries⁵⁰ overtime⁵¹ .⁵² The⁵³ platform⁵⁴ is⁵⁵ live⁵⁶ in⁵⁷ over⁵⁸ 20⁵⁹ countries⁶⁰ and⁶¹ major⁶² ##c⁶³ ##lient⁶⁴ ##s⁶⁵ include⁶⁶ Microsoft⁶⁷ ,⁶⁸ Un⁶⁹ ##ile⁷⁰ ##ver⁷¹ and⁷² Sage⁷³ .⁷⁴ Current⁷⁵ investors⁷⁶ include⁷⁷ Un⁷⁸ ##ile⁷⁹ ##ver⁸⁰ Ventures⁸¹ ,⁸² Down⁸³ ##ing⁸⁴ Ventures⁸⁵ and⁸⁶ Albion⁸⁷ Capital⁸⁸ .⁸⁹ Read⁹⁰ More⁹¹ [SEP]⁹²'

In [ ]:
#NER
start_ner_company = 1
end_ner_company = 3 + 1

#Q1
start_answer = 1
end_answer = 32 + 1

print(tokenizer.decode(encoded_input["input_ids"][start_ner_company:end_ner_company]))
print(tokenizer.decode(encoded_input["input_ids"][start_answer:end_answer]))

Limitless
Limitless enables businesses to connect customers with qualified experts to solve customer service enquiries on - demand, providing a faster and better service for significantly less cost.


In [ ]:
insert_new_labels(sample_id)

NER: {'start': 1, 'end': 4}
Answer: {'start': 1, 'end': 33}
Total Labeling Count: 14


In [ ]:
sample_id = next(data_sample_ids)
print(sample_id)
encoded_input = tokenizer(dict_data[sample_id]['desc'])
dict_data[sample_id]['desc']

2389


'ClauseMatch is a London-based financial technology company that provides a software-as-a-service platform for smart document management. Its flagship product is a next-gen innovative online editor which brings unprecedented productivity for compliance, legal, finance, operations, and risk teams saving millions in terms of time and resources, whilesignificantly reducing risk and providing accountability.ClauseMatch works as a real-time browser-based collaborative document editor containing in its core a detailed workflow, where comments, approvals, and changes are a part of a full audit trail. That brings complete control of content, streamlines complex workflows, removing human error for a better insight into senior management. Every change and approval made in a document is tracked in an organized manner providing full visibility and groundbreaking reporting capabilities.Current customers include global banks that use the ClauseMatch platform for centralized policies management, regu

In [ ]:
" ".join([tokenizer.decode(token) + get_super(str(i)) for i, token in enumerate(encoded_input["input_ids"])])

'[CLS]⁰ Claus¹ ##e² ##M³ ##atch⁴ is⁵ a⁶ London⁷ -⁸ based⁹ financial¹⁰ technology¹¹ company¹² that¹³ provides¹⁴ a¹⁵ software¹⁶ -¹⁷ as¹⁸ -¹⁹ a²⁰ -²¹ service²² platform²³ for²⁴ smart²⁵ document²⁶ management²⁷ .²⁸ Its²⁹ flagship³⁰ product³¹ is³² a³³ next³⁴ -³⁵ g³⁶ ##en³⁷ innovative³⁸ online³⁹ editor⁴⁰ which⁴¹ brings⁴² unprecedented⁴³ productivity⁴⁴ for⁴⁵ compliance⁴⁶ ,⁴⁷ legal⁴⁸ ,⁴⁹ finance⁵⁰ ,⁵¹ operations⁵² ,⁵³ and⁵⁴ risk⁵⁵ teams⁵⁶ saving⁵⁷ millions⁵⁸ in⁵⁹ terms⁶⁰ of⁶¹ time⁶² and⁶³ resources⁶⁴ ,⁶⁵ while⁶⁶ ##sign⁶⁷ ##ificant⁶⁸ ##ly⁶⁹ reducing⁷⁰ risk⁷¹ and⁷² providing⁷³ account⁷⁴ ##ability⁷⁵ .⁷⁶ Claus⁷⁷ ##e⁷⁸ ##M⁷⁹ ##atch⁸⁰ works⁸¹ as⁸² a⁸³ real⁸⁴ -⁸⁵ time⁸⁶ browser⁸⁷ -⁸⁸ based⁸⁹ collaborative⁹⁰ document⁹¹ editor⁹² containing⁹³ in⁹⁴ its⁹⁵ core⁹⁶ a⁹⁷ detailed⁹⁸ work⁹⁹ ##flow¹⁰⁰ ,¹⁰¹ where¹⁰² comments¹⁰³ ,¹⁰⁴ approval¹⁰⁵ ##s¹⁰⁶ ,¹⁰⁷ and¹⁰⁸ changes¹⁰⁹ are¹¹⁰ a¹¹¹ part¹¹² of¹¹³ a¹¹⁴ full¹¹⁵ audit¹¹⁶ trail¹¹⁷ .¹¹⁸ That¹¹⁹ brings¹²⁰ complete¹²¹ control¹²² of¹²³ content¹²⁴ ,¹²⁵ stream¹²⁶ ##lines¹

In [ ]:
#NER
start_ner_company = 1
end_ner_company = 4 + 1

#Q1
start_answer = 1
end_answer = 28 + 1

print(tokenizer.decode(encoded_input["input_ids"][start_ner_company:end_ner_company]))
print(tokenizer.decode(encoded_input["input_ids"][start_answer:end_answer]))

ClauseMatch
ClauseMatch is a London - based financial technology company that provides a software - as - a - service platform for smart document management.


In [ ]:
insert_new_labels(sample_id)

NER: {'start': 1, 'end': 5}
Answer: {'start': 1, 'end': 29}
Total Labeling Count: 15


In [ ]:
sample_id = next(data_sample_ids)
print(sample_id)
encoded_input = tokenizer(dict_data[sample_id]['desc'])
dict_data[sample_id]['desc']

894


'HighRadius is a Fintech enterprise Software-as-a-Service (SaaS) company that leverages Artificial Intelligence-based Autonomous Systems to help 600+ industry-leading companies automate their Accounts Receivable and Treasury processes. The HighRadius® Integrated Receivables platform reduces cycle times in your order-to-cash process throughautomation of receivables and payments processes across credit, electronic billing and payment processing, cash application, deductions, and collections. The HighRadius® RadiusOne AR Suite offers a pocket-friendly platform for hundreds of mid-sized businesses to enable faster A/R processing and enhance their working capital.  HighRadius® Treasury Management Applications help teams achieve touchless cash management and accurate cash forecasting. Powered by the Rivana™ Artificial Intelligence Engine and Freeda™ Digital Assistant for order-to-cash teams, HighRadius enables teams to leverage machine learning to predict future outcomes and automate routine

In [ ]:
" ".join([tokenizer.decode(token) + get_super(str(i)) for i, token in enumerate(encoded_input["input_ids"])])

'[CLS]⁰ High¹ ##R² ##adi³ ##us⁴ is⁵ a⁶ Fin⁷ ##tech⁸ enterprise⁹ Software¹⁰ -¹¹ as¹² -¹³ a¹⁴ -¹⁵ Service¹⁶ (¹⁷ Sa¹⁸ ##a¹⁹ ##S²⁰ )²¹ company²² that²³ leverage²⁴ ##s²⁵ Art²⁶ ##ific²⁷ ##ial²⁸ Intelligence²⁹ -³⁰ based³¹ Autonomous³² Systems³³ to³⁴ help³⁵ 600³⁶ +³⁷ industry³⁸ -³⁹ leading⁴⁰ companies⁴¹ auto⁴² ##mate⁴³ their⁴⁴ A⁴⁵ ##cco⁴⁶ ##unts⁴⁷ Re⁴⁸ ##ce⁴⁹ ##ivable⁵⁰ and⁵¹ Treasury⁵² processes⁵³ .⁵⁴ The⁵⁵ High⁵⁶ ##R⁵⁷ ##adi⁵⁸ ##us⁵⁹ ##®⁶⁰ Integrated⁶¹ Re⁶² ##ce⁶³ ##ivable⁶⁴ ##s⁶⁵ platform⁶⁶ reduces⁶⁷ cycle⁶⁸ times⁶⁹ in⁷⁰ your⁷¹ order⁷² -⁷³ to⁷⁴ -⁷⁵ cash⁷⁶ process⁷⁷ through⁷⁸ ##aut⁷⁹ ##oma⁸⁰ ##tion⁸¹ of⁸² re⁸³ ##ce⁸⁴ ##ivable⁸⁵ ##s⁸⁶ and⁸⁷ payments⁸⁸ processes⁸⁹ across⁹⁰ credit⁹¹ ,⁹² electronic⁹³ bill⁹⁴ ##ing⁹⁵ and⁹⁶ payment⁹⁷ processing⁹⁸ ,⁹⁹ cash¹⁰⁰ application¹⁰¹ ,¹⁰² de¹⁰³ ##duction¹⁰⁴ ##s¹⁰⁵ ,¹⁰⁶ and¹⁰⁷ collections¹⁰⁸ .¹⁰⁹ The¹¹⁰ High¹¹¹ ##R¹¹² ##adi¹¹³ ##us¹¹⁴ ##®¹¹⁵ Ra¹¹⁶ ##dius¹¹⁷ ##O¹¹⁸ ##ne¹¹⁹ AR¹²⁰ Suite¹²¹ offers¹²² a¹²³ pocket¹²⁴ -¹²⁵ friendly¹²⁶ platform¹²⁷ for¹²⁸ hundreds¹²⁹ o

In [ ]:
#NER
start_ner_company = 1
end_ner_company = 4 + 1

#Q1
start_answer = 1
end_answer = 54 + 1

print(tokenizer.decode(encoded_input["input_ids"][start_ner_company:end_ner_company]))
print(tokenizer.decode(encoded_input["input_ids"][start_answer:end_answer]))

HighRadius
HighRadius is a Fintech enterprise Software - as - a - Service ( SaaS ) company that leverages Artificial Intelligence - based Autonomous Systems to help 600 + industry - leading companies automate their Accounts Receivable and Treasury processes.


In [ ]:
insert_new_labels(sample_id)

NER: {'start': 1, 'end': 5}
Answer: {'start': 1, 'end': 55}
Total Labeling Count: 16


In [ ]:
sample_id = next(data_sample_ids)
print(sample_id)
encoded_input = tokenizer(dict_data[sample_id]['desc'])
dict_data[sample_id]['desc']

2067


'TransTRACK.ID provides data collection from fleet, driver and asset, gathered through GPS technology, sensors, and vehicle engine diagnostic, transmitted through cellular network, to track all vehicles from a central location and monitor driver behavior, proper vehicle use, fuel efficiency, works breaks and safety. TransTRACK.ID provides all in onesolution from Fleet Management System, Transportation  Management System, Vehicle Camera Solution, Vehicle Maintenance System, Driver Management System and Asset Management System to Fleet Consulting Services to provide tailored solutions across the fleet lifecycle to reduce costs, improve efficiency, enhance sustainability, and upgrade fleet policies to make the most effective use of fleet operation and maximize productivity.Read More'

In [ ]:
" ".join([tokenizer.decode(token) + get_super(str(i)) for i, token in enumerate(encoded_input["input_ids"])])

'[CLS]⁰ Trans¹ ##TR² ##AC³ ##K⁴ .⁵ ID⁶ provides⁷ data⁸ collection⁹ from¹⁰ fleet¹¹ ,¹² driver¹³ and¹⁴ asset¹⁵ ,¹⁶ gathered¹⁷ through¹⁸ GPS¹⁹ technology²⁰ ,²¹ sensors²² ,²³ and²⁴ vehicle²⁵ engine²⁶ diagnostic²⁷ ,²⁸ transmitted²⁹ through³⁰ cellular³¹ network³² ,³³ to³⁴ track³⁵ all³⁶ vehicles³⁷ from³⁸ a³⁹ central⁴⁰ location⁴¹ and⁴² monitor⁴³ driver⁴⁴ behavior⁴⁵ ,⁴⁶ proper⁴⁷ vehicle⁴⁸ use⁴⁹ ,⁵⁰ fuel⁵¹ efficiency⁵² ,⁵³ works⁵⁴ breaks⁵⁵ and⁵⁶ safety⁵⁷ .⁵⁸ Trans⁵⁹ ##TR⁶⁰ ##AC⁶¹ ##K⁶² .⁶³ ID⁶⁴ provides⁶⁵ all⁶⁶ in⁶⁷ ones⁶⁸ ##ol⁶⁹ ##ution⁷⁰ from⁷¹ Fleet⁷² Management⁷³ System⁷⁴ ,⁷⁵ Transportation⁷⁶ Management⁷⁷ System⁷⁸ ,⁷⁹ Vehicle⁸⁰ Camera⁸¹ Sol⁸² ##ution⁸³ ,⁸⁴ Vehicle⁸⁵ Maintenance⁸⁶ System⁸⁷ ,⁸⁸ Driver⁸⁹ Management⁹⁰ System⁹¹ and⁹² As⁹³ ##set⁹⁴ Management⁹⁵ System⁹⁶ to⁹⁷ Fleet⁹⁸ Consul⁹⁹ ##ting¹⁰⁰ Services¹⁰¹ to¹⁰² provide¹⁰³ tailored¹⁰⁴ solutions¹⁰⁵ across¹⁰⁶ the¹⁰⁷ fleet¹⁰⁸ life¹⁰⁹ ##cycle¹¹⁰ to¹¹¹ reduce¹¹² costs¹¹³ ,¹¹⁴ improve¹¹⁵ efficiency¹¹⁶ ,¹¹⁷ enhance¹¹⁸ sustainability¹¹⁹ ,¹²⁰ and¹²¹ 

In [ ]:
#NER
start_ner_company = 1
end_ner_company = 6 + 1

#Q1
start_answer = 1
end_answer = 58 + 1

print(tokenizer.decode(encoded_input["input_ids"][start_ner_company:end_ner_company]))
print(tokenizer.decode(encoded_input["input_ids"][start_answer:end_answer]))

TransTRACK. ID
TransTRACK. ID provides data collection from fleet, driver and asset, gathered through GPS technology, sensors, and vehicle engine diagnostic, transmitted through cellular network, to track all vehicles from a central location and monitor driver behavior, proper vehicle use, fuel efficiency, works breaks and safety.


In [ ]:
insert_new_labels(sample_id)

NER: {'start': 1, 'end': 7}
Answer: {'start': 1, 'end': 59}
Total Labeling Count: 17


In [ ]:
sample_id = next(data_sample_ids)
print(sample_id)
encoded_input = tokenizer(dict_data[sample_id]['desc'])
dict_data[sample_id]['desc']

570


'Clearstep is an AI care navigation platform that matches patients to the right level of care based on their symptoms, insurance, and location, ultimately helping them book the best in-person or virtual care.'

In [ ]:
" ".join([tokenizer.decode(token) + get_super(str(i)) for i, token in enumerate(encoded_input["input_ids"])])

'[CLS]⁰ Clear¹ ##ste² ##p³ is⁴ an⁵ AI⁶ care⁷ navigation⁸ platform⁹ that¹⁰ matches¹¹ patients¹² to¹³ the¹⁴ right¹⁵ level¹⁶ of¹⁷ care¹⁸ based¹⁹ on²⁰ their²¹ symptoms²² ,²³ insurance²⁴ ,²⁵ and²⁶ location²⁷ ,²⁸ ultimately²⁹ helping³⁰ them³¹ book³² the³³ best³⁴ in³⁵ -³⁶ person³⁷ or³⁸ virtual³⁹ care⁴⁰ .⁴¹ [SEP]⁴²'

In [ ]:
#NER
start_ner_company = 1
end_ner_company = 3 + 1

#Q1
start_answer = 1
end_answer = 41 + 1

print(tokenizer.decode(encoded_input["input_ids"][start_ner_company:end_ner_company]))
print(tokenizer.decode(encoded_input["input_ids"][start_answer:end_answer]))

Clearstep
Clearstep is an AI care navigation platform that matches patients to the right level of care based on their symptoms, insurance, and location, ultimately helping them book the best in - person or virtual care.


In [ ]:
insert_new_labels(sample_id)

NER: {'start': 1, 'end': 4}
Answer: {'start': 1, 'end': 42}
Total Labeling Count: 18


In [ ]:
sample_id = next(data_sample_ids)
print(sample_id)
encoded_input = tokenizer(dict_data[sample_id]['desc'])
dict_data[sample_id]['desc']

1154


'RetailNext engages in Applied Big Data for physical retail, delivering real-time analytics that enables retailers, shopping centers, and manufacturers to collect, analyze, and visualize in-store data. The patented solution uses best-in-class video analytics, Wi-Fi detection, Bluetooth, on-shelf sensors, beacons, and data from point-of-sale systemsand other sources to automatically inform retailers about how people engage with their stores. The highly scalable RetailNext platform easily integrates with promotional calendars, staffing systems, and even weather services to analyze how internal and external factors impact customer shopping patterns – providing retailers the ability to identify opportunities for growth, execute changes, and measure success.RetailNext tracks more than one billion shoppers per year by collecting data from more than 65,000 sensors in retail stores and analyzing trillions of data points annually. Headquartered in San Jose, California, RetailNext is a growing g

In [ ]:
" ".join([tokenizer.decode(token) + get_super(str(i)) for i, token in enumerate(encoded_input["input_ids"])])

'[CLS]⁰ Re¹ ##tail² ##N³ ##ex⁴ ##t⁵ engages⁶ in⁷ Applied⁸ Big⁹ Data¹⁰ for¹¹ physical¹² retail¹³ ,¹⁴ delivering¹⁵ real¹⁶ -¹⁷ time¹⁸ anal¹⁹ ##ytic²⁰ ##s²¹ that²² enables²³ retailers²⁴ ,²⁵ shopping²⁶ centers²⁷ ,²⁸ and²⁹ manufacturers³⁰ to³¹ collect³² ,³³ analyze³⁴ ,³⁵ and³⁶ visual³⁷ ##ize³⁸ in³⁹ -⁴⁰ store⁴¹ data⁴² .⁴³ The⁴⁴ patented⁴⁵ solution⁴⁶ uses⁴⁷ best⁴⁸ -⁴⁹ in⁵⁰ -⁵¹ class⁵² video⁵³ anal⁵⁴ ##ytic⁵⁵ ##s⁵⁶ ,⁵⁷ W⁵⁸ ##i⁵⁹ -⁶⁰ Fi⁶¹ detection⁶² ,⁶³ Blue⁶⁴ ##tooth⁶⁵ ,⁶⁶ on⁶⁷ -⁶⁸ shelf⁶⁹ sensors⁷⁰ ,⁷¹ beacon⁷² ##s⁷³ ,⁷⁴ and⁷⁵ data⁷⁶ from⁷⁷ point⁷⁸ -⁷⁹ of⁸⁰ -⁸¹ sale⁸² systems⁸³ ##and⁸⁴ other⁸⁵ sources⁸⁶ to⁸⁷ automatically⁸⁸ inform⁸⁹ retailers⁹⁰ about⁹¹ how⁹² people⁹³ engage⁹⁴ with⁹⁵ their⁹⁶ stores⁹⁷ .⁹⁸ The⁹⁹ highly¹⁰⁰ s¹⁰¹ ##cal¹⁰² ##able¹⁰³ Re¹⁰⁴ ##tail¹⁰⁵ ##N¹⁰⁶ ##ex¹⁰⁷ ##t¹⁰⁸ platform¹⁰⁹ easily¹¹⁰ integrate¹¹¹ ##s¹¹² with¹¹³ promotional¹¹⁴ calendar¹¹⁵ ##s¹¹⁶ ,¹¹⁷ staff¹¹⁸ ##ing¹¹⁹ systems¹²⁰ ,¹²¹ and¹²² even¹²³ weather¹²⁴ services¹²⁵ to¹²⁶ analyze¹²⁷ how¹²⁸ internal¹²⁹ and¹³⁰ external¹³¹ 

In [ ]:
#NER
start_ner_company = 1
end_ner_company = 5 + 1

#Q1
start_answer = 1
end_answer = 43 + 1

print(tokenizer.decode(encoded_input["input_ids"][start_ner_company:end_ner_company]))
print(tokenizer.decode(encoded_input["input_ids"][start_answer:end_answer]))

RetailNext
RetailNext engages in Applied Big Data for physical retail, delivering real - time analytics that enables retailers, shopping centers, and manufacturers to collect, analyze, and visualize in - store data.


In [ ]:
insert_new_labels(sample_id)

NER: {'start': 1, 'end': 6}
Answer: {'start': 1, 'end': 44}
Total Labeling Count: 19


In [ ]:
sample_id = next(data_sample_ids)
print(sample_id)
encoded_input = tokenizer(dict_data[sample_id]['desc'])
dict_data[sample_id]['desc']

572


'For years organizations have been focused on digitization and automation, stacking piles of data and improving networking and computation power; The era of Big Data has allowed us to ask more meaningful and more ambitious questions than ever before. It is now time to achieve genuine impact on business objectives with the power of artificialintelligence (AI).Razor Labs is a leader in the AI industry and is your partner on the journey to ROI from AI. We work day and night to empower enterprises to reap the benefits arising out of the AI revolution. Deep Learning technologies are in the heart of everything we do, transforming seas of data into insights that support your corporate objectives.Razor Labs is a trusted AI partner of MNCs companies, creating visionary breakthroughs together with world leaders in numerous fields, using the Israeli innovation and commando spirit. Together we will exceed human-level accuracy. This is the kind of promise we stand behind.Read More'

In [ ]:
" ".join([tokenizer.decode(token) + get_super(str(i)) for i, token in enumerate(encoded_input["input_ids"])])

'[CLS]⁰ For¹ years² organizations³ have⁴ been⁵ focused⁶ on⁷ digit⁸ ##ization⁹ and¹⁰ automation¹¹ ,¹² stack¹³ ##ing¹⁴ piles¹⁵ of¹⁶ data¹⁷ and¹⁸ improving¹⁹ networking²⁰ and²¹ com²² ##putation²³ power²⁴ ;²⁵ The²⁶ era²⁷ of²⁸ Big²⁹ Data³⁰ has³¹ allowed³² us³³ to³⁴ ask³⁵ more³⁶ meaningful³⁷ and³⁸ more³⁹ ambitious⁴⁰ questions⁴¹ than⁴² ever⁴³ before⁴⁴ .⁴⁵ It⁴⁶ is⁴⁷ now⁴⁸ time⁴⁹ to⁵⁰ achieve⁵¹ genuine⁵² impact⁵³ on⁵⁴ business⁵⁵ objectives⁵⁶ with⁵⁷ the⁵⁸ power⁵⁹ of⁶⁰ artificial⁶¹ ##int⁶² ##elli⁶³ ##gence⁶⁴ (⁶⁵ AI⁶⁶ )⁶⁷ .⁶⁸ Ra⁶⁹ ##zor⁷⁰ Labs⁷¹ is⁷² a⁷³ leader⁷⁴ in⁷⁵ the⁷⁶ AI⁷⁷ industry⁷⁸ and⁷⁹ is⁸⁰ your⁸¹ partner⁸² on⁸³ the⁸⁴ journey⁸⁵ to⁸⁶ R⁸⁷ ##O⁸⁸ ##I⁸⁹ from⁹⁰ AI⁹¹ .⁹² We⁹³ work⁹⁴ day⁹⁵ and⁹⁶ night⁹⁷ to⁹⁸ em⁹⁹ ##power¹⁰⁰ enterprises¹⁰¹ to¹⁰² re¹⁰³ ##ap¹⁰⁴ the¹⁰⁵ benefits¹⁰⁶ arising¹⁰⁷ out¹⁰⁸ of¹⁰⁹ the¹¹⁰ AI¹¹¹ revolution¹¹² .¹¹³ Deep¹¹⁴ Learning¹¹⁵ technologies¹¹⁶ are¹¹⁷ in¹¹⁸ the¹¹⁹ heart¹²⁰ of¹²¹ everything¹²² we¹²³ do¹²⁴ ,¹²⁵ transforming¹²⁶ seas¹²⁷ of¹²⁸ data¹²⁹ into¹³⁰ insights¹³¹ that¹³

In [ ]:
#NER
start_ner_company = 69
end_ner_company = 71 + 1

#Q1
start_answer = 69
end_answer = 92 + 1

print(tokenizer.decode(encoded_input["input_ids"][start_ner_company:end_ner_company]))
print(tokenizer.decode(encoded_input["input_ids"][start_answer:end_answer]))

Razor Labs
Razor Labs is a leader in the AI industry and is your partner on the journey to ROI from AI.


In [ ]:
insert_new_labels(sample_id)

NER: {'start': 69, 'end': 72}
Answer: {'start': 69, 'end': 93}
Total Labeling Count: 20


In [ ]:
sample_id = next(data_sample_ids)
print(sample_id)
encoded_input = tokenizer(dict_data[sample_id]['desc'])
dict_data[sample_id]['desc']

3095


'Deep, enabling technology that shifts the frontier on explainable AI and that will change the way we conceive, build and consume AI beyond the current limitations of opaque machine learning.'

In [ ]:
" ".join([tokenizer.decode(token) + get_super(str(i)) for i, token in enumerate(encoded_input["input_ids"])])

'[CLS]⁰ Deep¹ ,² enabling³ technology⁴ that⁵ shifts⁶ the⁷ frontier⁸ on⁹ explain¹⁰ ##able¹¹ AI¹² and¹³ that¹⁴ will¹⁵ change¹⁶ the¹⁷ way¹⁸ we¹⁹ con²⁰ ##ceive²¹ ,²² build²³ and²⁴ consume²⁵ AI²⁶ beyond²⁷ the²⁸ current²⁹ limitations³⁰ of³¹ op³² ##aque³³ machine³⁴ learning³⁵ .³⁶ [SEP]³⁷'

In [ ]:
#NER
start_ner_company = 1
end_ner_company = 1 + 1

#Q1
start_answer = 1
end_answer = 36 + 1

print(tokenizer.decode(encoded_input["input_ids"][start_ner_company:end_ner_company]))
print(tokenizer.decode(encoded_input["input_ids"][start_answer:end_answer]))

Deep
Deep, enabling technology that shifts the frontier on explainable AI and that will change the way we conceive, build and consume AI beyond the current limitations of opaque machine learning.


In [ ]:
insert_new_labels(sample_id)

NER: {'start': 1, 'end': 2}
Answer: {'start': 1, 'end': 37}
Total Labeling Count: 21


In [ ]:
sample_id = next(data_sample_ids)
print(sample_id)
encoded_input = tokenizer(dict_data[sample_id]['desc'])
dict_data[sample_id]['desc']

388


'Innoscentia AB is a privately owned chemical or biochemical company targeting different markets with printable sensor technologies for the detection of certain volatile food components that indicate the degradation of food. They have developed a sensing technology for meat packages that can reveal the actual status of the packaged product.Innoscentia’s  forefront technology has the potential to replace date labeling as a standard of telling how good your food is feeling.Read More'

In [ ]:
" ".join([tokenizer.decode(token) + get_super(str(i)) for i, token in enumerate(encoded_input["input_ids"])])

'[CLS]⁰ Inn¹ ##os² ##cent³ ##ia⁴ AB⁵ is⁶ a⁷ privately⁸ owned⁹ chemical¹⁰ or¹¹ bio¹² ##chemical¹³ company¹⁴ targeting¹⁵ different¹⁶ markets¹⁷ with¹⁸ print¹⁹ ##able²⁰ sensor²¹ technologies²² for²³ the²⁴ detection²⁵ of²⁶ certain²⁷ volatile²⁸ food²⁹ components³⁰ that³¹ indicate³² the³³ degradation³⁴ of³⁵ food³⁶ .³⁷ They³⁸ have³⁹ developed⁴⁰ a⁴¹ sensing⁴² technology⁴³ for⁴⁴ meat⁴⁵ packages⁴⁶ that⁴⁷ can⁴⁸ reveal⁴⁹ the⁵⁰ actual⁵¹ status⁵² of⁵³ the⁵⁴ packaged⁵⁵ product⁵⁶ .⁵⁷ Inn⁵⁸ ##os⁵⁹ ##cent⁶⁰ ##ia⁶¹ ’⁶² s⁶³ forefront⁶⁴ technology⁶⁵ has⁶⁶ the⁶⁷ potential⁶⁸ to⁶⁹ replace⁷⁰ date⁷¹ label⁷² ##ing⁷³ as⁷⁴ a⁷⁵ standard⁷⁶ of⁷⁷ telling⁷⁸ how⁷⁹ good⁸⁰ your⁸¹ food⁸² is⁸³ feeling⁸⁴ .⁸⁵ Read⁸⁶ More⁸⁷ [SEP]⁸⁸'

In [ ]:
#NER
start_ner_company = 1
end_ner_company = 5 + 1

#Q1
start_answer = 1
end_answer = 37 + 1

print(tokenizer.decode(encoded_input["input_ids"][start_ner_company:end_ner_company]))
print(tokenizer.decode(encoded_input["input_ids"][start_answer:end_answer]))

Innoscentia AB
Innoscentia AB is a privately owned chemical or biochemical company targeting different markets with printable sensor technologies for the detection of certain volatile food components that indicate the degradation of food.


In [ ]:
insert_new_labels(sample_id)

NER: {'start': 1, 'end': 6}
Answer: {'start': 1, 'end': 38}
Total Labeling Count: 22


In [ ]:
sample_id = next(data_sample_ids)
print(sample_id)
encoded_input = tokenizer(dict_data[sample_id]['desc'])
dict_data[sample_id]['desc']

2532


'Agritask is an open SaaS Agronomic Intelligence (AgI) Platform, powering solutions for key challenges faced by farmers, food companies, insurers and additional players in the agriculture ecosystem. Our solutions provide crucial agronomic insights to optimize operations across the food value chain and ensure sustainable farming and sourcingpractices.Read More'

In [ ]:
" ".join([tokenizer.decode(token) + get_super(str(i)) for i, token in enumerate(encoded_input["input_ids"])])

'[CLS]⁰ A¹ ##g² ##rita³ ##sk⁴ is⁵ an⁶ open⁷ Sa⁸ ##a⁹ ##S¹⁰ A¹¹ ##gro¹² ##no¹³ ##mic¹⁴ Intelligence¹⁵ (¹⁶ A¹⁷ ##g¹⁸ ##I¹⁹ )²⁰ Platform²¹ ,²² power²³ ##ing²⁴ solutions²⁵ for²⁶ key²⁷ challenges²⁸ faced²⁹ by³⁰ farmers³¹ ,³² food³³ companies³⁴ ,³⁵ ins³⁶ ##urers³⁷ and³⁸ additional³⁹ players⁴⁰ in⁴¹ the⁴² agriculture⁴³ ecosystem⁴⁴ .⁴⁵ Our⁴⁶ solutions⁴⁷ provide⁴⁸ crucial⁴⁹ a⁵⁰ ##gro⁵¹ ##no⁵² ##mic⁵³ insights⁵⁴ to⁵⁵ op⁵⁶ ##ti⁵⁷ ##mize⁵⁸ operations⁵⁹ across⁶⁰ the⁶¹ food⁶² value⁶³ chain⁶⁴ and⁶⁵ ensure⁶⁶ sustainable⁶⁷ farming⁶⁸ and⁶⁹ sour⁷⁰ ##cing⁷¹ ##pra⁷² ##ctic⁷³ ##es⁷⁴ .⁷⁵ Read⁷⁶ More⁷⁷ [SEP]⁷⁸'

In [ ]:
#NER
start_ner_company = 1
end_ner_company = 4 + 1

#Q1
start_answer = 1
end_answer = 45 + 1

print(tokenizer.decode(encoded_input["input_ids"][start_ner_company:end_ner_company]))
print(tokenizer.decode(encoded_input["input_ids"][start_answer:end_answer]))

Agritask
Agritask is an open SaaS Agronomic Intelligence ( AgI ) Platform, powering solutions for key challenges faced by farmers, food companies, insurers and additional players in the agriculture ecosystem.


In [ ]:
insert_new_labels(sample_id)

NER: {'start': 1, 'end': 5}
Answer: {'start': 1, 'end': 46}
Total Labeling Count: 23


In [ ]:
sample_id = next(data_sample_ids)
print(sample_id)
encoded_input = tokenizer(dict_data[sample_id]['desc'])
dict_data[sample_id]['desc']

3274


'Global Data Consortium is building a process for global digital identity verification for businesses. For businesses that need to know the identities of customers all over the world, it provide one point of access to local, in-country reference data for instant verification.It mission to empower businesses with the tools to get to know theircustomers, stay compliant, and deliver real-time solutions. Its Worldview platform facilitates the relationship between local and global markets by pairing businesses to locally sourced data from all over the world.Read More'

In [ ]:
" ".join([tokenizer.decode(token) + get_super(str(i)) for i, token in enumerate(encoded_input["input_ids"])])

'[CLS]⁰ Global¹ Data² Consortium³ is⁴ building⁵ a⁶ process⁷ for⁸ global⁹ digital¹⁰ identity¹¹ ve¹² ##rification¹³ for¹⁴ businesses¹⁵ .¹⁶ For¹⁷ businesses¹⁸ that¹⁹ need²⁰ to²¹ know²² the²³ identities²⁴ of²⁵ customers²⁶ all²⁷ over²⁸ the²⁹ world³⁰ ,³¹ it³² provide³³ one³⁴ point³⁵ of³⁶ access³⁷ to³⁸ local³⁹ ,⁴⁰ in⁴¹ -⁴² country⁴³ reference⁴⁴ data⁴⁵ for⁴⁶ instant⁴⁷ ve⁴⁸ ##rification⁴⁹ .⁵⁰ It⁵¹ mission⁵² to⁵³ em⁵⁴ ##power⁵⁵ businesses⁵⁶ with⁵⁷ the⁵⁸ tools⁵⁹ to⁶⁰ get⁶¹ to⁶² know⁶³ their⁶⁴ ##cus⁶⁵ ##tom⁶⁶ ##ers⁶⁷ ,⁶⁸ stay⁶⁹ com⁷⁰ ##pliant⁷¹ ,⁷² and⁷³ deliver⁷⁴ real⁷⁵ -⁷⁶ time⁷⁷ solutions⁷⁸ .⁷⁹ Its⁸⁰ World⁸¹ ##view⁸² platform⁸³ facilitate⁸⁴ ##s⁸⁵ the⁸⁶ relationship⁸⁷ between⁸⁸ local⁸⁹ and⁹⁰ global⁹¹ markets⁹² by⁹³ pairing⁹⁴ businesses⁹⁵ to⁹⁶ locally⁹⁷ sourced⁹⁸ data⁹⁹ from¹⁰⁰ all¹⁰¹ over¹⁰² the¹⁰³ world¹⁰⁴ .¹⁰⁵ Read¹⁰⁶ More¹⁰⁷ [SEP]¹⁰⁸'

In [ ]:
#NER
start_ner_company = 1
end_ner_company = 3 + 1

#Q1
start_answer = 1
end_answer = 16 + 1

print(tokenizer.decode(encoded_input["input_ids"][start_ner_company:end_ner_company]))
print(tokenizer.decode(encoded_input["input_ids"][start_answer:end_answer]))

Global Data Consortium
Global Data Consortium is building a process for global digital identity verification for businesses.


In [ ]:
insert_new_labels(sample_id)

NER: {'start': 1, 'end': 4}
Answer: {'start': 1, 'end': 17}
Total Labeling Count: 24


In [ ]:
sample_id = next(data_sample_ids)
print(sample_id)
encoded_input = tokenizer(dict_data[sample_id]['desc'])
dict_data[sample_id]['desc']

1026


'HCI mission Provision of technology, expertise, and management skills that ease the process of providing the best patient care and increase economic security. HCI provides financing alternatives in order to make Diagnostic Imaging a viable and effortless.'

In [ ]:
" ".join([tokenizer.decode(token) + get_super(str(i)) for i, token in enumerate(encoded_input["input_ids"])])

'[CLS]⁰ HC¹ ##I² mission³ Pro⁴ ##vision⁵ of⁶ technology⁷ ,⁸ expertise⁹ ,¹⁰ and¹¹ management¹² skills¹³ that¹⁴ ease¹⁵ the¹⁶ process¹⁷ of¹⁸ providing¹⁹ the²⁰ best²¹ patient²² care²³ and²⁴ increase²⁵ economic²⁶ security²⁷ .²⁸ HC²⁹ ##I³⁰ provides³¹ financing³² alternatives³³ in³⁴ order³⁵ to³⁶ make³⁷ Di³⁸ ##ag³⁹ ##nos⁴⁰ ##tic⁴¹ I⁴² ##maging⁴³ a⁴⁴ viable⁴⁵ and⁴⁶ effort⁴⁷ ##less⁴⁸ .⁴⁹ [SEP]⁵⁰'

In [ ]:
#NER
start_ner_company = 1
end_ner_company = 2 + 1

#Q1
start_answer = 1
end_answer = 28 + 1

print(tokenizer.decode(encoded_input["input_ids"][start_ner_company:end_ner_company]))
print(tokenizer.decode(encoded_input["input_ids"][start_answer:end_answer]))

HCI
HCI mission Provision of technology, expertise, and management skills that ease the process of providing the best patient care and increase economic security.


In [ ]:
insert_new_labels(sample_id)

NER: {'start': 1, 'end': 3}
Answer: {'start': 1, 'end': 29}
Total Labeling Count: 25


In [ ]:
sample_id = next(data_sample_ids)
print(sample_id)
encoded_input = tokenizer(dict_data[sample_id]['desc'])
dict_data[sample_id]['desc']

2181


"MParticle is a customer data platform for brands to accelerate growth. With MParticle, brands can unify data and simplify partner integrations with enterprise-class security and reliability. It improves marketing, advertising, and analytics by connecting data seamlessly across channels.Sophisticated marketers at companies like Walmart, Starwood,NBC Universal, Spotify, and Airbnb use MParticle to integrate and orchestrate their entire growth stack, allowing them to win in key moments of the customers' journey. It was recognized as one of Crain’s 100 Best Places to Work in New York City and named a “Vendor to Watch” in Gartner’s 2017 Digital Marketing Hubs Magic Quadrant report.MParticle is headquartered in New York City with offices in San Francisco, Seattle, Florida, and London.Read More"

In [ ]:
" ".join([tokenizer.decode(token) + get_super(str(i)) for i, token in enumerate(encoded_input["input_ids"])])

"[CLS]⁰ MP¹ ##art² ##icle³ is⁴ a⁵ customer⁶ data⁷ platform⁸ for⁹ brands¹⁰ to¹¹ accelerate¹² growth¹³ .¹⁴ With¹⁵ MP¹⁶ ##art¹⁷ ##icle¹⁸ ,¹⁹ brands²⁰ can²¹ un²² ##ify²³ data²⁴ and²⁵ si²⁶ ##mp²⁷ ##lify²⁸ partner²⁹ integration³⁰ ##s³¹ with³² enterprise³³ -³⁴ class³⁵ security³⁶ and³⁷ reliability³⁸ .³⁹ It⁴⁰ improve⁴¹ ##s⁴² marketing⁴³ ,⁴⁴ advertising⁴⁵ ,⁴⁶ and⁴⁷ anal⁴⁸ ##ytic⁴⁹ ##s⁵⁰ by⁵¹ connecting⁵² data⁵³ sea⁵⁴ ##m⁵⁵ ##lessly⁵⁶ across⁵⁷ channels⁵⁸ .⁵⁹ So⁶⁰ ##phi⁶¹ ##stic⁶² ##ated⁶³ market⁶⁴ ##ers⁶⁵ at⁶⁶ companies⁶⁷ like⁶⁸ W⁶⁹ ##al⁷⁰ ##mart⁷¹ ,⁷² Star⁷³ ##wood⁷⁴ ,⁷⁵ NBC⁷⁶ Universal⁷⁷ ,⁷⁸ Spot⁷⁹ ##ify⁸⁰ ,⁸¹ and⁸² Air⁸³ ##b⁸⁴ ##n⁸⁵ ##b⁸⁶ use⁸⁷ MP⁸⁸ ##art⁸⁹ ##icle⁹⁰ to⁹¹ integrate⁹² and⁹³ orchestra⁹⁴ ##te⁹⁵ their⁹⁶ entire⁹⁷ growth⁹⁸ stack⁹⁹ ,¹⁰⁰ allowing¹⁰¹ them¹⁰² to¹⁰³ win¹⁰⁴ in¹⁰⁵ key¹⁰⁶ moments¹⁰⁷ of¹⁰⁸ the¹⁰⁹ customers¹¹⁰ '¹¹¹ journey¹¹² .¹¹³ It¹¹⁴ was¹¹⁵ recognized¹¹⁶ as¹¹⁷ one¹¹⁸ of¹¹⁹ C¹²⁰ ##rain¹²¹ ’¹²² s¹²³ 100¹²⁴ Best¹²⁵ Places¹²⁶ to¹²⁷ Work¹²⁸ in¹²⁹ New¹³⁰ York¹³¹ City¹³² and¹³³ na

In [ ]:
#NER
start_ner_company = 1
end_ner_company = 3 + 1

#Q1
start_answer = 1
end_answer = 14 + 1

print(tokenizer.decode(encoded_input["input_ids"][start_ner_company:end_ner_company]))
print(tokenizer.decode(encoded_input["input_ids"][start_answer:end_answer]))

MParticle
MParticle is a customer data platform for brands to accelerate growth.


In [ ]:
insert_new_labels(sample_id)

NER: {'start': 1, 'end': 4}
Answer: {'start': 1, 'end': 15}
Total Labeling Count: 26


In [ ]:
sample_id = next(data_sample_ids)
print(sample_id)
encoded_input = tokenizer(dict_data[sample_id]['desc'])
dict_data[sample_id]['desc']

2888


'Kogniz leverages AI and computer vision to deliver the leading proactive safety platform for improving workplace safety and security. Its flagship platform, Kogniz AssureAI, can be implemented in days and integrates with the existing infrastructure already in use. More than three million employees rely on Kogniz for safer, smarter spaces withsolutions for occupational hazards, workplace violence, evacuations, personnel accountability, anomalous behavior detection, and vaccine management. Kogniz’ award-winning technology has been featured in The Wall Street Journal, The Washington Post, NBC and Fox. The company is headquartered in Berkeley, California with a development team in Montreal, Quebec. Learn more at www.kogniz.com.Read More'

In [ ]:
" ".join([tokenizer.decode(token) + get_super(str(i)) for i, token in enumerate(encoded_input["input_ids"])])

'[CLS]⁰ Ko¹ ##gni² ##z³ leverage⁴ ##s⁵ AI⁶ and⁷ computer⁸ vision⁹ to¹⁰ deliver¹¹ the¹² leading¹³ pro¹⁴ ##active¹⁵ safety¹⁶ platform¹⁷ for¹⁸ improving¹⁹ workplace²⁰ safety²¹ and²² security²³ .²⁴ Its²⁵ flagship²⁶ platform²⁷ ,²⁸ Ko²⁹ ##gni³⁰ ##z³¹ As³² ##sure³³ ##A³⁴ ##I³⁵ ,³⁶ can³⁷ be³⁸ implemented³⁹ in⁴⁰ days⁴¹ and⁴² integrate⁴³ ##s⁴⁴ with⁴⁵ the⁴⁶ existing⁴⁷ infrastructure⁴⁸ already⁴⁹ in⁵⁰ use⁵¹ .⁵² More⁵³ than⁵⁴ three⁵⁵ million⁵⁶ employees⁵⁷ rely⁵⁸ on⁵⁹ Ko⁶⁰ ##gni⁶¹ ##z⁶² for⁶³ safer⁶⁴ ,⁶⁵ smart⁶⁶ ##er⁶⁷ spaces⁶⁸ with⁶⁹ ##sol⁷⁰ ##ution⁷¹ ##s⁷² for⁷³ occupational⁷⁴ hazards⁷⁵ ,⁷⁶ workplace⁷⁷ violence⁷⁸ ,⁷⁹ evacuation⁸⁰ ##s⁸¹ ,⁸² personnel⁸³ account⁸⁴ ##ability⁸⁵ ,⁸⁶ an⁸⁷ ##oma⁸⁸ ##lous⁸⁹ behavior⁹⁰ detection⁹¹ ,⁹² and⁹³ vaccine⁹⁴ management⁹⁵ .⁹⁶ Ko⁹⁷ ##gni⁹⁸ ##z⁹⁹ ’¹⁰⁰ award¹⁰¹ -¹⁰² winning¹⁰³ technology¹⁰⁴ has¹⁰⁵ been¹⁰⁶ featured¹⁰⁷ in¹⁰⁸ The¹⁰⁹ Wall¹¹⁰ Street¹¹¹ Journal¹¹² ,¹¹³ The¹¹⁴ Washington¹¹⁵ Post¹¹⁶ ,¹¹⁷ NBC¹¹⁸ and¹¹⁹ Fox¹²⁰ .¹²¹ The¹²² company¹²³ is¹²⁴ headquartered¹²⁵ in¹²⁶ B

In [ ]:
#NER
start_ner_company = 1
end_ner_company = 3 + 1

#Q1
start_answer = 1
end_answer = 24 + 1

print(tokenizer.decode(encoded_input["input_ids"][start_ner_company:end_ner_company]))
print(tokenizer.decode(encoded_input["input_ids"][start_answer:end_answer]))

Kogniz
Kogniz leverages AI and computer vision to deliver the leading proactive safety platform for improving workplace safety and security.


In [ ]:
insert_new_labels(sample_id)

NER: {'start': 1, 'end': 4}
Answer: {'start': 1, 'end': 25}
Total Labeling Count: 27


In [ ]:
sample_id = next(data_sample_ids)
print(sample_id)
encoded_input = tokenizer(dict_data[sample_id]['desc'])
dict_data[sample_id]['desc']

3318


"Source Defense provides websites with prevention technology for attacks of third-party origin.In an industry focused on detection, Source Defense provides websites with the first ever prevention technology for attacks of third-party origin. We are the first ever SAAS platform that allows a site owner to set and enforce permissions, receivereal-time alerts and monitor third-party vendors' behavior on their site. By removing the security considerations from third-party integrations, we can save countless man-hours spent on tests and integrations, allowing sites to focus on generating revenues and new opportunities while keeping the site visitors safe and the site in high performance.Read More"

In [ ]:
" ".join([tokenizer.decode(token) + get_super(str(i)) for i, token in enumerate(encoded_input["input_ids"])])

"[CLS]⁰ Source¹ Defense² provides³ websites⁴ with⁵ prevention⁶ technology⁷ for⁸ attacks⁹ of¹⁰ third¹¹ -¹² party¹³ origin¹⁴ .¹⁵ In¹⁶ an¹⁷ industry¹⁸ focused¹⁹ on²⁰ detection²¹ ,²² Source²³ Defense²⁴ provides²⁵ websites²⁶ with²⁷ the²⁸ first²⁹ ever³⁰ prevention³¹ technology³² for³³ attacks³⁴ of³⁵ third³⁶ -³⁷ party³⁸ origin³⁹ .⁴⁰ We⁴¹ are⁴² the⁴³ first⁴⁴ ever⁴⁵ SA⁴⁶ ##AS⁴⁷ platform⁴⁸ that⁴⁹ allows⁵⁰ a⁵¹ site⁵² owner⁵³ to⁵⁴ set⁵⁵ and⁵⁶ enforce⁵⁷ permission⁵⁸ ##s⁵⁹ ,⁶⁰ receiver⁶¹ ##eal⁶² -⁶³ time⁶⁴ alert⁶⁵ ##s⁶⁶ and⁶⁷ monitor⁶⁸ third⁶⁹ -⁷⁰ party⁷¹ vendors⁷² '⁷³ behavior⁷⁴ on⁷⁵ their⁷⁶ site⁷⁷ .⁷⁸ By⁷⁹ removing⁸⁰ the⁸¹ security⁸² considerations⁸³ from⁸⁴ third⁸⁵ -⁸⁶ party⁸⁷ integration⁸⁸ ##s⁸⁹ ,⁹⁰ we⁹¹ can⁹² save⁹³ countless⁹⁴ man⁹⁵ -⁹⁶ hours⁹⁷ spent⁹⁸ on⁹⁹ tests¹⁰⁰ and¹⁰¹ integration¹⁰² ##s¹⁰³ ,¹⁰⁴ allowing¹⁰⁵ sites¹⁰⁶ to¹⁰⁷ focus¹⁰⁸ on¹⁰⁹ generating¹¹⁰ revenues¹¹¹ and¹¹² new¹¹³ opportunities¹¹⁴ while¹¹⁵ keeping¹¹⁶ the¹¹⁷ site¹¹⁸ visitors¹¹⁹ safe¹²⁰ and¹²¹ the¹²² site¹²³ in¹²⁴ high¹²⁵ performa

In [ ]:
#NER
start_ner_company = 1
end_ner_company = 2 + 1

#Q1
start_answer = 1
end_answer = 15 + 1

print(tokenizer.decode(encoded_input["input_ids"][start_ner_company:end_ner_company]))
print(tokenizer.decode(encoded_input["input_ids"][start_answer:end_answer]))

Source Defense
Source Defense provides websites with prevention technology for attacks of third - party origin.


In [ ]:
insert_new_labels(sample_id)

NER: {'start': 1, 'end': 3}
Answer: {'start': 1, 'end': 16}
Total Labeling Count: 28


In [ ]:
sample_id = next(data_sample_ids)
print(sample_id)
encoded_input = tokenizer(dict_data[sample_id]['desc'])
dict_data[sample_id]['desc']

2465


'3DR makes advanced drone technology easily accessible to the construction, mining, and surveying industries for use in data analysis. It helps users in land mapping and site scanning services in collecting accurate aerial data of the Jobsite, turning urns the drone photos into maps, models, point clouds, and providing comprehensive onboarding andongoing support, enabling clients to take the batter and informed decisions related to construction sites.It was founded in 2009 and headquartered in Berkeley, California.Read More'

In [ ]:
" ".join([tokenizer.decode(token) + get_super(str(i)) for i, token in enumerate(encoded_input["input_ids"])])

'[CLS]⁰ 3D¹ ##R² makes³ advanced⁴ drone⁵ technology⁶ easily⁷ accessible⁸ to⁹ the¹⁰ construction¹¹ ,¹² mining¹³ ,¹⁴ and¹⁵ surveying¹⁶ industries¹⁷ for¹⁸ use¹⁹ in²⁰ data²¹ analysis²² .²³ It²⁴ helps²⁵ users²⁶ in²⁷ land²⁸ mapping²⁹ and³⁰ site³¹ scanning³² services³³ in³⁴ collecting³⁵ accurate³⁶ aerial³⁷ data³⁸ of³⁹ the⁴⁰ Job⁴¹ ##site⁴² ,⁴³ turning⁴⁴ u⁴⁵ ##rns⁴⁶ the⁴⁷ drone⁴⁸ photos⁴⁹ into⁵⁰ maps⁵¹ ,⁵² models⁵³ ,⁵⁴ point⁵⁵ clouds⁵⁶ ,⁵⁷ and⁵⁸ providing⁵⁹ comprehensive⁶⁰ on⁶¹ ##boarding⁶² and⁶³ ##ong⁶⁴ ##oing⁶⁵ support⁶⁶ ,⁶⁷ enabling⁶⁸ clients⁶⁹ to⁷⁰ take⁷¹ the⁷² batter⁷³ and⁷⁴ informed⁷⁵ decisions⁷⁶ related⁷⁷ to⁷⁸ construction⁷⁹ sites⁸⁰ .⁸¹ It⁸² was⁸³ founded⁸⁴ in⁸⁵ 2009⁸⁶ and⁸⁷ headquartered⁸⁸ in⁸⁹ Berkeley⁹⁰ ,⁹¹ California⁹² .⁹³ Read⁹⁴ More⁹⁵ [SEP]⁹⁶'

In [ ]:
#NER
start_ner_company = 1
end_ner_company = 2 + 1

#Q1
start_answer = 1
end_answer = 23 + 1

print(tokenizer.decode(encoded_input["input_ids"][start_ner_company:end_ner_company]))
print(tokenizer.decode(encoded_input["input_ids"][start_answer:end_answer]))

3DR
3DR makes advanced drone technology easily accessible to the construction, mining, and surveying industries for use in data analysis.


In [ ]:
insert_new_labels(sample_id)

NER: {'start': 1, 'end': 3}
Answer: {'start': 1, 'end': 24}
Total Labeling Count: 29


In [ ]:
sample_id = next(data_sample_ids)
print(sample_id)
encoded_input = tokenizer(dict_data[sample_id]['desc'])
dict_data[sample_id]['desc']

601


"IOMED Medical Solutions develops a medical record management software aimed at structuring clinical data and the generation of artificial intelligence tools.A technology, based on Natural Language Processing, that allows structure clinical text, obtaining reusable data for the development of medical predictive models. It saves costs andeffectively assigns resources by extracting insights from a hospital's clinical notes and data management. The company's software uses artificial intelligence and cognitive computing to organize and visualize medical records information and offers data structuring, integration, and interoperability, providing clinicians with efficiency, information, and savings needed to make more efficient decisions.Alvaro Abella, Gabriel de Maeztu, and Javier Oca founded the company in Barcelona, Catalonia in 2016.Read More"

In [ ]:
" ".join([tokenizer.decode(token) + get_super(str(i)) for i, token in enumerate(encoded_input["input_ids"])])

"[CLS]⁰ I¹ ##OM² ##ED³ Medical⁴ Solutions⁵ develops⁶ a⁷ medical⁸ record⁹ management¹⁰ software¹¹ aimed¹² at¹³ s¹⁴ ##truct¹⁵ ##uring¹⁶ clinical¹⁷ data¹⁸ and¹⁹ the²⁰ generation²¹ of²² artificial²³ intelligence²⁴ tools²⁵ .²⁶ A²⁷ technology²⁸ ,²⁹ based³⁰ on³¹ Natural³² Language³³ Process³⁴ ##ing³⁵ ,³⁶ that³⁷ allows³⁸ structure³⁹ clinical⁴⁰ text⁴¹ ,⁴² obtaining⁴³ re⁴⁴ ##usa⁴⁵ ##ble⁴⁶ data⁴⁷ for⁴⁸ the⁴⁹ development⁵⁰ of⁵¹ medical⁵² predict⁵³ ##ive⁵⁴ models⁵⁵ .⁵⁶ It⁵⁷ saves⁵⁸ costs⁵⁹ and⁶⁰ ##ef⁶¹ ##fect⁶² ##ively⁶³ assigns⁶⁴ resources⁶⁵ by⁶⁶ extract⁶⁷ ##ing⁶⁸ insights⁶⁹ from⁷⁰ a⁷¹ hospital⁷² '⁷³ s⁷⁴ clinical⁷⁵ notes⁷⁶ and⁷⁷ data⁷⁸ management⁷⁹ .⁸⁰ The⁸¹ company⁸² '⁸³ s⁸⁴ software⁸⁵ uses⁸⁶ artificial⁸⁷ intelligence⁸⁸ and⁸⁹ cognitive⁹⁰ computing⁹¹ to⁹² organize⁹³ and⁹⁴ visual⁹⁵ ##ize⁹⁶ medical⁹⁷ records⁹⁸ information⁹⁹ and¹⁰⁰ offers¹⁰¹ data¹⁰² s¹⁰³ ##truct¹⁰⁴ ##uring¹⁰⁵ ,¹⁰⁶ integration¹⁰⁷ ,¹⁰⁸ and¹⁰⁹ inter¹¹⁰ ##oper¹¹¹ ##ability¹¹² ,¹¹³ providing¹¹⁴ clinic¹¹⁵ ##ians¹¹⁶ with¹¹⁷ efficiency¹¹⁸ ,¹

In [ ]:
#NER
start_ner_company = 1
end_ner_company = 5 + 1

#Q1
start_answer = 1
end_answer = 26 + 1

print(tokenizer.decode(encoded_input["input_ids"][start_ner_company:end_ner_company]))
print(tokenizer.decode(encoded_input["input_ids"][start_answer:end_answer]))

IOMED Medical Solutions
IOMED Medical Solutions develops a medical record management software aimed at structuring clinical data and the generation of artificial intelligence tools.


In [ ]:
insert_new_labels(sample_id)

NER: {'start': 1, 'end': 6}
Answer: {'start': 1, 'end': 27}
Total Labeling Count: 30


In [ ]:
sample_id = next(data_sample_ids)
print(sample_id)
encoded_input = tokenizer(dict_data[sample_id]['desc'])
dict_data[sample_id]['desc']

1270


"Founded in 2000, and headquartered in San Jose, California, Validity is the world leader in Natural ID™ authentication, providing fingerprint sensor solutions with the highest levels of performance, security, cost-effectiveness, and design flexibility.Validity’s Natural ID solutions provide an ideal platform for addressing the explosive growth inmobile payment transactions and cloud-based services. Passwords and PINs are ineffective at meeting the needs of end users and service providers as they're either too easy to hack or too hard to remember. Natural ID provides a simple and effective way to optimize usability while providing strong security.Natural ID authenticates a user by their unique human characteristics such as fingerprint, voice, and face. Of these, fingerprints have been proven to provide the best combination of effectiveness, cost, and usability.Read More"

In [ ]:
" ".join([tokenizer.decode(token) + get_super(str(i)) for i, token in enumerate(encoded_input["input_ids"])])

"[CLS]⁰ Founded¹ in² 2000³ ,⁴ and⁵ headquartered⁶ in⁷ San⁸ Jose⁹ ,¹⁰ California¹¹ ,¹² Val¹³ ##id¹⁴ ##ity¹⁵ is¹⁶ the¹⁷ world¹⁸ leader¹⁹ in²⁰ Natural²¹ [UNK]²² authentic²³ ##ation²⁴ ,²⁵ providing²⁶ finger²⁷ ##print²⁸ sensor²⁹ solutions³⁰ with³¹ the³² highest³³ levels³⁴ of³⁵ performance³⁶ ,³⁷ security³⁸ ,³⁹ cost⁴⁰ -⁴¹ effectiveness⁴² ,⁴³ and⁴⁴ design⁴⁵ flexibility⁴⁶ .⁴⁷ Val⁴⁸ ##id⁴⁹ ##ity⁵⁰ ’⁵¹ s⁵² Natural⁵³ ID⁵⁴ solutions⁵⁵ provide⁵⁶ an⁵⁷ ideal⁵⁸ platform⁵⁹ for⁶⁰ addressing⁶¹ the⁶² explosive⁶³ growth⁶⁴ in⁶⁵ ##mobile⁶⁶ payment⁶⁷ transactions⁶⁸ and⁶⁹ cloud⁷⁰ -⁷¹ based⁷² services⁷³ .⁷⁴ Pass⁷⁵ ##words⁷⁶ and⁷⁷ P⁷⁸ ##IN⁷⁹ ##s⁸⁰ are⁸¹ ineffective⁸² at⁸³ meeting⁸⁴ the⁸⁵ needs⁸⁶ of⁸⁷ end⁸⁸ users⁸⁹ and⁹⁰ service⁹¹ providers⁹² as⁹³ they⁹⁴ '⁹⁵ re⁹⁶ either⁹⁷ too⁹⁸ easy⁹⁹ to¹⁰⁰ ha¹⁰¹ ##ck¹⁰² or¹⁰³ too¹⁰⁴ hard¹⁰⁵ to¹⁰⁶ remember¹⁰⁷ .¹⁰⁸ Natural¹⁰⁹ ID¹¹⁰ provides¹¹¹ a¹¹² simple¹¹³ and¹¹⁴ effective¹¹⁵ way¹¹⁶ to¹¹⁷ op¹¹⁸ ##ti¹¹⁹ ##mize¹²⁰ us¹²¹ ##ability¹²² while¹²³ providing¹²⁴ strong¹²⁵ security¹²⁶ .¹²⁷ 

In [ ]:
#NER
start_ner_company = 13
end_ner_company = 15 + 1

#Q1
start_answer = 13
end_answer = 25 + 1

print(tokenizer.decode(encoded_input["input_ids"][start_ner_company:end_ner_company]))
print(tokenizer.decode(encoded_input["input_ids"][start_answer:end_answer]))

Validity
Validity is the world leader in Natural [UNK] authentication,


In [ ]:
insert_new_labels(sample_id)

NER: {'start': 13, 'end': 16}
Answer: {'start': 13, 'end': 26}
Total Labeling Count: 31


In [ ]:
sample_id = next(data_sample_ids)
print(sample_id)
encoded_input = tokenizer(dict_data[sample_id]['desc'])
dict_data[sample_id]['desc']

404


'Hubs, formerly 3D Hubs, is an online platform for custom part manufacturing, providing access to a global network of manufacturing services, including CNC machining, 3D printing, injection molding and sheet metal fabrication.\n\nFounded in Amsterdam in 2013, Hubs was acquired by Protolabs in January 2021 and has produced 7 million parts.'

In [ ]:
" ".join([tokenizer.decode(token) + get_super(str(i)) for i, token in enumerate(encoded_input["input_ids"])])

'[CLS]⁰ Hu¹ ##bs² ,³ formerly⁴ 3D⁵ Hu⁶ ##bs⁷ ,⁸ is⁹ an¹⁰ online¹¹ platform¹² for¹³ custom¹⁴ part¹⁵ manufacturing¹⁶ ,¹⁷ providing¹⁸ access¹⁹ to²⁰ a²¹ global²² network²³ of²⁴ manufacturing²⁵ services²⁶ ,²⁷ including²⁸ C²⁹ ##NC³⁰ mac³¹ ##hin³² ##ing³³ ,³⁴ 3D³⁵ printing³⁶ ,³⁷ injection³⁸ mold³⁹ ##ing⁴⁰ and⁴¹ sheet⁴² metal⁴³ fabric⁴⁴ ##ation⁴⁵ .⁴⁶ Founded⁴⁷ in⁴⁸ Amsterdam⁴⁹ in⁵⁰ 2013⁵¹ ,⁵² Hu⁵³ ##bs⁵⁴ was⁵⁵ acquired⁵⁶ by⁵⁷ Proto⁵⁸ ##la⁵⁹ ##bs⁶⁰ in⁶¹ January⁶² 202⁶³ ##1⁶⁴ and⁶⁵ has⁶⁶ produced⁶⁷ 7⁶⁸ million⁶⁹ parts⁷⁰ .⁷¹ [SEP]⁷²'

In [ ]:
#NER
start_ner_company = 1
end_ner_company = 2 + 1

#Q1
start_answer = 1
end_answer = 46 + 1

print(tokenizer.decode(encoded_input["input_ids"][start_ner_company:end_ner_company]))
print(tokenizer.decode(encoded_input["input_ids"][start_answer:end_answer]))

Hubs
Hubs, formerly 3D Hubs, is an online platform for custom part manufacturing, providing access to a global network of manufacturing services, including CNC machining, 3D printing, injection molding and sheet metal fabrication.


In [ ]:
insert_new_labels(sample_id)

NER: {'start': 1, 'end': 3}
Answer: {'start': 1, 'end': 47}
Total Labeling Count: 32


In [ ]:
sample_id = next(data_sample_ids)
print(sample_id)
encoded_input = tokenizer(dict_data[sample_id]['desc'])
dict_data[sample_id]['desc']

2989


"Napier's intelligent compliance platform, trusted by the world’s leading data providers, transforms financial crime compliance from legal obligation to competitive edge.\u202f Napier are compliance technology specialists who help financial institutions to fight financial crime more efficiently and effectively.\u202f Founded on broad experience anddeep expertise, Napier’s cutting-edge platform increases efficiency and minimises risk by successfully combining big data technologies with AI. Our intelligent approach to financial crime compliance underpins organisational policy, process and procedure, so financial institutions can focus on being more effective at detecting suspicious financial activity in a more cost-efficient way.\u202f The Napier platform is fast, scalable, and easily configurable. It rapidly strengthens financial crime defences while meeting compliance obligations and challenges in any sector.Read More"

In [ ]:
" ".join([tokenizer.decode(token) + get_super(str(i)) for i, token in enumerate(encoded_input["input_ids"])])

"[CLS]⁰ Napier¹ '² s³ intelligent⁴ compliance⁵ platform⁶ ,⁷ trusted⁸ by⁹ the¹⁰ world¹¹ ’¹² s¹³ leading¹⁴ data¹⁵ providers¹⁶ ,¹⁷ transforms¹⁸ financial¹⁹ crime²⁰ compliance²¹ from²² legal²³ obligation²⁴ to²⁵ competitive²⁶ edge²⁷ .²⁸ Napier²⁹ are³⁰ compliance³¹ technology³² specialists³³ who³⁴ help³⁵ financial³⁶ institutions³⁷ to³⁸ fight³⁹ financial⁴⁰ crime⁴¹ more⁴² efficiently⁴³ and⁴⁴ effectively⁴⁵ .⁴⁶ Founded⁴⁷ on⁴⁸ broad⁴⁹ experience⁵⁰ and⁵¹ ##de⁵² ##ep⁵³ expertise⁵⁴ ,⁵⁵ Napier⁵⁶ ’⁵⁷ s⁵⁸ cutting⁵⁹ -⁶⁰ edge⁶¹ platform⁶² increases⁶³ efficiency⁶⁴ and⁶⁵ mini⁶⁶ ##mise⁶⁷ ##s⁶⁸ risk⁶⁹ by⁷⁰ successfully⁷¹ combining⁷² big⁷³ data⁷⁴ technologies⁷⁵ with⁷⁶ AI⁷⁷ .⁷⁸ Our⁷⁹ intelligent⁸⁰ approach⁸¹ to⁸² financial⁸³ crime⁸⁴ compliance⁸⁵ under⁸⁶ ##pin⁸⁷ ##s⁸⁸ organisation⁸⁹ ##al⁹⁰ policy⁹¹ ,⁹² process⁹³ and⁹⁴ procedure⁹⁵ ,⁹⁶ so⁹⁷ financial⁹⁸ institutions⁹⁹ can¹⁰⁰ focus¹⁰¹ on¹⁰² being¹⁰³ more¹⁰⁴ effective¹⁰⁵ at¹⁰⁶ detect¹⁰⁷ ##ing¹⁰⁸ suspicious¹⁰⁹ financial¹¹⁰ activity¹¹¹ in¹¹² a¹¹³ more¹¹⁴ cost¹¹⁵ -¹¹⁶ 

In [ ]:
#NER
start_ner_company = 1
end_ner_company = 1 + 1

#Q1
start_answer = 29
end_answer = 46 + 1

print(tokenizer.decode(encoded_input["input_ids"][start_ner_company:end_ner_company]))
print(tokenizer.decode(encoded_input["input_ids"][start_answer:end_answer]))

Napier
Napier are compliance technology specialists who help financial institutions to fight financial crime more efficiently and effectively.


In [ ]:
insert_new_labels(sample_id)

NER: {'start': 1, 'end': 2}
Answer: {'start': 29, 'end': 47}
Total Labeling Count: 33


In [ ]:
sample_id = next(data_sample_ids)
print(sample_id)
encoded_input = tokenizer(dict_data[sample_id]['desc'])
dict_data[sample_id]['desc']

302


'Quell delivers a virtual fitness world that offers a unique experience of intense workout while gaming to achieve fitness goals. The startup uses resistance bands to help players get fit while fighting their way through a virtual fitness world. It coins itself as a “Peloton meets gaming”, and charges a monthly fee to keep content fresh.'

In [ ]:
" ".join([tokenizer.decode(token) + get_super(str(i)) for i, token in enumerate(encoded_input["input_ids"])])

'[CLS]⁰ Que¹ ##ll² delivers³ a⁴ virtual⁵ fitness⁶ world⁷ that⁸ offers⁹ a¹⁰ unique¹¹ experience¹² of¹³ intense¹⁴ work¹⁵ ##out¹⁶ while¹⁷ gaming¹⁸ to¹⁹ achieve²⁰ fitness²¹ goals²² .²³ The²⁴ start²⁵ ##up²⁶ uses²⁷ resistance²⁸ bands²⁹ to³⁰ help³¹ players³² get³³ fit³⁴ while³⁵ fighting³⁶ their³⁷ way³⁸ through³⁹ a⁴⁰ virtual⁴¹ fitness⁴² world⁴³ .⁴⁴ It⁴⁵ coins⁴⁶ itself⁴⁷ as⁴⁸ a⁴⁹ “⁵⁰ P⁵¹ ##elo⁵² ##ton⁵³ meets⁵⁴ gaming⁵⁵ ”⁵⁶ ,⁵⁷ and⁵⁸ charges⁵⁹ a⁶⁰ monthly⁶¹ fee⁶² to⁶³ keep⁶⁴ content⁶⁵ fresh⁶⁶ .⁶⁷ [SEP]⁶⁸'

In [ ]:
#NER
start_ner_company = 1
end_ner_company = 2 + 1

#Q1
start_answer = 1
end_answer = 23 + 1

print(tokenizer.decode(encoded_input["input_ids"][start_ner_company:end_ner_company]))
print(tokenizer.decode(encoded_input["input_ids"][start_answer:end_answer]))

Quell
Quell delivers a virtual fitness world that offers a unique experience of intense workout while gaming to achieve fitness goals.


In [ ]:
insert_new_labels(sample_id)

NER: {'start': 1, 'end': 3}
Answer: {'start': 1, 'end': 24}
Total Labeling Count: 34


In [ ]:
sample_id = next(data_sample_ids)
print(sample_id)
encoded_input = tokenizer(dict_data[sample_id]['desc'])
dict_data[sample_id]['desc']

3483


"Clari is a connected revenue operations platform that uses automation and AI to unlock all the activity data captured in key business systems such as marketing automation, CRM, email, calendar, phone, content management, conversations, and more.Clari automatically aligns that data to accounts and opportunities, to deliver visibility, forecasting,and apply predictive insights, which results in more insight, less guesswork, and more predictable revenue. Clari helps companies by changing their revenue operations to be more connected, efficient, and predictable. Clari's platform is used by hundreds of sales, marketing, and customer success teams at B2B companies such as Qualtrics, Lenovo, Adobe, Dropbox, and Okta to control pipeline, audit deals and accounts, forecast the business, and reduce churn.Clari was founded by Venkat Rangan and Andy Byrne in 2013 and is headquartered in Sunnyvale, California.Read More"

In [ ]:
" ".join([tokenizer.decode(token) + get_super(str(i)) for i, token in enumerate(encoded_input["input_ids"])])

"[CLS]⁰ C¹ ##lar² ##i³ is⁴ a⁵ connected⁶ revenue⁷ operations⁸ platform⁹ that¹⁰ uses¹¹ automation¹² and¹³ AI¹⁴ to¹⁵ unlock¹⁶ all¹⁷ the¹⁸ activity¹⁹ data²⁰ captured²¹ in²² key²³ business²⁴ systems²⁵ such²⁶ as²⁷ marketing²⁸ automation²⁹ ,³⁰ CR³¹ ##M³² ,³³ email³⁴ ,³⁵ calendar³⁶ ,³⁷ phone³⁸ ,³⁹ content⁴⁰ management⁴¹ ,⁴² conversations⁴³ ,⁴⁴ and⁴⁵ more⁴⁶ .⁴⁷ C⁴⁸ ##lar⁴⁹ ##i⁵⁰ automatically⁵¹ al⁵² ##ign⁵³ ##s⁵⁴ that⁵⁵ data⁵⁶ to⁵⁷ accounts⁵⁸ and⁵⁹ opportunities⁶⁰ ,⁶¹ to⁶² deliver⁶³ visibility⁶⁴ ,⁶⁵ forecast⁶⁶ ##ing⁶⁷ ,⁶⁸ and⁶⁹ apply⁷⁰ predict⁷¹ ##ive⁷² insights⁷³ ,⁷⁴ which⁷⁵ results⁷⁶ in⁷⁷ more⁷⁸ insight⁷⁹ ,⁸⁰ less⁸¹ guess⁸² ##work⁸³ ,⁸⁴ and⁸⁵ more⁸⁶ predictable⁸⁷ revenue⁸⁸ .⁸⁹ C⁹⁰ ##lar⁹¹ ##i⁹² helps⁹³ companies⁹⁴ by⁹⁵ changing⁹⁶ their⁹⁷ revenue⁹⁸ operations⁹⁹ to¹⁰⁰ be¹⁰¹ more¹⁰² connected¹⁰³ ,¹⁰⁴ efficient¹⁰⁵ ,¹⁰⁶ and¹⁰⁷ predictable¹⁰⁸ .¹⁰⁹ C¹¹⁰ ##lar¹¹¹ ##i¹¹² '¹¹³ s¹¹⁴ platform¹¹⁵ is¹¹⁶ used¹¹⁷ by¹¹⁸ hundreds¹¹⁹ of¹²⁰ sales¹²¹ ,¹²² marketing¹²³ ,¹²⁴ and¹²⁵ customer¹²⁶ success¹²⁷ teams¹²⁸ 

In [ ]:
#NER
start_ner_company = 1
end_ner_company = 3 + 1

#Q1
start_answer = 1
end_answer = 89 + 1

print(tokenizer.decode(encoded_input["input_ids"][start_ner_company:end_ner_company]))
print(tokenizer.decode(encoded_input["input_ids"][start_answer:end_answer]))

Clari
Clari is a connected revenue operations platform that uses automation and AI to unlock all the activity data captured in key business systems such as marketing automation, CRM, email, calendar, phone, content management, conversations, and more. Clari automatically aligns that data to accounts and opportunities, to deliver visibility, forecasting, and apply predictive insights, which results in more insight, less guesswork, and more predictable revenue.


In [ ]:
insert_new_labels(sample_id)

NER: {'start': 1, 'end': 4}
Answer: {'start': 1, 'end': 90}
Total Labeling Count: 35


In [ ]:
sample_id = next(data_sample_ids)
print(sample_id)
encoded_input = tokenizer(dict_data[sample_id]['desc'])
dict_data[sample_id]['desc']

2801


'Stripe is a global technology company that builds economic infrastructure for the internet.Businesses of every size from new startups to public companies like Salesforce and Facebook use the company’s software to accept online payments and run complex global operations. The company combines economic infrastructure with a set of applications fornew business models like crowdfunding and marketplaces, fraud prevention, analytics, and more. Stripe navigates global regulatory uncertainty and partners closely with internet leaders like Apple, Google, Alipay, Tencent, Facebook, Twitter to launch new capabilities.It was founded in 2010 by John and Patrick Collison and headquartered in San Francisco, California.Read More'

In [ ]:
" ".join([tokenizer.decode(token) + get_super(str(i)) for i, token in enumerate(encoded_input["input_ids"])])

'[CLS]⁰ Strip¹ ##e² is³ a⁴ global⁵ technology⁶ company⁷ that⁸ builds⁹ economic¹⁰ infrastructure¹¹ for¹² the¹³ internet¹⁴ .¹⁵ Business¹⁶ ##es¹⁷ of¹⁸ every¹⁹ size²⁰ from²¹ new²² start²³ ##ups²⁴ to²⁵ public²⁶ companies²⁷ like²⁸ Sales²⁹ ##force³⁰ and³¹ Facebook³² use³³ the³⁴ company³⁵ ’³⁶ s³⁷ software³⁸ to³⁹ accept⁴⁰ online⁴¹ payments⁴² and⁴³ run⁴⁴ complex⁴⁵ global⁴⁶ operations⁴⁷ .⁴⁸ The⁴⁹ company⁵⁰ combines⁵¹ economic⁵² infrastructure⁵³ with⁵⁴ a⁵⁵ set⁵⁶ of⁵⁷ applications⁵⁸ for⁵⁹ ##ne⁶⁰ ##w⁶¹ business⁶² models⁶³ like⁶⁴ crowd⁶⁵ ##fu⁶⁶ ##nding⁶⁷ and⁶⁸ marketplace⁶⁹ ##s⁷⁰ ,⁷¹ fraud⁷² prevention⁷³ ,⁷⁴ anal⁷⁵ ##ytic⁷⁶ ##s⁷⁷ ,⁷⁸ and⁷⁹ more⁸⁰ .⁸¹ Strip⁸² ##e⁸³ navigate⁸⁴ ##s⁸⁵ global⁸⁶ regulatory⁸⁷ uncertainty⁸⁸ and⁸⁹ partners⁹⁰ closely⁹¹ with⁹² internet⁹³ leaders⁹⁴ like⁹⁵ Apple⁹⁶ ,⁹⁷ Google⁹⁸ ,⁹⁹ Ali¹⁰⁰ ##pa¹⁰¹ ##y¹⁰² ,¹⁰³ Ten¹⁰⁴ ##cent¹⁰⁵ ,¹⁰⁶ Facebook¹⁰⁷ ,¹⁰⁸ Twitter¹⁰⁹ to¹¹⁰ launch¹¹¹ new¹¹² capabilities¹¹³ .¹¹⁴ It¹¹⁵ was¹¹⁶ founded¹¹⁷ in¹¹⁸ 2010¹¹⁹ by¹²⁰ John¹²¹ and¹²² Patrick¹²³ Col¹²⁴ ##li

In [ ]:
#NER
start_ner_company = 1
end_ner_company = 2 + 1

#Q1
start_answer = 1
end_answer = 15 + 1

print(tokenizer.decode(encoded_input["input_ids"][start_ner_company:end_ner_company]))
print(tokenizer.decode(encoded_input["input_ids"][start_answer:end_answer]))

Stripe
Stripe is a global technology company that builds economic infrastructure for the internet.


In [ ]:
insert_new_labels(sample_id)

NER: {'start': 1, 'end': 3}
Answer: {'start': 1, 'end': 16}
Total Labeling Count: 36


In [ ]:
sample_id = next(data_sample_ids)
print(sample_id)
encoded_input = tokenizer(dict_data[sample_id]['desc'])
dict_data[sample_id]['desc']

1352


"Vue.ai is an end-to-end retail automation platform that is redesigning the future of fashion retail with Artificial Intelligence. Using Image Recognition and Data Science - we extract catalog data, analyze it with user behavior and help your marketing, product and cataloging teams get actionable insights that improve customer experiences, driveconversions and reduce costs. We help you digitally maps your products' DNA to create one-of-a-kind retail experiences for your customers and translate product information into the language that your teams can understand and make business decisions with.Read More"

In [ ]:
" ".join([tokenizer.decode(token) + get_super(str(i)) for i, token in enumerate(encoded_input["input_ids"])])

"[CLS]⁰ V¹ ##ue² .³ a⁴ ##i⁵ is⁶ an⁷ end⁸ -⁹ to¹⁰ -¹¹ end¹² retail¹³ automation¹⁴ platform¹⁵ that¹⁶ is¹⁷ red¹⁸ ##esign¹⁹ ##ing²⁰ the²¹ future²² of²³ fashion²⁴ retail²⁵ with²⁶ Art²⁷ ##ific²⁸ ##ial²⁹ Intelligence³⁰ .³¹ Using³² Image³³ Re³⁴ ##cognition³⁵ and³⁶ Data³⁷ Science³⁸ -³⁹ we⁴⁰ extract⁴¹ catalog⁴² data⁴³ ,⁴⁴ analyze⁴⁵ it⁴⁶ with⁴⁷ user⁴⁸ behavior⁴⁹ and⁵⁰ help⁵¹ your⁵² marketing⁵³ ,⁵⁴ product⁵⁵ and⁵⁶ catalog⁵⁷ ##ing⁵⁸ teams⁵⁹ get⁶⁰ action⁶¹ ##able⁶² insights⁶³ that⁶⁴ improve⁶⁵ customer⁶⁶ experiences⁶⁷ ,⁶⁸ drive⁶⁹ ##con⁷⁰ ##version⁷¹ ##s⁷² and⁷³ reduce⁷⁴ costs⁷⁵ .⁷⁶ We⁷⁷ help⁷⁸ you⁷⁹ digitally⁸⁰ maps⁸¹ your⁸² products⁸³ '⁸⁴ DNA⁸⁵ to⁸⁶ create⁸⁷ one⁸⁸ -⁸⁹ of⁹⁰ -⁹¹ a⁹² -⁹³ kind⁹⁴ retail⁹⁵ experiences⁹⁶ for⁹⁷ your⁹⁸ customers⁹⁹ and¹⁰⁰ translate¹⁰¹ product¹⁰² information¹⁰³ into¹⁰⁴ the¹⁰⁵ language¹⁰⁶ that¹⁰⁷ your¹⁰⁸ teams¹⁰⁹ can¹¹⁰ understand¹¹¹ and¹¹² make¹¹³ business¹¹⁴ decisions¹¹⁵ with¹¹⁶ .¹¹⁷ Read¹¹⁸ More¹¹⁹ [SEP]¹²⁰"

In [ ]:
#NER
start_ner_company = 1
end_ner_company = 5 + 1

#Q1
start_answer = 1
end_answer = 31 + 1

print(tokenizer.decode(encoded_input["input_ids"][start_ner_company:end_ner_company]))
print(tokenizer.decode(encoded_input["input_ids"][start_answer:end_answer]))

Vue. ai
Vue. ai is an end - to - end retail automation platform that is redesigning the future of fashion retail with Artificial Intelligence.


In [ ]:
insert_new_labels(sample_id)

NER: {'start': 1, 'end': 6}
Answer: {'start': 1, 'end': 32}
Total Labeling Count: 37


In [ ]:
sample_id = next(data_sample_ids)
print(sample_id)
encoded_input = tokenizer(dict_data[sample_id]['desc'])
dict_data[sample_id]['desc']

1933


'Arist is a tool that helps enterprise L&D and HR teams rapidly create, deliver, and analyze message-based courses, which are bite-size learning experiences delivered in the flow of SMS, Slack, and MS Teams.Along with saving over $200/employee/year with more efficient training, Arist helps companies create and track clear behaviorchange and performance improvement outcomes while delivering learning with an NPS of >90 and adoption rates that are 5x higher than traditional learning tools.Read More'

In [ ]:
" ".join([tokenizer.decode(token) + get_super(str(i)) for i, token in enumerate(encoded_input["input_ids"])])

'[CLS]⁰ Ari¹ ##st² is³ a⁴ tool⁵ that⁶ helps⁷ enterprise⁸ L⁹ &¹⁰ D¹¹ and¹² H¹³ ##R¹⁴ teams¹⁵ rapidly¹⁶ create¹⁷ ,¹⁸ deliver¹⁹ ,²⁰ and²¹ analyze²² message²³ -²⁴ based²⁵ courses²⁶ ,²⁷ which²⁸ are²⁹ bite³⁰ -³¹ size³² learning³³ experiences³⁴ delivered³⁵ in³⁶ the³⁷ flow³⁸ of³⁹ SMS⁴⁰ ,⁴¹ S⁴² ##la⁴³ ##ck⁴⁴ ,⁴⁵ and⁴⁶ MS⁴⁷ Teams⁴⁸ .⁴⁹ Along⁵⁰ with⁵¹ saving⁵² over⁵³ $⁵⁴ 200⁵⁵ /⁵⁶ employee⁵⁷ /⁵⁸ year⁵⁹ with⁶⁰ more⁶¹ efficient⁶² training⁶³ ,⁶⁴ Ari⁶⁵ ##st⁶⁶ helps⁶⁷ companies⁶⁸ create⁶⁹ and⁷⁰ track⁷¹ clear⁷² behavior⁷³ ##chang⁷⁴ ##e⁷⁵ and⁷⁶ performance⁷⁷ improvement⁷⁸ outcomes⁷⁹ while⁸⁰ delivering⁸¹ learning⁸² with⁸³ an⁸⁴ N⁸⁵ ##PS⁸⁶ of⁸⁷ >⁸⁸ 90⁸⁹ and⁹⁰ adoption⁹¹ rates⁹² that⁹³ are⁹⁴ 5⁹⁵ ##x⁹⁶ higher⁹⁷ than⁹⁸ traditional⁹⁹ learning¹⁰⁰ tools¹⁰¹ .¹⁰² Read¹⁰³ More¹⁰⁴ [SEP]¹⁰⁵'

In [ ]:
#NER
start_ner_company = 1
end_ner_company = 2 + 1

#Q1
start_answer = 1
end_answer = 49 + 1

print(tokenizer.decode(encoded_input["input_ids"][start_ner_company:end_ner_company]))
print(tokenizer.decode(encoded_input["input_ids"][start_answer:end_answer]))

Arist
Arist is a tool that helps enterprise L & D and HR teams rapidly create, deliver, and analyze message - based courses, which are bite - size learning experiences delivered in the flow of SMS, Slack, and MS Teams.


In [ ]:
insert_new_labels(sample_id)

NER: {'start': 1, 'end': 3}
Answer: {'start': 1, 'end': 50}
Total Labeling Count: 38


In [ ]:
sample_id = next(data_sample_ids)
print(sample_id)
encoded_input = tokenizer(dict_data[sample_id]['desc'])
dict_data[sample_id]['desc']

2292


'Dental Monitoring was founded in 2014 by its CEO, Philippe Salah, who holds a MS in Bioinformatics and a PhD in Biophysics from Ecole Polytechnique, Paris. From humble beginnings they have experienced a remarkable growth, from a handful of forward-thinkers to an international team of more than 400 people in Europe, the USA, Australia, Hong Kong andSingapore.Read More'

In [ ]:
" ".join([tokenizer.decode(token) + get_super(str(i)) for i, token in enumerate(encoded_input["input_ids"])])

'[CLS]⁰ Dental¹ Monitor² ##ing³ was⁴ founded⁵ in⁶ 2014⁷ by⁸ its⁹ CEO¹⁰ ,¹¹ Philippe¹² Sal¹³ ##ah¹⁴ ,¹⁵ who¹⁶ holds¹⁷ a¹⁸ MS¹⁹ in²⁰ B²¹ ##io²² ##in²³ ##formatics²⁴ and²⁵ a²⁶ PhD²⁷ in²⁸ B²⁹ ##io³⁰ ##physics³¹ from³² E³³ ##co³⁴ ##le³⁵ Pol³⁶ ##yte³⁷ ##ch³⁸ ##ni³⁹ ##que⁴⁰ ,⁴¹ Paris⁴² .⁴³ From⁴⁴ humble⁴⁵ beginnings⁴⁶ they⁴⁷ have⁴⁸ experienced⁴⁹ a⁵⁰ remarkable⁵¹ growth⁵² ,⁵³ from⁵⁴ a⁵⁵ handful⁵⁶ of⁵⁷ forward⁵⁸ -⁵⁹ think⁶⁰ ##ers⁶¹ to⁶² an⁶³ international⁶⁴ team⁶⁵ of⁶⁶ more⁶⁷ than⁶⁸ 400⁶⁹ people⁷⁰ in⁷¹ Europe⁷² ,⁷³ the⁷⁴ USA⁷⁵ ,⁷⁶ Australia⁷⁷ ,⁷⁸ Hong⁷⁹ Kong⁸⁰ and⁸¹ ##S⁸² ##inga⁸³ ##pore⁸⁴ .⁸⁵ Read⁸⁶ More⁸⁷ [SEP]⁸⁸'

In [ ]:
#NER
start_ner_company = 1
end_ner_company = 3 + 1

print(tokenizer.decode(encoded_input["input_ids"][start_ner_company:end_ner_company]))

Dental Monitoring


In [ ]:
insert_new_labels(sample_id, skip_answer = True)

NER: {'start': 1, 'end': 4}
Total Labeling Count: 39


In [ ]:
sample_id = next(data_sample_ids)
print(sample_id)
encoded_input = tokenizer(dict_data[sample_id]['desc'])
dict_data[sample_id]['desc']

412


'Slope connects a clinical trial to its supply chain, coordinating the operational complexity of getting devices, drugs, & kits from suppliers to sites to patients in real-time with complete traceability.'

In [ ]:
" ".join([tokenizer.decode(token) + get_super(str(i)) for i, token in enumerate(encoded_input["input_ids"])])

'[CLS]⁰ S¹ ##lop² ##e³ connects⁴ a⁵ clinical⁶ trial⁷ to⁸ its⁹ supply¹⁰ chain¹¹ ,¹² coordinating¹³ the¹⁴ operational¹⁵ complexity¹⁶ of¹⁷ getting¹⁸ devices¹⁹ ,²⁰ drugs²¹ ,²² &²³ kits²⁴ from²⁵ suppliers²⁶ to²⁷ sites²⁸ to²⁹ patients³⁰ in³¹ real³² -³³ time³⁴ with³⁵ complete³⁶ trace³⁷ ##ability³⁸ .³⁹ [SEP]⁴⁰'

In [ ]:
#NER
start_ner_company = 1
end_ner_company = 3 + 1

#Q1
start_answer = 1
end_answer = 39 + 1

print(tokenizer.decode(encoded_input["input_ids"][start_ner_company:end_ner_company]))
print(tokenizer.decode(encoded_input["input_ids"][start_answer:end_answer]))

Slope
Slope connects a clinical trial to its supply chain, coordinating the operational complexity of getting devices, drugs, & kits from suppliers to sites to patients in real - time with complete traceability.


In [ ]:
insert_new_labels(sample_id)

NER: {'start': 1, 'end': 4}
Answer: {'start': 1, 'end': 40}
Total Labeling Count: 40


In [ ]:
sample_id = next(data_sample_ids)
print(sample_id)
encoded_input = tokenizer(dict_data[sample_id]['desc'])
dict_data[sample_id]['desc']

1449


'Powerhouse Dynamics is a privately held technology firm focused on using cloud-based controls and analytics to deliver energy and operational efficiencies to its customers. Thousands of locations, including restaurants, convenience stores, retailers, and others, use the SiteSage platform (formerly known as the eMonitor) to save money bycentralizing control, analysis, and management of  energy-consuming equipment. Engineered specifically for smaller facilities, SiteSage features powerful analytics delivered via simple online and mobile interfaces. The system continuously improves efficiency by controlling major loads, monitoring power consumption and performance of individual pieces of equipment, and tracking gas and water usage. Advanced analytics deliver actionable intelligence via diagnostics and alerts, and SiteSage’s enterprise-wide benchmarking helps identify stars and underperforming assets. To learn more about how Powerhouse Dynamics’ customers maximize energy and equipment mai

In [ ]:
" ".join([tokenizer.decode(token) + get_super(str(i)) for i, token in enumerate(encoded_input["input_ids"])])

'[CLS]⁰ Power¹ ##house² Dynamics³ is⁴ a⁵ privately⁶ held⁷ technology⁸ firm⁹ focused¹⁰ on¹¹ using¹² cloud¹³ -¹⁴ based¹⁵ controls¹⁶ and¹⁷ anal¹⁸ ##ytic¹⁹ ##s²⁰ to²¹ deliver²² energy²³ and²⁴ operational²⁵ e²⁶ ##ff²⁷ ##ici²⁸ ##encies²⁹ to³⁰ its³¹ customers³² .³³ Thousands³⁴ of³⁵ locations³⁶ ,³⁷ including³⁸ restaurants³⁹ ,⁴⁰ convenience⁴¹ stores⁴² ,⁴³ retailers⁴⁴ ,⁴⁵ and⁴⁶ others⁴⁷ ,⁴⁸ use⁴⁹ the⁵⁰ Site⁵¹ ##S⁵² ##age⁵³ platform⁵⁴ (⁵⁵ formerly⁵⁶ known⁵⁷ as⁵⁸ the⁵⁹ e⁶⁰ ##M⁶¹ ##oni⁶² ##tor⁶³ )⁶⁴ to⁶⁵ save⁶⁶ money⁶⁷ by⁶⁸ ##cent⁶⁹ ##ral⁷⁰ ##izing⁷¹ control⁷² ,⁷³ analysis⁷⁴ ,⁷⁵ and⁷⁶ management⁷⁷ of⁷⁸ energy⁷⁹ -⁸⁰ consuming⁸¹ equipment⁸² .⁸³ Engineer⁸⁴ ##ed⁸⁵ specifically⁸⁶ for⁸⁷ smaller⁸⁸ facilities⁸⁹ ,⁹⁰ Site⁹¹ ##S⁹² ##age⁹³ features⁹⁴ powerful⁹⁵ anal⁹⁶ ##ytic⁹⁷ ##s⁹⁸ delivered⁹⁹ via¹⁰⁰ simple¹⁰¹ online¹⁰² and¹⁰³ mobile¹⁰⁴ interfaces¹⁰⁵ .¹⁰⁶ The¹⁰⁷ system¹⁰⁸ continuously¹⁰⁹ improve¹¹⁰ ##s¹¹¹ efficiency¹¹² by¹¹³ controlling¹¹⁴ major¹¹⁵ loads¹¹⁶ ,¹¹⁷ monitoring¹¹⁸ power¹¹⁹ consumption¹²⁰ and¹²¹ pe

In [ ]:
#NER
start_ner_company = 1
end_ner_company = 3 + 1

#Q1
start_answer = 1
end_answer = 33 + 1

print(tokenizer.decode(encoded_input["input_ids"][start_ner_company:end_ner_company]))
print(tokenizer.decode(encoded_input["input_ids"][start_answer:end_answer]))

Powerhouse Dynamics
Powerhouse Dynamics is a privately held technology firm focused on using cloud - based controls and analytics to deliver energy and operational efficiencies to its customers.


In [ ]:
insert_new_labels(sample_id)

NER: {'start': 1, 'end': 4}
Answer: {'start': 1, 'end': 34}
Total Labeling Count: 41


In [ ]:
sample_id = next(data_sample_ids)
print(sample_id)
encoded_input = tokenizer(dict_data[sample_id]['desc'])
dict_data[sample_id]['desc']

1778


'Consulta Remédios is an online tool for consulting health information, comparing medicines and perfumery. With more than 14 years, besides becoming a reference, the CR is one of the very few consolidated players that really have online experience in the healthcare market.'

In [ ]:
" ".join([tokenizer.decode(token) + get_super(str(i)) for i, token in enumerate(encoded_input["input_ids"])])

'[CLS]⁰ Consul¹ ##ta² Re³ ##mé⁴ ##dio⁵ ##s⁶ is⁷ an⁸ online⁹ tool¹⁰ for¹¹ consulting¹² health¹³ information¹⁴ ,¹⁵ comparing¹⁶ medicines¹⁷ and¹⁸ perfume¹⁹ ##ry²⁰ .²¹ With²² more²³ than²⁴ 14²⁵ years²⁶ ,²⁷ besides²⁸ becoming²⁹ a³⁰ reference³¹ ,³² the³³ CR³⁴ is³⁵ one³⁶ of³⁷ the³⁸ very³⁹ few⁴⁰ consolidated⁴¹ players⁴² that⁴³ really⁴⁴ have⁴⁵ online⁴⁶ experience⁴⁷ in⁴⁸ the⁴⁹ healthcare⁵⁰ market⁵¹ .⁵² [SEP]⁵³'

In [ ]:
#NER
start_ner_company = 1
end_ner_company = 6 + 1

#Q1
start_answer = 1
end_answer = 21 + 1

print(tokenizer.decode(encoded_input["input_ids"][start_ner_company:end_ner_company]))
print(tokenizer.decode(encoded_input["input_ids"][start_answer:end_answer]))

Consulta Remédios
Consulta Remédios is an online tool for consulting health information, comparing medicines and perfumery.


In [ ]:
insert_new_labels(sample_id)

NER: {'start': 1, 'end': 7}
Answer: {'start': 1, 'end': 22}
Total Labeling Count: 42


In [ ]:
sample_id = next(data_sample_ids)
print(sample_id)
encoded_input = tokenizer(dict_data[sample_id]['desc'])
dict_data[sample_id]['desc']

1295


'Wayz.ai is a Chinese smart location and mapping service startup.'

In [ ]:
" ".join([tokenizer.decode(token) + get_super(str(i)) for i, token in enumerate(encoded_input["input_ids"])])

'[CLS]⁰ Way¹ ##z² .³ a⁴ ##i⁵ is⁶ a⁷ Chinese⁸ smart⁹ location¹⁰ and¹¹ mapping¹² service¹³ start¹⁴ ##up¹⁵ .¹⁶ [SEP]¹⁷'

In [ ]:
#NER
start_ner_company = 1
end_ner_company = 5 + 1

#Q1
start_answer = 1
end_answer = 16 + 1

print(tokenizer.decode(encoded_input["input_ids"][start_ner_company:end_ner_company]))
print(tokenizer.decode(encoded_input["input_ids"][start_answer:end_answer]))

Wayz. ai
Wayz. ai is a Chinese smart location and mapping service startup.


In [ ]:
insert_new_labels(sample_id)

NER: {'start': 1, 'end': 6}
Answer: {'start': 1, 'end': 17}
Total Labeling Count: 43


In [ ]:
sample_id = next(data_sample_ids)
print(sample_id)
encoded_input = tokenizer(dict_data[sample_id]['desc'])
dict_data[sample_id]['desc']

2502


'IntensiCare specializes in ICU management and telemedicine care services for hospitals.'

In [ ]:
" ".join([tokenizer.decode(token) + get_super(str(i)) for i, token in enumerate(encoded_input["input_ids"])])

'[CLS]⁰ In¹ ##tens² ##i³ ##C⁴ ##are⁵ specializes⁶ in⁷ I⁸ ##CU⁹ management¹⁰ and¹¹ te¹² ##lem¹³ ##edic¹⁴ ##ine¹⁵ care¹⁶ services¹⁷ for¹⁸ hospitals¹⁹ .²⁰ [SEP]²¹'

In [ ]:
#NER
start_ner_company = 1
end_ner_company = 5 + 1

#Q1
start_answer = 1
end_answer = 20 + 1

print(tokenizer.decode(encoded_input["input_ids"][start_ner_company:end_ner_company]))
print(tokenizer.decode(encoded_input["input_ids"][start_answer:end_answer]))

IntensiCare
IntensiCare specializes in ICU management and telemedicine care services for hospitals.


In [ ]:
insert_new_labels(sample_id)

NER: {'start': 1, 'end': 6}
Answer: {'start': 1, 'end': 21}
Total Labeling Count: 44


In [ ]:
sample_id = next(data_sample_ids)
print(sample_id)
encoded_input = tokenizer(dict_data[sample_id]['desc'])
dict_data[sample_id]['desc']

2623


'DeepRoute.ai is an international self-driving technology company committed to popularizing robotaxis and advancing urban logistics. DeepRoute.ai is the first autonomous driving company in China with Alibaba as a strategic investor, and its team includes leading experts from Ford, Google, Intel, Microsoft, and other innovators. \nDeepRoute.ai is thefirst to provide its robotaxi service in downtown Shenzhen the megacity and the first to deliver Covid-19 test samples in China using its robotaxis. Additionally, DeepRoute.ai has earned a Passenger Carry Permit from the CPUC, and partnered with Dongfeng Motors and Cao Cao Mobility to build a robotaxi fleet, accumulating over one million miles driven on complex urban roads in China. The company was also recognized with a CES Innovation Award in 2020.Read More'

In [ ]:
" ".join([tokenizer.decode(token) + get_super(str(i)) for i, token in enumerate(encoded_input["input_ids"])])

'[CLS]⁰ Deep¹ ##R² ##out³ ##e⁴ .⁵ a⁶ ##i⁷ is⁸ an⁹ international¹⁰ self¹¹ -¹² driving¹³ technology¹⁴ company¹⁵ committed¹⁶ to¹⁷ popular¹⁸ ##izing¹⁹ robot²⁰ ##ax²¹ ##is²² and²³ advancing²⁴ urban²⁵ logistics²⁶ .²⁷ Deep²⁸ ##R²⁹ ##out³⁰ ##e³¹ .³² a³³ ##i³⁴ is³⁵ the³⁶ first³⁷ autonomous³⁸ driving³⁹ company⁴⁰ in⁴¹ China⁴² with⁴³ Ali⁴⁴ ##ba⁴⁵ ##ba⁴⁶ as⁴⁷ a⁴⁸ strategic⁴⁹ investor⁵⁰ ,⁵¹ and⁵² its⁵³ team⁵⁴ includes⁵⁵ leading⁵⁶ experts⁵⁷ from⁵⁸ Ford⁵⁹ ,⁶⁰ Google⁶¹ ,⁶² Intel⁶³ ,⁶⁴ Microsoft⁶⁵ ,⁶⁶ and⁶⁷ other⁶⁸ inn⁶⁹ ##ova⁷⁰ ##tors⁷¹ .⁷² Deep⁷³ ##R⁷⁴ ##out⁷⁵ ##e⁷⁶ .⁷⁷ a⁷⁸ ##i⁷⁹ is⁸⁰ the⁸¹ ##fi⁸² ##rst⁸³ to⁸⁴ provide⁸⁵ its⁸⁶ robot⁸⁷ ##ax⁸⁸ ##i⁸⁹ service⁹⁰ in⁹¹ downtown⁹² Shen⁹³ ##zhen⁹⁴ the⁹⁵ me⁹⁶ ##ga⁹⁷ ##city⁹⁸ and⁹⁹ the¹⁰⁰ first¹⁰¹ to¹⁰² deliver¹⁰³ Co¹⁰⁴ ##vid¹⁰⁵ -¹⁰⁶ 19¹⁰⁷ test¹⁰⁸ samples¹⁰⁹ in¹¹⁰ China¹¹¹ using¹¹² its¹¹³ robot¹¹⁴ ##ax¹¹⁵ ##is¹¹⁶ .¹¹⁷ Additionally¹¹⁸ ,¹¹⁹ Deep¹²⁰ ##R¹²¹ ##out¹²² ##e¹²³ .¹²⁴ a¹²⁵ ##i¹²⁶ has¹²⁷ earned¹²⁸ a¹²⁹ Passenger¹³⁰ Carr¹³¹ ##y¹³² Per¹³³ ##mit¹³⁴ from¹³⁵ the¹

In [ ]:
#NER
start_ner_company = 1
end_ner_company = 7 + 1

#Q1
start_answer = 1
end_answer = 27 + 1

print(tokenizer.decode(encoded_input["input_ids"][start_ner_company:end_ner_company]))
print(tokenizer.decode(encoded_input["input_ids"][start_answer:end_answer]))

DeepRoute. ai
DeepRoute. ai is an international self - driving technology company committed to popularizing robotaxis and advancing urban logistics.


In [ ]:
insert_new_labels(sample_id)

NER: {'start': 1, 'end': 8}
Answer: {'start': 1, 'end': 28}
Total Labeling Count: 45


In [ ]:
sample_id = next(data_sample_ids)
print(sample_id)
encoded_input = tokenizer(dict_data[sample_id]['desc'])
dict_data[sample_id]['desc']

837


'Chance Technologies was established to provide Information Technology services to individuals and small businesses in the Southwest and Midwest with emphasis on quality of work, honesty, and security.Their services include total business continuity, managed services, cloud computing and virtualization, security auditing, remote assistancetroubleshooting, residential computer support, small-business systems support as well as network design and implementation. They are creative in designing the right plan for your business and layer their services in a way that works for your environment and budget. By choosing us to be your IT support team, you are getting peace of mind knowing that your business technology is being operated securely and efficiently at all times, as well as having proactive computer and networking support.They also provide specialty solutions including email accounts, web hosting and design, shell accounts, and more. Their clients enjoy a variety of services in combin

In [ ]:
" ".join([tokenizer.decode(token) + get_super(str(i)) for i, token in enumerate(encoded_input["input_ids"])])

'[CLS]⁰ Chance¹ Technologies² was³ established⁴ to⁵ provide⁶ Information⁷ Technology⁸ services⁹ to¹⁰ individuals¹¹ and¹² small¹³ businesses¹⁴ in¹⁵ the¹⁶ Southwest¹⁷ and¹⁸ Midwest¹⁹ with²⁰ emphasis²¹ on²² quality²³ of²⁴ work²⁵ ,²⁶ honesty²⁷ ,²⁸ and²⁹ security³⁰ .³¹ Their³² services³³ include³⁴ total³⁵ business³⁶ continuity³⁷ ,³⁸ managed³⁹ services⁴⁰ ,⁴¹ cloud⁴² computing⁴³ and⁴⁴ virtual⁴⁵ ##ization⁴⁶ ,⁴⁷ security⁴⁸ audit⁴⁹ ##ing⁵⁰ ,⁵¹ remote⁵² assistance⁵³ ##tro⁵⁴ ##ub⁵⁵ ##lesh⁵⁶ ##oot⁵⁷ ##ing⁵⁸ ,⁵⁹ residential⁶⁰ computer⁶¹ support⁶² ,⁶³ small⁶⁴ -⁶⁵ business⁶⁶ systems⁶⁷ support⁶⁸ as⁶⁹ well⁷⁰ as⁷¹ network⁷² design⁷³ and⁷⁴ implementation⁷⁵ .⁷⁶ They⁷⁷ are⁷⁸ creative⁷⁹ in⁸⁰ designing⁸¹ the⁸² right⁸³ plan⁸⁴ for⁸⁵ your⁸⁶ business⁸⁷ and⁸⁸ layer⁸⁹ their⁹⁰ services⁹¹ in⁹² a⁹³ way⁹⁴ that⁹⁵ works⁹⁶ for⁹⁷ your⁹⁸ environment⁹⁹ and¹⁰⁰ budget¹⁰¹ .¹⁰² By¹⁰³ choosing¹⁰⁴ us¹⁰⁵ to¹⁰⁶ be¹⁰⁷ your¹⁰⁸ IT¹⁰⁹ support¹¹⁰ team¹¹¹ ,¹¹² you¹¹³ are¹¹⁴ getting¹¹⁵ peace¹¹⁶ of¹¹⁷ mind¹¹⁸ knowing¹¹⁹ that¹²⁰ your¹²¹ busi

In [ ]:
#NER
start_ner_company = 1
end_ner_company = 2 + 1

#Q1
start_answer = 1
end_answer = 31 + 1

print(tokenizer.decode(encoded_input["input_ids"][start_ner_company:end_ner_company]))
print(tokenizer.decode(encoded_input["input_ids"][start_answer:end_answer]))

Chance Technologies
Chance Technologies was established to provide Information Technology services to individuals and small businesses in the Southwest and Midwest with emphasis on quality of work, honesty, and security.


In [ ]:
insert_new_labels(sample_id)

NER: {'start': 1, 'end': 3}
Answer: {'start': 1, 'end': 32}
Total Labeling Count: 46


In [ ]:
sample_id = next(data_sample_ids)
print(sample_id)
encoded_input = tokenizer(dict_data[sample_id]['desc'])
dict_data[sample_id]['desc']

2263


'SafeCharge is now Nuvei the payment technology partner of thriving brands. We provide the intelligence and technology businesses need to succeed locally and globally, through one integration — propelling them further, faster. Uniting payment consulting and payment technology, we help businesses remove payment barriers, optimize operating costs andincrease acceptance rates both locally and globally. We deliver all the solutions businesses need to provide a flawless payment experience. Wherever and however payments are made. But our edge doesn’t stop there. We have proven tools to help them sell more, convert more and extract more revenue out of each transaction. With the best merchant services out there, our goal is to empower businesses to make the world their local marketplace.Read More'

In [ ]:
" ".join([tokenizer.decode(token) + get_super(str(i)) for i, token in enumerate(encoded_input["input_ids"])])

'[CLS]⁰ Safe¹ ##C² ##har³ ##ge⁴ is⁵ now⁶ N⁷ ##uve⁸ ##i⁹ the¹⁰ payment¹¹ technology¹² partner¹³ of¹⁴ thriving¹⁵ brands¹⁶ .¹⁷ We¹⁸ provide¹⁹ the²⁰ intelligence²¹ and²² technology²³ businesses²⁴ need²⁵ to²⁶ succeed²⁷ locally²⁸ and²⁹ globally³⁰ ,³¹ through³² one³³ integration³⁴ —³⁵ prop³⁶ ##elling³⁷ them³⁸ further³⁹ ,⁴⁰ faster⁴¹ .⁴² Unit⁴³ ##ing⁴⁴ payment⁴⁵ consulting⁴⁶ and⁴⁷ payment⁴⁸ technology⁴⁹ ,⁵⁰ we⁵¹ help⁵² businesses⁵³ remove⁵⁴ payment⁵⁵ barriers⁵⁶ ,⁵⁷ op⁵⁸ ##ti⁵⁹ ##mize⁶⁰ operating⁶¹ costs⁶² and⁶³ ##in⁶⁴ ##cre⁶⁵ ##ase⁶⁶ acceptance⁶⁷ rates⁶⁸ both⁶⁹ locally⁷⁰ and⁷¹ globally⁷² .⁷³ We⁷⁴ deliver⁷⁵ all⁷⁶ the⁷⁷ solutions⁷⁸ businesses⁷⁹ need⁸⁰ to⁸¹ provide⁸² a⁸³ fl⁸⁴ ##aw⁸⁵ ##less⁸⁶ payment⁸⁷ experience⁸⁸ .⁸⁹ Where⁹⁰ ##ver⁹¹ and⁹² however⁹³ payments⁹⁴ are⁹⁵ made⁹⁶ .⁹⁷ But⁹⁸ our⁹⁹ edge¹⁰⁰ doesn¹⁰¹ ’¹⁰² t¹⁰³ stop¹⁰⁴ there¹⁰⁵ .¹⁰⁶ We¹⁰⁷ have¹⁰⁸ proven¹⁰⁹ tools¹¹⁰ to¹¹¹ help¹¹² them¹¹³ sell¹¹⁴ more¹¹⁵ ,¹¹⁶ convert¹¹⁷ more¹¹⁸ and¹¹⁹ extract¹²⁰ more¹²¹ revenue¹²² out¹²³ of¹²⁴ each¹²⁵ transactio

In [ ]:
#NER
start_ner_company = 1
end_ner_company = 4 + 1

#Q1
start_answer = 1
end_answer = 17 + 1

print(tokenizer.decode(encoded_input["input_ids"][start_ner_company:end_ner_company]))
print(tokenizer.decode(encoded_input["input_ids"][start_answer:end_answer]))

SafeCharge
SafeCharge is now Nuvei the payment technology partner of thriving brands.


In [ ]:
insert_new_labels(sample_id)

NER: {'start': 1, 'end': 5}
Answer: {'start': 1, 'end': 18}
Total Labeling Count: 47


In [ ]:
sample_id = next(data_sample_ids)
print(sample_id)
encoded_input = tokenizer(dict_data[sample_id]['desc'])
dict_data[sample_id]['desc']

1953


'Kalido is an amazing AI powered platform that matches enterprise workforce skills and strengths to people, teams, projects and jobs.'

In [ ]:
" ".join([tokenizer.decode(token) + get_super(str(i)) for i, token in enumerate(encoded_input["input_ids"])])

'[CLS]⁰ Kali¹ ##do² is³ an⁴ amazing⁵ AI⁶ powered⁷ platform⁸ that⁹ matches¹⁰ enterprise¹¹ workforce¹² skills¹³ and¹⁴ strengths¹⁵ to¹⁶ people¹⁷ ,¹⁸ teams¹⁹ ,²⁰ projects²¹ and²² jobs²³ .²⁴ [SEP]²⁵'

In [ ]:
#NER
start_ner_company = 1
end_ner_company = 2 + 1

#Q1
start_answer = 1
end_answer = 24 + 1

print(tokenizer.decode(encoded_input["input_ids"][start_ner_company:end_ner_company]))
print(tokenizer.decode(encoded_input["input_ids"][start_answer:end_answer]))

Kalido
Kalido is an amazing AI powered platform that matches enterprise workforce skills and strengths to people, teams, projects and jobs.


In [ ]:
insert_new_labels(sample_id)

NER: {'start': 1, 'end': 3}
Answer: {'start': 1, 'end': 25}
Total Labeling Count: 48


In [ ]:
sample_id = next(data_sample_ids)
print(sample_id)
encoded_input = tokenizer(dict_data[sample_id]['desc'])
dict_data[sample_id]['desc']

1813


'XIKANG is a cloud-based medical and health management platform.'

In [ ]:
" ".join([tokenizer.decode(token) + get_super(str(i)) for i, token in enumerate(encoded_input["input_ids"])])

'[CLS]⁰ XI¹ ##KA² ##NG³ is⁴ a⁵ cloud⁶ -⁷ based⁸ medical⁹ and¹⁰ health¹¹ management¹² platform¹³ .¹⁴ [SEP]¹⁵'

In [ ]:
#NER
start_ner_company = 1
end_ner_company = 3 + 1

#Q1
start_answer = 1
end_answer = 14 + 1

print(tokenizer.decode(encoded_input["input_ids"][start_ner_company:end_ner_company]))
print(tokenizer.decode(encoded_input["input_ids"][start_answer:end_answer]))

XIKANG
XIKANG is a cloud - based medical and health management platform.


In [ ]:
insert_new_labels(sample_id)

NER: {'start': 1, 'end': 4}
Answer: {'start': 1, 'end': 15}
Total Labeling Count: 49


In [ ]:
sample_id = next(data_sample_ids)
print(sample_id)
encoded_input = tokenizer(dict_data[sample_id]['desc'])
dict_data[sample_id]['desc']

3544


'Genera is currently the main national reference in genetic tests for Ancestry, Health and Well-being. Visit our website and discover the secrets of your DNA!'

In [ ]:
" ".join([tokenizer.decode(token) + get_super(str(i)) for i, token in enumerate(encoded_input["input_ids"])])

'[CLS]⁰ Gene¹ ##ra² is³ currently⁴ the⁵ main⁶ national⁷ reference⁸ in⁹ genetic¹⁰ tests¹¹ for¹² An¹³ ##ces¹⁴ ##try¹⁵ ,¹⁶ Health¹⁷ and¹⁸ Well¹⁹ -²⁰ being²¹ .²² V²³ ##isi²⁴ ##t²⁵ our²⁶ website²⁷ and²⁸ discover²⁹ the³⁰ secrets³¹ of³² your³³ DNA³⁴ !³⁵ [SEP]³⁶'

In [ ]:
#NER
start_ner_company = 1
end_ner_company = 2 + 1

#Q1
start_answer = 1
end_answer = 22 + 1

print(tokenizer.decode(encoded_input["input_ids"][start_ner_company:end_ner_company]))
print(tokenizer.decode(encoded_input["input_ids"][start_answer:end_answer]))

Genera
Genera is currently the main national reference in genetic tests for Ancestry, Health and Well - being.


In [ ]:
insert_new_labels(sample_id)

NER: {'start': 1, 'end': 3}
Answer: {'start': 1, 'end': 23}
Total Labeling Count: 50


In [ ]:
sample_id = next(data_sample_ids)
print(sample_id)
encoded_input = tokenizer(dict_data[sample_id]['desc'])
dict_data[sample_id]['desc']

2135


'Cognism is an end-to-end sales-acceleration solution that provides sales organizations with a more efficient way to prospect. Delivered as a software service (SaaS), with its unique data asset and compliance engine, Cognism is helping to enrich CRM records, stream leads into the funnel and is using artificial intelligence to surface opportunitiesand identify customer trends. The company is a pure AI sales technology company that generates prospect data at scale, cleaning and enriching it, helping sales teams to grow and scale across all levels of the sales process. \xa0Cognism using patented AI technology, to provide B2B sales teams with a blend of real-time company, people and event data to streamline prospecting, find and deliver new revenue. It integrates seamlessly with all major CRM and Email Service systems. They use it to manage their database and provide high-quality, real-time B2B data to their clients.Cognism was founded in 2015 and is headquartered in London, England, UK, w

In [ ]:
" ".join([tokenizer.decode(token) + get_super(str(i)) for i, token in enumerate(encoded_input["input_ids"])])

'[CLS]⁰ Co¹ ##gni² ##sm³ is⁴ an⁵ end⁶ -⁷ to⁸ -⁹ end¹⁰ sales¹¹ -¹² acceleration¹³ solution¹⁴ that¹⁵ provides¹⁶ sales¹⁷ organizations¹⁸ with¹⁹ a²⁰ more²¹ efficient²² way²³ to²⁴ prospect²⁵ .²⁶ Del²⁷ ##iver²⁸ ##ed²⁹ as³⁰ a³¹ software³² service³³ (³⁴ Sa³⁵ ##a³⁶ ##S³⁷ )³⁸ ,³⁹ with⁴⁰ its⁴¹ unique⁴² data⁴³ asset⁴⁴ and⁴⁵ compliance⁴⁶ engine⁴⁷ ,⁴⁸ Co⁴⁹ ##gni⁵⁰ ##sm⁵¹ is⁵² helping⁵³ to⁵⁴ en⁵⁵ ##rich⁵⁶ CR⁵⁷ ##M⁵⁸ records⁵⁹ ,⁶⁰ stream⁶¹ leads⁶² into⁶³ the⁶⁴ fun⁶⁵ ##nel⁶⁶ and⁶⁷ is⁶⁸ using⁶⁹ artificial⁷⁰ intelligence⁷¹ to⁷² surface⁷³ opportunities⁷⁴ ##and⁷⁵ identify⁷⁶ customer⁷⁷ trends⁷⁸ .⁷⁹ The⁸⁰ company⁸¹ is⁸² a⁸³ pure⁸⁴ AI⁸⁵ sales⁸⁶ technology⁸⁷ company⁸⁸ that⁸⁹ generates⁹⁰ prospect⁹¹ data⁹² at⁹³ scale⁹⁴ ,⁹⁵ cleaning⁹⁶ and⁹⁷ en⁹⁸ ##rich⁹⁹ ##ing¹⁰⁰ it¹⁰¹ ,¹⁰² helping¹⁰³ sales¹⁰⁴ teams¹⁰⁵ to¹⁰⁶ grow¹⁰⁷ and¹⁰⁸ scale¹⁰⁹ across¹¹⁰ all¹¹¹ levels¹¹² of¹¹³ the¹¹⁴ sales¹¹⁵ process¹¹⁶ .¹¹⁷ Co¹¹⁸ ##gni¹¹⁹ ##sm¹²⁰ using¹²¹ patented¹²² AI¹²³ technology¹²⁴ ,¹²⁵ to¹²⁶ provide¹²⁷ B¹²⁸ ##2¹²⁹ ##B¹³⁰ sales¹³¹ teams

In [ ]:
#NER
start_ner_company = 1
end_ner_company = 3 + 1

#Q1
start_answer = 1
end_answer = 26 + 1

print(tokenizer.decode(encoded_input["input_ids"][start_ner_company:end_ner_company]))
print(tokenizer.decode(encoded_input["input_ids"][start_answer:end_answer]))

Cognism
Cognism is an end - to - end sales - acceleration solution that provides sales organizations with a more efficient way to prospect.


In [ ]:
insert_new_labels(sample_id)

NER: {'start': 1, 'end': 4}
Answer: {'start': 1, 'end': 27}
Total Labeling Count: 51


In [ ]:
sample_id = next(data_sample_ids)
print(sample_id)
encoded_input = tokenizer(dict_data[sample_id]['desc'])
dict_data[sample_id]['desc']

1066


'Paerpay helps merchants nationwide accept contactless payments without the hassle of changing point-of-sales systems, processors or requiring guests to download an app. The company was founded in 2017 and is headquartered in Boston, Massachusetts, United States.'

In [ ]:
" ".join([tokenizer.decode(token) + get_super(str(i)) for i, token in enumerate(encoded_input["input_ids"])])

'[CLS]⁰ Pa¹ ##er² ##pa³ ##y⁴ helps⁵ merchants⁶ nationwide⁷ accept⁸ contact⁹ ##less¹⁰ payments¹¹ without¹² the¹³ has¹⁴ ##sle¹⁵ of¹⁶ changing¹⁷ point¹⁸ -¹⁹ of²⁰ -²¹ sales²² systems²³ ,²⁴ processors²⁵ or²⁶ requiring²⁷ guests²⁸ to²⁹ download³⁰ an³¹ app³² .³³ The³⁴ company³⁵ was³⁶ founded³⁷ in³⁸ 2017³⁹ and⁴⁰ is⁴¹ headquartered⁴² in⁴³ Boston⁴⁴ ,⁴⁵ Massachusetts⁴⁶ ,⁴⁷ United⁴⁸ States⁴⁹ .⁵⁰ [SEP]⁵¹'

In [ ]:
#NER
start_ner_company = 1
end_ner_company = 4 + 1

#Q1
start_answer = 1
end_answer = 33 + 1

print(tokenizer.decode(encoded_input["input_ids"][start_ner_company:end_ner_company]))
print(tokenizer.decode(encoded_input["input_ids"][start_answer:end_answer]))

Paerpay
Paerpay helps merchants nationwide accept contactless payments without the hassle of changing point - of - sales systems, processors or requiring guests to download an app.


In [ ]:
insert_new_labels(sample_id)

NER: {'start': 1, 'end': 5}
Answer: {'start': 1, 'end': 34}
Total Labeling Count: 52


In [ ]:
sample_id = next(data_sample_ids)
print(sample_id)
encoded_input = tokenizer(dict_data[sample_id]['desc'])
dict_data[sample_id]['desc']

255


'Flow solves the complexity of fund infrastructure. We connect information across investors, providers, investments and systems to realize our mission of empowering thought. Powered by dynamic tools and human centric interfaces, our cross platform SaaS technology networks and adapts to the myriad of ways funds operate to optimize productivity andincrease time invested into making decisions. By bringing Flow to the $10 Trillion alternative asset market, we are able to increase the speed of decisions, the speed of capital and the speed of growth that serve as the lifeblood for all market based economies.Read More'

In [ ]:
" ".join([tokenizer.decode(token) + get_super(str(i)) for i, token in enumerate(encoded_input["input_ids"])])

'[CLS]⁰ Flow¹ solve² ##s³ the⁴ complexity⁵ of⁶ fund⁷ infrastructure⁸ .⁹ We¹⁰ connect¹¹ information¹² across¹³ investors¹⁴ ,¹⁵ providers¹⁶ ,¹⁷ investments¹⁸ and¹⁹ systems²⁰ to²¹ realize²² our²³ mission²⁴ of²⁵ em²⁶ ##powering²⁷ thought²⁸ .²⁹ Power³⁰ ##ed³¹ by³² dynamic³³ tools³⁴ and³⁵ human³⁶ cent³⁷ ##ric³⁸ interfaces³⁹ ,⁴⁰ our⁴¹ cross⁴² platform⁴³ Sa⁴⁴ ##a⁴⁵ ##S⁴⁶ technology⁴⁷ networks⁴⁸ and⁴⁹ adapt⁵⁰ ##s⁵¹ to⁵² the⁵³ my⁵⁴ ##riad⁵⁵ of⁵⁶ ways⁵⁷ funds⁵⁸ operate⁵⁹ to⁶⁰ op⁶¹ ##ti⁶² ##mize⁶³ productivity⁶⁴ and⁶⁵ ##in⁶⁶ ##cre⁶⁷ ##ase⁶⁸ time⁶⁹ invested⁷⁰ into⁷¹ making⁷² decisions⁷³ .⁷⁴ By⁷⁵ bringing⁷⁶ Flow⁷⁷ to⁷⁸ the⁷⁹ $⁸⁰ 10⁸¹ Tri⁸² ##lli⁸³ ##on⁸⁴ alternative⁸⁵ asset⁸⁶ market⁸⁷ ,⁸⁸ we⁸⁹ are⁹⁰ able⁹¹ to⁹² increase⁹³ the⁹⁴ speed⁹⁵ of⁹⁶ decisions⁹⁷ ,⁹⁸ the⁹⁹ speed¹⁰⁰ of¹⁰¹ capital¹⁰² and¹⁰³ the¹⁰⁴ speed¹⁰⁵ of¹⁰⁶ growth¹⁰⁷ that¹⁰⁸ serve¹⁰⁹ as¹¹⁰ the¹¹¹ life¹¹² ##b¹¹³ ##lo¹¹⁴ ##od¹¹⁵ for¹¹⁶ all¹¹⁷ market¹¹⁸ based¹¹⁹ economies¹²⁰ .¹²¹ Read¹²² More¹²³ [SEP]¹²⁴'

In [ ]:
#NER
start_ner_company = 1
end_ner_company = 3 + 1

#Q1
start_answer = 1
end_answer = 9 + 1

print(tokenizer.decode(encoded_input["input_ids"][start_ner_company:end_ner_company]))
print(tokenizer.decode(encoded_input["input_ids"][start_answer:end_answer]))

Flow solves
Flow solves the complexity of fund infrastructure.


In [ ]:
insert_new_labels(sample_id)

NER: {'start': 1, 'end': 4}
Answer: {'start': 1, 'end': 10}
Total Labeling Count: 53


In [ ]:
sample_id = next(data_sample_ids)
print(sample_id)
encoded_input = tokenizer(dict_data[sample_id]['desc'])
dict_data[sample_id]['desc']

3297


'Loadsure is a spot freight cargo insurance company.'

In [ ]:
" ".join([tokenizer.decode(token) + get_super(str(i)) for i, token in enumerate(encoded_input["input_ids"])])

'[CLS]⁰ Lo¹ ##ads² ##ure³ is⁴ a⁵ spot⁶ freight⁷ cargo⁸ insurance⁹ company¹⁰ .¹¹ [SEP]¹²'

In [ ]:
#NER
start_ner_company = 1
end_ner_company = 3 + 1

#Q1
start_answer = 1
end_answer = 11 + 1

print(tokenizer.decode(encoded_input["input_ids"][start_ner_company:end_ner_company]))
print(tokenizer.decode(encoded_input["input_ids"][start_answer:end_answer]))

Loadsure
Loadsure is a spot freight cargo insurance company.


In [ ]:
insert_new_labels(sample_id)

NER: {'start': 1, 'end': 4}
Answer: {'start': 1, 'end': 12}
Total Labeling Count: 54


In [ ]:
sample_id = next(data_sample_ids)
print(sample_id)
encoded_input = tokenizer(dict_data[sample_id]['desc'])
dict_data[sample_id]['desc']

2247


'WorkRamp is an enterprise learning platform for employee training and customer education at top companies like Zoom, Box, and Workiva. The company is committed to transforming the future of work by delivering an unparalleled, digital learning experience for employees and customers. With WorkRamp, users feel empowered to learn, collaborate, anddevelop their skills in a digital workspace that bridges physical and virtual training.The company is building a dynamic learning platform that focuses on delivering training based on driving business outcomes. Its platform allows companies to manage role-specific training and increase team performance, while also supporting larger HR and compliance needs. WorkRamp developed training products to address the specific learning needs for sales, support, and customer education teams–that in turn, helps bolster employee productivity, increase company revenue, and drive customer loyalty.WorkRamp was founded in 2015 and is based in San Francisco, Califo

In [ ]:
" ".join([tokenizer.decode(token) + get_super(str(i)) for i, token in enumerate(encoded_input["input_ids"])])

'[CLS]⁰ Work¹ ##R² ##amp³ is⁴ an⁵ enterprise⁶ learning⁷ platform⁸ for⁹ employee¹⁰ training¹¹ and¹² customer¹³ education¹⁴ at¹⁵ top¹⁶ companies¹⁷ like¹⁸ Zoo¹⁹ ##m²⁰ ,²¹ Box²² ,²³ and²⁴ Work²⁵ ##iva²⁶ .²⁷ The²⁸ company²⁹ is³⁰ committed³¹ to³² transforming³³ the³⁴ future³⁵ of³⁶ work³⁷ by³⁸ delivering³⁹ an⁴⁰ un⁴¹ ##par⁴² ##alle⁴³ ##led⁴⁴ ,⁴⁵ digital⁴⁶ learning⁴⁷ experience⁴⁸ for⁴⁹ employees⁵⁰ and⁵¹ customers⁵² .⁵³ With⁵⁴ Work⁵⁵ ##R⁵⁶ ##amp⁵⁷ ,⁵⁸ users⁵⁹ feel⁶⁰ em⁶¹ ##powered⁶² to⁶³ learn⁶⁴ ,⁶⁵ collaborate⁶⁶ ,⁶⁷ and⁶⁸ ##de⁶⁹ ##vel⁷⁰ ##op⁷¹ their⁷² skills⁷³ in⁷⁴ a⁷⁵ digital⁷⁶ works⁷⁷ ##pace⁷⁸ that⁷⁹ bridges⁸⁰ physical⁸¹ and⁸² virtual⁸³ training⁸⁴ .⁸⁵ The⁸⁶ company⁸⁷ is⁸⁸ building⁸⁹ a⁹⁰ dynamic⁹¹ learning⁹² platform⁹³ that⁹⁴ focuses⁹⁵ on⁹⁶ delivering⁹⁷ training⁹⁸ based⁹⁹ on¹⁰⁰ driving¹⁰¹ business¹⁰² outcomes¹⁰³ .¹⁰⁴ Its¹⁰⁵ platform¹⁰⁶ allows¹⁰⁷ companies¹⁰⁸ to¹⁰⁹ manage¹¹⁰ role¹¹¹ -¹¹² specific¹¹³ training¹¹⁴ and¹¹⁵ increase¹¹⁶ team¹¹⁷ performance¹¹⁸ ,¹¹⁹ while¹²⁰ also¹²¹ supporting¹²² larger

In [ ]:
#NER
start_ner_company = 1
end_ner_company = 3 + 1

#Q1
start_answer = 1
end_answer = 27 + 1

print(tokenizer.decode(encoded_input["input_ids"][start_ner_company:end_ner_company]))
print(tokenizer.decode(encoded_input["input_ids"][start_answer:end_answer]))

WorkRamp
WorkRamp is an enterprise learning platform for employee training and customer education at top companies like Zoom, Box, and Workiva.


In [ ]:
insert_new_labels(sample_id)

NER: {'start': 1, 'end': 4}
Answer: {'start': 1, 'end': 28}
Total Labeling Count: 55


In [ ]:
sample_id = next(data_sample_ids)
print(sample_id)
encoded_input = tokenizer(dict_data[sample_id]['desc'])
dict_data[sample_id]['desc']

57


'Sift applies insights from a global network of data to detect fraud and increase positive user experience.It prevents fraud with industry-leading technology and expertise, an unrivaled global data network, and a commitment to building long-term partnerships with its customers. Twitter, Airbnb, and Twilio rely on Sift to stay competitive andsecure. Sift is known in digital trust and safety, empowering companies of all sizes to unlock revenue without risk.In 2011, Brandon Ballinger and Jason Tan founded Sift in San Francisco, California.Read More'

In [ ]:
" ".join([tokenizer.decode(token) + get_super(str(i)) for i, token in enumerate(encoded_input["input_ids"])])

'[CLS]⁰ Si¹ ##ft² applies³ insights⁴ from⁵ a⁶ global⁷ network⁸ of⁹ data¹⁰ to¹¹ detect¹² fraud¹³ and¹⁴ increase¹⁵ positive¹⁶ user¹⁷ experience¹⁸ .¹⁹ It²⁰ prevents²¹ fraud²² with²³ industry²⁴ -²⁵ leading²⁶ technology²⁷ and²⁸ expertise²⁹ ,³⁰ an³¹ un³² ##ri³³ ##vale³⁴ ##d³⁵ global³⁶ data³⁷ network³⁸ ,³⁹ and⁴⁰ a⁴¹ commitment⁴² to⁴³ building⁴⁴ long⁴⁵ -⁴⁶ term⁴⁷ partnerships⁴⁸ with⁴⁹ its⁵⁰ customers⁵¹ .⁵² Twitter⁵³ ,⁵⁴ Air⁵⁵ ##b⁵⁶ ##n⁵⁷ ##b⁵⁸ ,⁵⁹ and⁶⁰ T⁶¹ ##wi⁶² ##lio⁶³ rely⁶⁴ on⁶⁵ Si⁶⁶ ##ft⁶⁷ to⁶⁸ stay⁶⁹ competitive⁷⁰ and⁷¹ ##se⁷² ##cure⁷³ .⁷⁴ Si⁷⁵ ##ft⁷⁶ is⁷⁷ known⁷⁸ in⁷⁹ digital⁸⁰ trust⁸¹ and⁸² safety⁸³ ,⁸⁴ em⁸⁵ ##powering⁸⁶ companies⁸⁷ of⁸⁸ all⁸⁹ sizes⁹⁰ to⁹¹ unlock⁹² revenue⁹³ without⁹⁴ risk⁹⁵ .⁹⁶ In⁹⁷ 2011⁹⁸ ,⁹⁹ Brandon¹⁰⁰ Ball¹⁰¹ ##inger¹⁰² and¹⁰³ Jason¹⁰⁴ Tan¹⁰⁵ founded¹⁰⁶ Si¹⁰⁷ ##ft¹⁰⁸ in¹⁰⁹ San¹¹⁰ Francisco¹¹¹ ,¹¹² California¹¹³ .¹¹⁴ Read¹¹⁵ More¹¹⁶ [SEP]¹¹⁷'

In [ ]:
#NER
start_ner_company = 1
end_ner_company = 2 + 1

#Q1
start_answer = 1
end_answer = 19 + 1

print(tokenizer.decode(encoded_input["input_ids"][start_ner_company:end_ner_company]))
print(tokenizer.decode(encoded_input["input_ids"][start_answer:end_answer]))

Sift
Sift applies insights from a global network of data to detect fraud and increase positive user experience.


In [ ]:
insert_new_labels(sample_id)

NER: {'start': 1, 'end': 3}
Answer: {'start': 1, 'end': 20}
Total Labeling Count: 56


In [ ]:
sample_id = next(data_sample_ids)
print(sample_id)
encoded_input = tokenizer(dict_data[sample_id]['desc'])
dict_data[sample_id]['desc']

382


"Aidoc is a provider of artificial intelligence healthcare solutions that empower physicians to expedite patient treatment and enhance efficiencies. Aidoc's AI-driven solutions analyze medical images directly after the patient is scanned, suggesting prioritization of time-sensitive pathologies, as well as notifying and activatingmultidisciplinary teams to reduce turnaround time, shorten length of stay and improve overall patient outcomes.Aidoc was founded in 2016 and is based in Tel Aviv, Israel.Read More"

In [ ]:
" ".join([tokenizer.decode(token) + get_super(str(i)) for i, token in enumerate(encoded_input["input_ids"])])

"[CLS]⁰ Aid¹ ##oc² is³ a⁴ provider⁵ of⁶ artificial⁷ intelligence⁸ healthcare⁹ solutions¹⁰ that¹¹ em¹² ##power¹³ physicians¹⁴ to¹⁵ ex¹⁶ ##ped¹⁷ ##ite¹⁸ patient¹⁹ treatment²⁰ and²¹ enhance²² e²³ ##ff²⁴ ##ici²⁵ ##encies²⁶ .²⁷ Aid²⁸ ##oc²⁹ '³⁰ s³¹ AI³² -³³ driven³⁴ solutions³⁵ analyze³⁶ medical³⁷ images³⁸ directly³⁹ after⁴⁰ the⁴¹ patient⁴² is⁴³ scanned⁴⁴ ,⁴⁵ suggesting⁴⁶ prior⁴⁷ ##iti⁴⁸ ##zation⁴⁹ of⁵⁰ time⁵¹ -⁵² sensitive⁵³ path⁵⁴ ##ologies⁵⁵ ,⁵⁶ as⁵⁷ well⁵⁸ as⁵⁹ not⁶⁰ ##ifying⁶¹ and⁶² act⁶³ ##ivating⁶⁴ ##mu⁶⁵ ##lt⁶⁶ ##id⁶⁷ ##is⁶⁸ ##ci⁶⁹ ##p⁷⁰ ##linary⁷¹ teams⁷² to⁷³ reduce⁷⁴ turn⁷⁵ ##around⁷⁶ time⁷⁷ ,⁷⁸ short⁷⁹ ##en⁸⁰ length⁸¹ of⁸² stay⁸³ and⁸⁴ improve⁸⁵ overall⁸⁶ patient⁸⁷ outcomes⁸⁸ .⁸⁹ Aid⁹⁰ ##oc⁹¹ was⁹² founded⁹³ in⁹⁴ 2016⁹⁵ and⁹⁶ is⁹⁷ based⁹⁸ in⁹⁹ Tel¹⁰⁰ Aviv¹⁰¹ ,¹⁰² Israel¹⁰³ .¹⁰⁴ Read¹⁰⁵ More¹⁰⁶ [SEP]¹⁰⁷"

In [ ]:
#NER
start_ner_company = 1
end_ner_company = 2 + 1

#Q1
start_answer = 1
end_answer = 27 + 1

print(tokenizer.decode(encoded_input["input_ids"][start_ner_company:end_ner_company]))
print(tokenizer.decode(encoded_input["input_ids"][start_answer:end_answer]))

Aidoc
Aidoc is a provider of artificial intelligence healthcare solutions that empower physicians to expedite patient treatment and enhance efficiencies.


In [ ]:
insert_new_labels(sample_id)

NER: {'start': 1, 'end': 3}
Answer: {'start': 1, 'end': 28}
Total Labeling Count: 57


In [ ]:
sample_id = next(data_sample_ids)
print(sample_id)
encoded_input = tokenizer(dict_data[sample_id]['desc'])
dict_data[sample_id]['desc']

2947


' We enable people to find the food they love, enjoy every meal, and get healthier. In addition, we equip restaurants with the ability to delight their customers.'

In [ ]:
" ".join([tokenizer.decode(token) + get_super(str(i)) for i, token in enumerate(encoded_input["input_ids"])])

'[CLS]⁰ We¹ enable² people³ to⁴ find⁵ the⁶ food⁷ they⁸ love⁹ ,¹⁰ enjoy¹¹ every¹² meal¹³ ,¹⁴ and¹⁵ get¹⁶ health¹⁷ ##ier¹⁸ .¹⁹ In²⁰ addition²¹ ,²² we²³ e²⁴ ##qui²⁵ ##p²⁶ restaurants²⁷ with²⁸ the²⁹ ability³⁰ to³¹ delight³² their³³ customers³⁴ .³⁵ [SEP]³⁶'

In [ ]:
#Q1
start_answer = 23
end_answer = 35 + 1

print(tokenizer.decode(encoded_input["input_ids"][start_answer:end_answer]))

we equip restaurants with the ability to delight their customers.


In [ ]:
insert_new_labels(sample_id, skip_ner=True)

Answer: {'start': 23, 'end': 36}
Total Labeling Count: 58


In [ ]:
sample_id = next(data_sample_ids)
print(sample_id)
encoded_input = tokenizer(dict_data[sample_id]['desc'])
dict_data[sample_id]['desc']

3441


'Bitdeal - Leading Enterprise Blockchain Solutions Provider offers A to Z blockchain-based services & solutions to 20+ industries. We have 200+ blockchain experts who are skilled in all new technologies and trends such as Blockchain, Cryptocurrency, and NFT.\nSome of the key services of Bitdeal includes cryptocurrency exchange development,private blockchain development, defi development, NFT development and so on.\nWe have delivered quality outputs via which we have 400+ happy clients across the globe.\nIn 2015, we have just started our journey by offering feature-rich cryptocurrency exchange script and now we have developed ourselves to develop metaverse platforms with 3D VR & AR technologies.\nSome of the industries we serve our service involves,\nBanking\nGovernment\nCrypto Exchange\nGaming\nFinance\nReal Estate\nEducation\nEcommerce and more.\nSome of our crypto & blockchain solutions areBlockchain Development Services\nP2P Exchange Development\nInitial Coin Offering Services\nSmar

In [ ]:
" ".join([tokenizer.decode(token) + get_super(str(i)) for i, token in enumerate(encoded_input["input_ids"])])

'[CLS]⁰ Bit¹ ##dea² ##l³ -⁴ Leading⁵ Enterprise⁶ Block⁷ ##cha⁸ ##in⁹ Solutions¹⁰ Pro¹¹ ##vide¹² ##r¹³ offers¹⁴ A¹⁵ to¹⁶ Z¹⁷ block¹⁸ ##cha¹⁹ ##in²⁰ -²¹ based²² services²³ &²⁴ solutions²⁵ to²⁶ 20²⁷ +²⁸ industries²⁹ .³⁰ We³¹ have³² 200³³ +³⁴ block³⁵ ##cha³⁶ ##in³⁷ experts³⁸ who³⁹ are⁴⁰ skilled⁴¹ in⁴² all⁴³ new⁴⁴ technologies⁴⁵ and⁴⁶ trends⁴⁷ such⁴⁸ as⁴⁹ Block⁵⁰ ##cha⁵¹ ##in⁵² ,⁵³ Cry⁵⁴ ##pt⁵⁵ ##oc⁵⁶ ##ur⁵⁷ ##ren⁵⁸ ##cy⁵⁹ ,⁶⁰ and⁶¹ N⁶² ##FT⁶³ .⁶⁴ Some⁶⁵ of⁶⁶ the⁶⁷ key⁶⁸ services⁶⁹ of⁷⁰ Bit⁷¹ ##dea⁷² ##l⁷³ includes⁷⁴ cry⁷⁵ ##pt⁷⁶ ##oc⁷⁷ ##ur⁷⁸ ##ren⁷⁹ ##cy⁸⁰ exchange⁸¹ development⁸² ,⁸³ private⁸⁴ block⁸⁵ ##cha⁸⁶ ##in⁸⁷ development⁸⁸ ,⁸⁹ def⁹⁰ ##i⁹¹ development⁹² ,⁹³ N⁹⁴ ##FT⁹⁵ development⁹⁶ and⁹⁷ so⁹⁸ on⁹⁹ .¹⁰⁰ We¹⁰¹ have¹⁰² delivered¹⁰³ quality¹⁰⁴ output¹⁰⁵ ##s¹⁰⁶ via¹⁰⁷ which¹⁰⁸ we¹⁰⁹ have¹¹⁰ 400¹¹¹ +¹¹² happy¹¹³ clients¹¹⁴ across¹¹⁵ the¹¹⁶ globe¹¹⁷ .¹¹⁸ In¹¹⁹ 2015¹²⁰ ,¹²¹ we¹²² have¹²³ just¹²⁴ started¹²⁵ our¹²⁶ journey¹²⁷ by¹²⁸ offering¹²⁹ feature¹³⁰ -¹³¹ rich¹³² cry¹³³ ##pt¹³⁴ ##oc¹³⁵ #

In [ ]:
#NER
start_ner_company = 1
end_ner_company = 3 + 1

#Q1
start_answer = 1
end_answer = 30 + 1

print(tokenizer.decode(encoded_input["input_ids"][start_ner_company:end_ner_company]))
print(tokenizer.decode(encoded_input["input_ids"][start_answer:end_answer]))

Bitdeal
Bitdeal - Leading Enterprise Blockchain Solutions Provider offers A to Z blockchain - based services & solutions to 20 + industries.


In [ ]:
insert_new_labels(sample_id)

NER: {'start': 1, 'end': 4}
Answer: {'start': 1, 'end': 31}
Total Labeling Count: 59


In [ ]:
sample_id = next(data_sample_ids)
print(sample_id)
encoded_input = tokenizer(dict_data[sample_id]['desc'])
dict_data[sample_id]['desc']

1633


'Hoy Health is combining the best of technology and  human interaction in a model that allows consumers to navigate their health and wellness journey in a guided manner allowing them to achieve better healthcare outcomes. Hoy Health offers consumers with medication access and chronic condition managment programs allowing them to save on theirprimary care healthcare costs across the USA, Mexico, Central America and the CaribbeanRead More'

In [ ]:
" ".join([tokenizer.decode(token) + get_super(str(i)) for i, token in enumerate(encoded_input["input_ids"])])

'[CLS]⁰ Ho¹ ##y² Health³ is⁴ combining⁵ the⁶ best⁷ of⁸ technology⁹ and¹⁰ human¹¹ interaction¹² in¹³ a¹⁴ model¹⁵ that¹⁶ allows¹⁷ consumers¹⁸ to¹⁹ navigate²⁰ their²¹ health²² and²³ well²⁴ ##ness²⁵ journey²⁶ in²⁷ a²⁸ guided²⁹ manner³⁰ allowing³¹ them³² to³³ achieve³⁴ better³⁵ healthcare³⁶ outcomes³⁷ .³⁸ Ho³⁹ ##y⁴⁰ Health⁴¹ offers⁴² consumers⁴³ with⁴⁴ medication⁴⁵ access⁴⁶ and⁴⁷ chronic⁴⁸ condition⁴⁹ man⁵⁰ ##ag⁵¹ ##ment⁵² programs⁵³ allowing⁵⁴ them⁵⁵ to⁵⁶ save⁵⁷ on⁵⁸ their⁵⁹ ##p⁶⁰ ##rima⁶¹ ##ry⁶² care⁶³ healthcare⁶⁴ costs⁶⁵ across⁶⁶ the⁶⁷ USA⁶⁸ ,⁶⁹ Mexico⁷⁰ ,⁷¹ Central⁷² America⁷³ and⁷⁴ the⁷⁵ Caribbean⁷⁶ ##R⁷⁷ ##ead⁷⁸ More⁷⁹ [SEP]⁸⁰'

In [ ]:
#NER
start_ner_company = 1
end_ner_company = 3 + 1

#Q1
start_answer = 1
end_answer = 38 + 1

print(tokenizer.decode(encoded_input["input_ids"][start_ner_company:end_ner_company]))
print(tokenizer.decode(encoded_input["input_ids"][start_answer:end_answer]))

Hoy Health
Hoy Health is combining the best of technology and human interaction in a model that allows consumers to navigate their health and wellness journey in a guided manner allowing them to achieve better healthcare outcomes.


In [ ]:
insert_new_labels(sample_id)

NER: {'start': 1, 'end': 4}
Answer: {'start': 1, 'end': 39}
Total Labeling Count: 60


In [ ]:
sample_id = next(data_sample_ids)
print(sample_id)
encoded_input = tokenizer(dict_data[sample_id]['desc'])
dict_data[sample_id]['desc']

2909


'Perfect Corp. is dedicated to transforming how consumers, content creators and beauty brands interact together. Their unique platform is the premier online destination for all beauty lovers. Their apps, YouCam Makeup, recently voted Best Fashion & Apparels App* of 2015, and market-leading YouCam Perfect redefine virtual beauty for tens ofmillions of users. Their experienced team of engineers and beauty aficionados is pushing the frontiers of technology to create the beauty platform of the future – a fluid environment where individuals express themselves, learn the latest about fashion and beauty, and enjoy instant access to the products from their favorite brands.*YouCam Makeup is the winner of 2015 Appy Award of Best Fashion & Apparels App.Read More'

In [ ]:
" ".join([tokenizer.decode(token) + get_super(str(i)) for i, token in enumerate(encoded_input["input_ids"])])

'[CLS]⁰ Perfect¹ Corp² .³ is⁴ dedicated⁵ to⁶ transforming⁷ how⁸ consumers⁹ ,¹⁰ content¹¹ creators¹² and¹³ beauty¹⁴ brands¹⁵ interact¹⁶ together¹⁷ .¹⁸ Their¹⁹ unique²⁰ platform²¹ is²² the²³ premier²⁴ online²⁵ destination²⁶ for²⁷ all²⁸ beauty²⁹ lovers³⁰ .³¹ Their³² apps³³ ,³⁴ You³⁵ ##C³⁶ ##am³⁷ Make³⁸ ##up³⁹ ,⁴⁰ recently⁴¹ voted⁴² Best⁴³ Fashion⁴⁴ &⁴⁵ A⁴⁶ ##ppa⁴⁷ ##rel⁴⁸ ##s⁴⁹ A⁵⁰ ##pp⁵¹ *⁵² of⁵³ 2015⁵⁴ ,⁵⁵ and⁵⁶ market⁵⁷ -⁵⁸ leading⁵⁹ You⁶⁰ ##C⁶¹ ##am⁶² Perfect⁶³ red⁶⁴ ##ef⁶⁵ ##ine⁶⁶ virtual⁶⁷ beauty⁶⁸ for⁶⁹ tens⁷⁰ of⁷¹ ##mill⁷² ##ions⁷³ of⁷⁴ users⁷⁵ .⁷⁶ Their⁷⁷ experienced⁷⁸ team⁷⁹ of⁸⁰ engineers⁸¹ and⁸² beauty⁸³ a⁸⁴ ##fic⁸⁵ ##ion⁸⁶ ##ado⁸⁷ ##s⁸⁸ is⁸⁹ pushing⁹⁰ the⁹¹ frontier⁹² ##s⁹³ of⁹⁴ technology⁹⁵ to⁹⁶ create⁹⁷ the⁹⁸ beauty⁹⁹ platform¹⁰⁰ of¹⁰¹ the¹⁰² future¹⁰³ –¹⁰⁴ a¹⁰⁵ fluid¹⁰⁶ environment¹⁰⁷ where¹⁰⁸ individuals¹⁰⁹ express¹¹⁰ themselves¹¹¹ ,¹¹² learn¹¹³ the¹¹⁴ latest¹¹⁵ about¹¹⁶ fashion¹¹⁷ and¹¹⁸ beauty¹¹⁹ ,¹²⁰ and¹²¹ enjoy¹²² instant¹²³ access¹²⁴ to¹²⁵ the¹²⁶ products¹²⁷ from¹²⁸

In [ ]:
#NER
start_ner_company = 1
end_ner_company = 3 + 1

#Q1
start_answer = 1
end_answer = 18 + 1

print(tokenizer.decode(encoded_input["input_ids"][start_ner_company:end_ner_company]))
print(tokenizer.decode(encoded_input["input_ids"][start_answer:end_answer]))

Perfect Corp.
Perfect Corp. is dedicated to transforming how consumers, content creators and beauty brands interact together.


In [ ]:
insert_new_labels(sample_id)

NER: {'start': 1, 'end': 4}
Answer: {'start': 1, 'end': 19}
Total Labeling Count: 61


In [ ]:
sample_id = next(data_sample_ids)
print(sample_id)
encoded_input = tokenizer(dict_data[sample_id]['desc'])
dict_data[sample_id]['desc']

3378


'As the UK’s first internet pharmacy, established in 1999, Pharmacy2U has played a pioneering role in bringing about these changes and in shaping the health agenda.It is now the country’s largest dedicated internet and mail order pharmacy, serving the growing number of people who are unable or unwilling to visit high street pharmacies. It allowsprescriptions, over the counter medicines and personal care products to be delivered directly to your doorstep without having to leave the comfort of your own home – and many at prices lower than the high street.Like traditional pharmacies, Pharmacy2U can offer patients expert advice. However, its investment in technology has also allowed it to introduce new and innovative services, such as electronic requesting of repeat prescriptions, the unique repeat prescription reminder service and prescription synchronization that ensures patients are ordering and taking their medicines appropriately.Pharmacy2U is run by pharmacists and operates to the hi

In [ ]:
" ".join([tokenizer.decode(token) + get_super(str(i)) for i, token in enumerate(encoded_input["input_ids"])])

'[CLS]⁰ As¹ the² UK³ ’⁴ s⁵ first⁶ internet⁷ pharmacy⁸ ,⁹ established¹⁰ in¹¹ 1999¹² ,¹³ Pharmacy¹⁴ ##2¹⁵ ##U¹⁶ has¹⁷ played¹⁸ a¹⁹ pioneering²⁰ role²¹ in²² bringing²³ about²⁴ these²⁵ changes²⁶ and²⁷ in²⁸ shaping²⁹ the³⁰ health³¹ agenda³² .³³ It³⁴ is³⁵ now³⁶ the³⁷ country³⁸ ’³⁹ s⁴⁰ largest⁴¹ dedicated⁴² internet⁴³ and⁴⁴ mail⁴⁵ order⁴⁶ pharmacy⁴⁷ ,⁴⁸ serving⁴⁹ the⁵⁰ growing⁵¹ number⁵² of⁵³ people⁵⁴ who⁵⁵ are⁵⁶ unable⁵⁷ or⁵⁸ unwilling⁵⁹ to⁶⁰ visit⁶¹ high⁶² street⁶³ p⁶⁴ ##har⁶⁵ ##ma⁶⁶ ##cies⁶⁷ .⁶⁸ It⁶⁹ allows⁷⁰ ##p⁷¹ ##res⁷² ##cription⁷³ ##s⁷⁴ ,⁷⁵ over⁷⁶ the⁷⁷ counter⁷⁸ medicines⁷⁹ and⁸⁰ personal⁸¹ care⁸² products⁸³ to⁸⁴ be⁸⁵ delivered⁸⁶ directly⁸⁷ to⁸⁸ your⁸⁹ doors⁹⁰ ##tep⁹¹ without⁹² having⁹³ to⁹⁴ leave⁹⁵ the⁹⁶ comfort⁹⁷ of⁹⁸ your⁹⁹ own¹⁰⁰ home¹⁰¹ –¹⁰² and¹⁰³ many¹⁰⁴ at¹⁰⁵ prices¹⁰⁶ lower¹⁰⁷ than¹⁰⁸ the¹⁰⁹ high¹¹⁰ street¹¹¹ .¹¹² Like¹¹³ traditional¹¹⁴ p¹¹⁵ ##har¹¹⁶ ##ma¹¹⁷ ##cies¹¹⁸ ,¹¹⁹ Pharmacy¹²⁰ ##2¹²¹ ##U¹²² can¹²³ offer¹²⁴ patients¹²⁵ expert¹²⁶ advice¹²⁷ .¹²⁸ However¹²⁹ ,¹³⁰ its¹³¹ i

In [ ]:
#NER
start_ner_company = 14
end_ner_company = 14 + 1

#Q1
start_answer = 34
end_answer = 68 + 1

print(tokenizer.decode(encoded_input["input_ids"][start_ner_company:end_ner_company]))
print(tokenizer.decode(encoded_input["input_ids"][start_answer:end_answer]))

Pharmacy
It is now the country ’ s largest dedicated internet and mail order pharmacy, serving the growing number of people who are unable or unwilling to visit high street pharmacies.


In [ ]:
insert_new_labels(sample_id)

NER: {'start': 14, 'end': 15}
Answer: {'start': 34, 'end': 69}
Total Labeling Count: 62


In [ ]:
sample_id = next(data_sample_ids)
print(sample_id)
encoded_input = tokenizer(dict_data[sample_id]['desc'])
dict_data[sample_id]['desc']

3215


"Convin is a conversation intelligence platform that helps your remote sales team to sell more by understanding what's working on calls and replicating best performing sales reps. Convin also helps businesses develop their personalized and the most relevant sales playbook that new employees can utilize for learning purposes."

In [ ]:
" ".join([tokenizer.decode(token) + get_super(str(i)) for i, token in enumerate(encoded_input["input_ids"])])

"[CLS]⁰ Con¹ ##vin² is³ a⁴ conversation⁵ intelligence⁶ platform⁷ that⁸ helps⁹ your¹⁰ remote¹¹ sales¹² team¹³ to¹⁴ sell¹⁵ more¹⁶ by¹⁷ understanding¹⁸ what¹⁹ '²⁰ s²¹ working²² on²³ calls²⁴ and²⁵ replica²⁶ ##ting²⁷ best²⁸ performing²⁹ sales³⁰ re³¹ ##ps³² .³³ Con³⁴ ##vin³⁵ also³⁶ helps³⁷ businesses³⁸ develop³⁹ their⁴⁰ personal⁴¹ ##ized⁴² and⁴³ the⁴⁴ most⁴⁵ relevant⁴⁶ sales⁴⁷ play⁴⁸ ##book⁴⁹ that⁵⁰ new⁵¹ employees⁵² can⁵³ utilize⁵⁴ for⁵⁵ learning⁵⁶ purposes⁵⁷ .⁵⁸ [SEP]⁵⁹"

In [ ]:
#NER
start_ner_company = 1
end_ner_company = 2 + 1

#Q1
start_answer = 1
end_answer = 33 + 1

print(tokenizer.decode(encoded_input["input_ids"][start_ner_company:end_ner_company]))
print(tokenizer.decode(encoded_input["input_ids"][start_answer:end_answer]))

Convin
Convin is a conversation intelligence platform that helps your remote sales team to sell more by understanding what's working on calls and replicating best performing sales reps.


In [ ]:
insert_new_labels(sample_id)

NER: {'start': 1, 'end': 3}
Answer: {'start': 1, 'end': 34}
Total Labeling Count: 63


In [ ]:
sample_id = next(data_sample_ids)
print(sample_id)
encoded_input = tokenizer(dict_data[sample_id]['desc'])
dict_data[sample_id]['desc']

2736


'Lucerna they are focused on developing and commercializing the fluorescent aptamer, providing plug-and-play RNA imaging and detection systems as research reagents, developing screening platforms to enable drug discovery of previous intractable targets, developing fluorescent sensors for industrial production applications.'

In [ ]:
" ".join([tokenizer.decode(token) + get_super(str(i)) for i, token in enumerate(encoded_input["input_ids"])])

'[CLS]⁰ Luce¹ ##rna² they³ are⁴ focused⁵ on⁶ developing⁷ and⁸ commercial⁹ ##izing¹⁰ the¹¹ fluorescent¹² a¹³ ##pta¹⁴ ##mer¹⁵ ,¹⁶ providing¹⁷ plug¹⁸ -¹⁹ and²⁰ -²¹ play²² RNA²³ imaging²⁴ and²⁵ detection²⁶ systems²⁷ as²⁸ research²⁹ re³⁰ ##age³¹ ##nts³² ,³³ developing³⁴ screening³⁵ platforms³⁶ to³⁷ enable³⁸ drug³⁹ discovery⁴⁰ of⁴¹ previous⁴² in⁴³ ##tract⁴⁴ ##able⁴⁵ targets⁴⁶ ,⁴⁷ developing⁴⁸ fluorescent⁴⁹ sensors⁵⁰ for⁵¹ industrial⁵² production⁵³ applications⁵⁴ .⁵⁵ [SEP]⁵⁶'

In [ ]:
#NER
start_ner_company = 1
end_ner_company = 2 + 1

#Q1
start_answer = 1
end_answer = 16 + 1

print(tokenizer.decode(encoded_input["input_ids"][start_ner_company:end_ner_company]))
print(tokenizer.decode(encoded_input["input_ids"][start_answer:end_answer]))

Lucerna
Lucerna they are focused on developing and commercializing the fluorescent aptamer,


In [ ]:
insert_new_labels(sample_id)

NER: {'start': 1, 'end': 3}
Answer: {'start': 1, 'end': 17}
Total Labeling Count: 64


In [ ]:
sample_id = next(data_sample_ids)
print(sample_id)
encoded_input = tokenizer(dict_data[sample_id]['desc'])
dict_data[sample_id]['desc']

2561


'Justos is disrupting the insurance market in Latin America.'

In [ ]:
" ".join([tokenizer.decode(token) + get_super(str(i)) for i, token in enumerate(encoded_input["input_ids"])])

'[CLS]⁰ Just¹ ##os² is³ disrupt⁴ ##ing⁵ the⁶ insurance⁷ market⁸ in⁹ Latin¹⁰ America¹¹ .¹² [SEP]¹³'

In [ ]:
#NER
start_ner_company = 1
end_ner_company = 2 + 1

#Q1
start_answer = 1
end_answer = 12 + 1

print(tokenizer.decode(encoded_input["input_ids"][start_ner_company:end_ner_company]))
print(tokenizer.decode(encoded_input["input_ids"][start_answer:end_answer]))

Justos
Justos is disrupting the insurance market in Latin America.


In [ ]:
insert_new_labels(sample_id)

NER: {'start': 1, 'end': 3}
Answer: {'start': 1, 'end': 13}
Total Labeling Count: 65


In [ ]:
sample_id = next(data_sample_ids)
print(sample_id)
encoded_input = tokenizer(dict_data[sample_id]['desc'])
dict_data[sample_id]['desc']

4


'Shift Technology delivers AI-native decision automation and optimization solutions built specifically for the global insurance industry. Addressing several critical processes across the insurance policy lifecycle, Shift helps insurers achieve faster, more accurate claims and policy resolutions. Shift has analyzed billions of insurance transactionsto date and was presented Frost & Sullivan’s 2020 Global Claims Solutions for Insurance Market Leadership Award.Read More'

In [ ]:
" ".join([tokenizer.decode(token) + get_super(str(i)) for i, token in enumerate(encoded_input["input_ids"])])

'[CLS]⁰ Shi¹ ##ft² Technology³ delivers⁴ AI⁵ -⁶ native⁷ decision⁸ automation⁹ and¹⁰ optimization¹¹ solutions¹² built¹³ specifically¹⁴ for¹⁵ the¹⁶ global¹⁷ insurance¹⁸ industry¹⁹ .²⁰ Ad²¹ ##dress²² ##ing²³ several²⁴ critical²⁵ processes²⁶ across²⁷ the²⁸ insurance²⁹ policy³⁰ life³¹ ##cycle³² ,³³ Shi³⁴ ##ft³⁵ helps³⁶ ins³⁷ ##urers³⁸ achieve³⁹ faster⁴⁰ ,⁴¹ more⁴² accurate⁴³ claims⁴⁴ and⁴⁵ policy⁴⁶ resolutions⁴⁷ .⁴⁸ Shi⁴⁹ ##ft⁵⁰ has⁵¹ analyzed⁵² billion⁵³ ##s⁵⁴ of⁵⁵ insurance⁵⁶ transactions⁵⁷ ##to⁵⁸ date⁵⁹ and⁶⁰ was⁶¹ presented⁶² Frost⁶³ &⁶⁴ Sullivan⁶⁵ ’⁶⁶ s⁶⁷ 2020⁶⁸ Global⁶⁹ C⁷⁰ ##lai⁷¹ ##ms⁷² Solutions⁷³ for⁷⁴ Insurance⁷⁵ Market⁷⁶ Leadership⁷⁷ Award⁷⁸ .⁷⁹ Read⁸⁰ More⁸¹ [SEP]⁸²'

In [ ]:
#NER
start_ner_company = 1
end_ner_company = 3 + 1

#Q1
start_answer = 1
end_answer = 20 + 1

print(tokenizer.decode(encoded_input["input_ids"][start_ner_company:end_ner_company]))
print(tokenizer.decode(encoded_input["input_ids"][start_answer:end_answer]))

Shift Technology
Shift Technology delivers AI - native decision automation and optimization solutions built specifically for the global insurance industry.


In [ ]:
insert_new_labels(sample_id)

NER: {'start': 1, 'end': 4}
Answer: {'start': 1, 'end': 21}
Total Labeling Count: 66


In [ ]:
sample_id = next(data_sample_ids)
print(sample_id)
encoded_input = tokenizer(dict_data[sample_id]['desc'])
dict_data[sample_id]['desc']

2506


'Using proprietary algorithms and proven AI, Carpe Data harnesses the power of emerging and alternative data for insurance carriers around the globe. Utilizing Carpe Data’s data, insurers gain deeper insight into risks and significantly enhance many aspects of the insurance life cycle, including underwriting, claims, and book assessment.'

In [ ]:
" ".join([tokenizer.decode(token) + get_super(str(i)) for i, token in enumerate(encoded_input["input_ids"])])

'[CLS]⁰ Using¹ proprietary² algorithms³ and⁴ proven⁵ AI⁶ ,⁷ Car⁸ ##pe⁹ Data¹⁰ harness¹¹ ##es¹² the¹³ power¹⁴ of¹⁵ emerging¹⁶ and¹⁷ alternative¹⁸ data¹⁹ for²⁰ insurance²¹ carriers²² around²³ the²⁴ globe²⁵ .²⁶ U²⁷ ##til²⁸ ##izing²⁹ Car³⁰ ##pe³¹ Data³² ’³³ s³⁴ data³⁵ ,³⁶ ins³⁷ ##urers³⁸ gain³⁹ deeper⁴⁰ insight⁴¹ into⁴² risks⁴³ and⁴⁴ significantly⁴⁵ enhance⁴⁶ many⁴⁷ aspects⁴⁸ of⁴⁹ the⁵⁰ insurance⁵¹ life⁵² cycle⁵³ ,⁵⁴ including⁵⁵ under⁵⁶ ##writing⁵⁷ ,⁵⁸ claims⁵⁹ ,⁶⁰ and⁶¹ book⁶² assessment⁶³ .⁶⁴ [SEP]⁶⁵'

In [ ]:
#NER
start_ner_company = 8
end_ner_company = 10 + 1

#Q1
start_answer = 8
end_answer = 26 + 1

print(tokenizer.decode(encoded_input["input_ids"][start_ner_company:end_ner_company]))
print(tokenizer.decode(encoded_input["input_ids"][start_answer:end_answer]))

Carpe Data
Carpe Data harnesses the power of emerging and alternative data for insurance carriers around the globe.


In [ ]:
insert_new_labels(sample_id)

NER: {'start': 8, 'end': 11}
Answer: {'start': 8, 'end': 27}
Total Labeling Count: 67


In [ ]:
sample_id = next(data_sample_ids)
print(sample_id)
encoded_input = tokenizer(dict_data[sample_id]['desc'])
dict_data[sample_id]['desc']

2021


'Medically Home is a tech-enabled clinical enterprise that provides all the necessary capabilities to safely-shift medical care from hospitals to patients’ homes. Its platform offers a virtual hospital program that connects patients with caregivers and supports the monitoring of, and communication with patients with high acuity illness that helppatients and their families to access a medical command center that provides centralized, on-demand acute medical care management from the comfort of their homes.The company was founded in 2016 and is headquartered in Boston, Massachusetts.Read More'

In [ ]:
" ".join([tokenizer.decode(token) + get_super(str(i)) for i, token in enumerate(encoded_input["input_ids"])])

'[CLS]⁰ Medical¹ ##ly² Home³ is⁴ a⁵ tech⁶ -⁷ enabled⁸ clinical⁹ enterprise¹⁰ that¹¹ provides¹² all¹³ the¹⁴ necessary¹⁵ capabilities¹⁶ to¹⁷ safely¹⁸ -¹⁹ shift²⁰ medical²¹ care²² from²³ hospitals²⁴ to²⁵ patients²⁶ ’²⁷ homes²⁸ .²⁹ Its³⁰ platform³¹ offers³² a³³ virtual³⁴ hospital³⁵ program³⁶ that³⁷ connects³⁸ patients³⁹ with⁴⁰ care⁴¹ ##gi⁴² ##vers⁴³ and⁴⁴ supports⁴⁵ the⁴⁶ monitoring⁴⁷ of⁴⁸ ,⁴⁹ and⁵⁰ communication⁵¹ with⁵² patients⁵³ with⁵⁴ high⁵⁵ a⁵⁶ ##cu⁵⁷ ##ity⁵⁸ illness⁵⁹ that⁶⁰ help⁶¹ ##patient⁶² ##s⁶³ and⁶⁴ their⁶⁵ families⁶⁶ to⁶⁷ access⁶⁸ a⁶⁹ medical⁷⁰ command⁷¹ center⁷² that⁷³ provides⁷⁴ centralized⁷⁵ ,⁷⁶ on⁷⁷ -⁷⁸ demand⁷⁹ acute⁸⁰ medical⁸¹ care⁸² management⁸³ from⁸⁴ the⁸⁵ comfort⁸⁶ of⁸⁷ their⁸⁸ homes⁸⁹ .⁹⁰ The⁹¹ company⁹² was⁹³ founded⁹⁴ in⁹⁵ 2016⁹⁶ and⁹⁷ is⁹⁸ headquartered⁹⁹ in¹⁰⁰ Boston¹⁰¹ ,¹⁰² Massachusetts¹⁰³ .¹⁰⁴ Read¹⁰⁵ More¹⁰⁶ [SEP]¹⁰⁷'

In [ ]:
#NER
start_ner_company = 1
end_ner_company = 3 + 1

#Q1
start_answer = 1
end_answer = 29 + 1

print(tokenizer.decode(encoded_input["input_ids"][start_ner_company:end_ner_company]))
print(tokenizer.decode(encoded_input["input_ids"][start_answer:end_answer]))

Medically Home
Medically Home is a tech - enabled clinical enterprise that provides all the necessary capabilities to safely - shift medical care from hospitals to patients ’ homes.


In [ ]:
insert_new_labels(sample_id)

NER: {'start': 1, 'end': 4}
Answer: {'start': 1, 'end': 30}
Total Labeling Count: 68


In [ ]:
sample_id = next(data_sample_ids)
print(sample_id)
encoded_input = tokenizer(dict_data[sample_id]['desc'])
dict_data[sample_id]['desc']

3391


'2Morrow is a pioneer in providing science-based, digital health solutions. Founded in 2012 and based in Washington state, 2Morrow focuses on evidence-based programs that combine technology and behavioral science to address some of today’s largest healthcare issues including smoking, vaping, weight, stress/anxiety and chronic pain. Thisinnovative approach grew out of a collaboration with researchers at Fred Hutchinson Cancer Research Center in Seattle and incorporates Acceptance and Commitment Therapy (ACT). 2Morrow currently works with employers, retailers, states, health plans, providers, and public health organizations, placing programs directly into the hands of people who need them most via the convenience and privacy of their smartphones. Next up: 2Morrow continues to dive deeper into healthcare with a focus on integrated care as we work with providers and payers to help improve the lives of patients.www.2morrowinc.comRead More'

In [ ]:
" ".join([tokenizer.decode(token) + get_super(str(i)) for i, token in enumerate(encoded_input["input_ids"])])

'[CLS]⁰ 2¹ ##M² ##or³ ##row⁴ is⁵ a⁶ pioneer⁷ in⁸ providing⁹ science¹⁰ -¹¹ based¹² ,¹³ digital¹⁴ health¹⁵ solutions¹⁶ .¹⁷ Founded¹⁸ in¹⁹ 2012²⁰ and²¹ based²² in²³ Washington²⁴ state²⁵ ,²⁶ 2²⁷ ##M²⁸ ##or²⁹ ##row³⁰ focuses³¹ on³² evidence³³ -³⁴ based³⁵ programs³⁶ that³⁷ combine³⁸ technology³⁹ and⁴⁰ behavioral⁴¹ science⁴² to⁴³ address⁴⁴ some⁴⁵ of⁴⁶ today⁴⁷ ’⁴⁸ s⁴⁹ largest⁵⁰ healthcare⁵¹ issues⁵² including⁵³ smoking⁵⁴ ,⁵⁵ v⁵⁶ ##ap⁵⁷ ##ing⁵⁸ ,⁵⁹ weight⁶⁰ ,⁶¹ stress⁶² /⁶³ anxiety⁶⁴ and⁶⁵ chronic⁶⁶ pain⁶⁷ .⁶⁸ This⁶⁹ ##inn⁷⁰ ##ova⁷¹ ##tive⁷² approach⁷³ grew⁷⁴ out⁷⁵ of⁷⁶ a⁷⁷ collaboration⁷⁸ with⁷⁹ researchers⁸⁰ at⁸¹ Fred⁸² Hutchinson⁸³ Cancer⁸⁴ Research⁸⁵ Center⁸⁶ in⁸⁷ Seattle⁸⁸ and⁸⁹ incorporates⁹⁰ A⁹¹ ##cc⁹² ##ept⁹³ ##ance⁹⁴ and⁹⁵ Co⁹⁶ ##mm⁹⁷ ##it⁹⁸ ##ment⁹⁹ Therapy¹⁰⁰ (¹⁰¹ ACT¹⁰² )¹⁰³ .¹⁰⁴ 2¹⁰⁵ ##M¹⁰⁶ ##or¹⁰⁷ ##row¹⁰⁸ currently¹⁰⁹ works¹¹⁰ with¹¹¹ employers¹¹² ,¹¹³ retailers¹¹⁴ ,¹¹⁵ states¹¹⁶ ,¹¹⁷ health¹¹⁸ plans¹¹⁹ ,¹²⁰ providers¹²¹ ,¹²² and¹²³ public¹²⁴ health¹²⁵ organizations¹²⁶ ,¹²⁷ placi

In [ ]:
#NER
start_ner_company = 1
end_ner_company = 4 + 1

#Q1
start_answer = 1
end_answer = 17 + 1

print(tokenizer.decode(encoded_input["input_ids"][start_ner_company:end_ner_company]))
print(tokenizer.decode(encoded_input["input_ids"][start_answer:end_answer]))

2Morrow
2Morrow is a pioneer in providing science - based, digital health solutions.


In [ ]:
insert_new_labels(sample_id)

NER: {'start': 1, 'end': 5}
Answer: {'start': 1, 'end': 18}
Total Labeling Count: 69


In [ ]:
sample_id = next(data_sample_ids)
print(sample_id)
encoded_input = tokenizer(dict_data[sample_id]['desc'])
dict_data[sample_id]['desc']

1364


'Jamm is a lightweight, spontaneous voice and video collaboration app for remote teams.\nThe app enables day-to-day interactions through video so teams can brainstorm, spontaneously huddle, and provide team updates no matter where they are.'

In [ ]:
" ".join([tokenizer.decode(token) + get_super(str(i)) for i, token in enumerate(encoded_input["input_ids"])])

'[CLS]⁰ Jam¹ ##m² is³ a⁴ lightweight⁵ ,⁶ spontaneous⁷ voice⁸ and⁹ video¹⁰ collaboration¹¹ app¹² for¹³ remote¹⁴ teams¹⁵ .¹⁶ The¹⁷ app¹⁸ enables¹⁹ day²⁰ -²¹ to²² -²³ day²⁴ interactions²⁵ through²⁶ video²⁷ so²⁸ teams²⁹ can³⁰ brains³¹ ##tor³² ##m³³ ,³⁴ spontaneous³⁵ ##ly³⁶ h³⁷ ##udd³⁸ ##le³⁹ ,⁴⁰ and⁴¹ provide⁴² team⁴³ updates⁴⁴ no⁴⁵ matter⁴⁶ where⁴⁷ they⁴⁸ are⁴⁹ .⁵⁰ [SEP]⁵¹'

In [ ]:
#NER
start_ner_company = 1
end_ner_company = 2 + 1

#Q1
start_answer = 1
end_answer = 16 + 1

print(tokenizer.decode(encoded_input["input_ids"][start_ner_company:end_ner_company]))
print(tokenizer.decode(encoded_input["input_ids"][start_answer:end_answer]))

Jamm
Jamm is a lightweight, spontaneous voice and video collaboration app for remote teams.


In [ ]:
insert_new_labels(sample_id)

NER: {'start': 1, 'end': 3}
Answer: {'start': 1, 'end': 17}
Total Labeling Count: 70


In [ ]:
sample_id = next(data_sample_ids)
print(sample_id)
encoded_input = tokenizer(dict_data[sample_id]['desc'])
dict_data[sample_id]['desc']

999


'XRHealth is a medical device company that develops XR (VR\\AR) therapeutic platform. The XRHealth Platform combines medical applications with advanced data analytics and remote monitoring, providing a comprehensive solution for clinicians and patients and pharma companies. The XRHealth Platform collects and analyzes patient interactions withvirtual objects in virtual environments. AI cloud computing algorithms let us deliver an enhanced experience and real-time data analytics that can be used in clinics and in the home. With our technology, patients can analyze and quantify their own performance and track their own improvement over time.XRHealth was established on July 5, 2016 by Eran Orr in Brookline, Massachusetts.Read More'

In [ ]:
" ".join([tokenizer.decode(token) + get_super(str(i)) for i, token in enumerate(encoded_input["input_ids"])])

'[CLS]⁰ X¹ ##R² ##H³ ##eal⁴ ##th⁵ is⁶ a⁷ medical⁸ device⁹ company¹⁰ that¹¹ develops¹² X¹³ ##R¹⁴ (¹⁵ V¹⁶ ##R¹⁷ \\¹⁸ AR¹⁹ )²⁰ therapeutic²¹ platform²² .²³ The²⁴ X²⁵ ##R²⁶ ##H²⁷ ##eal²⁸ ##th²⁹ Platform³⁰ combines³¹ medical³² applications³³ with³⁴ advanced³⁵ data³⁶ anal³⁷ ##ytic³⁸ ##s³⁹ and⁴⁰ remote⁴¹ monitoring⁴² ,⁴³ providing⁴⁴ a⁴⁵ comprehensive⁴⁶ solution⁴⁷ for⁴⁸ clinic⁴⁹ ##ians⁵⁰ and⁵¹ patients⁵² and⁵³ p⁵⁴ ##har⁵⁵ ##ma⁵⁶ companies⁵⁷ .⁵⁸ The⁵⁹ X⁶⁰ ##R⁶¹ ##H⁶² ##eal⁶³ ##th⁶⁴ Platform⁶⁵ collects⁶⁶ and⁶⁷ analyze⁶⁸ ##s⁶⁹ patient⁷⁰ interactions⁷¹ with⁷² ##vir⁷³ ##tu⁷⁴ ##al⁷⁵ objects⁷⁶ in⁷⁷ virtual⁷⁸ environments⁷⁹ .⁸⁰ AI⁸¹ cloud⁸² computing⁸³ algorithms⁸⁴ let⁸⁵ us⁸⁶ deliver⁸⁷ an⁸⁸ enhanced⁸⁹ experience⁹⁰ and⁹¹ real⁹² -⁹³ time⁹⁴ data⁹⁵ anal⁹⁶ ##ytic⁹⁷ ##s⁹⁸ that⁹⁹ can¹⁰⁰ be¹⁰¹ used¹⁰² in¹⁰³ clinics¹⁰⁴ and¹⁰⁵ in¹⁰⁶ the¹⁰⁷ home¹⁰⁸ .¹⁰⁹ With¹¹⁰ our¹¹¹ technology¹¹² ,¹¹³ patients¹¹⁴ can¹¹⁵ analyze¹¹⁶ and¹¹⁷ q¹¹⁸ ##uant¹¹⁹ ##ify¹²⁰ their¹²¹ own¹²² performance¹²³ and¹²⁴ track¹²⁵ their¹²⁶ own¹²⁷ imp

In [ ]:
#NER
start_ner_company = 1
end_ner_company = 5 + 1

#Q1
start_answer = 1
end_answer = 23 + 1

print(tokenizer.decode(encoded_input["input_ids"][start_ner_company:end_ner_company]))
print(tokenizer.decode(encoded_input["input_ids"][start_answer:end_answer]))

XRHealth
XRHealth is a medical device company that develops XR ( VR \ AR ) therapeutic platform.


In [ ]:
insert_new_labels(sample_id)

NER: {'start': 1, 'end': 6}
Answer: {'start': 1, 'end': 24}
Total Labeling Count: 71


In [ ]:
sample_id = next(data_sample_ids)
print(sample_id)
encoded_input = tokenizer(dict_data[sample_id]['desc'])
dict_data[sample_id]['desc']

2991


'Qlik simplifies the way people use data by making it a natural part of how they make decisions. We help people to do more than just report findings; we help them to change their worlds, in ways both small and large, through understanding and sharing data more naturally and effectively to create value. Our mission, “Simplifying decisions foreveryone, everywhere,” is about harnessing the real potential of data – big and small - and unleashing its power to change the world.Read More'

In [ ]:
" ".join([tokenizer.decode(token) + get_super(str(i)) for i, token in enumerate(encoded_input["input_ids"])])

'[CLS]⁰ Q¹ ##lik² si³ ##mp⁴ ##li⁵ ##fies⁶ the⁷ way⁸ people⁹ use¹⁰ data¹¹ by¹² making¹³ it¹⁴ a¹⁵ natural¹⁶ part¹⁷ of¹⁸ how¹⁹ they²⁰ make²¹ decisions²² .²³ We²⁴ help²⁵ people²⁶ to²⁷ do²⁸ more²⁹ than³⁰ just³¹ report³² findings³³ ;³⁴ we³⁵ help³⁶ them³⁷ to³⁸ change³⁹ their⁴⁰ worlds⁴¹ ,⁴² in⁴³ ways⁴⁴ both⁴⁵ small⁴⁶ and⁴⁷ large⁴⁸ ,⁴⁹ through⁵⁰ understanding⁵¹ and⁵² sharing⁵³ data⁵⁴ more⁵⁵ naturally⁵⁶ and⁵⁷ effectively⁵⁸ to⁵⁹ create⁶⁰ value⁶¹ .⁶² Our⁶³ mission⁶⁴ ,⁶⁵ “⁶⁶ Si⁶⁷ ##mp⁶⁸ ##lify⁶⁹ ##ing⁷⁰ decisions⁷¹ forever⁷² ##yon⁷³ ##e⁷⁴ ,⁷⁵ everywhere⁷⁶ ,⁷⁷ ”⁷⁸ is⁷⁹ about⁸⁰ harness⁸¹ ##ing⁸² the⁸³ real⁸⁴ potential⁸⁵ of⁸⁶ data⁸⁷ –⁸⁸ big⁸⁹ and⁹⁰ small⁹¹ -⁹² and⁹³ un⁹⁴ ##lea⁹⁵ ##shing⁹⁶ its⁹⁷ power⁹⁸ to⁹⁹ change¹⁰⁰ the¹⁰¹ world¹⁰² .¹⁰³ Read¹⁰⁴ More¹⁰⁵ [SEP]¹⁰⁶'

In [ ]:
#NER
start_ner_company = 1
end_ner_company = 2 + 1

#Q1
start_answer = 1
end_answer = 23 + 1

print(tokenizer.decode(encoded_input["input_ids"][start_ner_company:end_ner_company]))
print(tokenizer.decode(encoded_input["input_ids"][start_answer:end_answer]))

Qlik
Qlik simplifies the way people use data by making it a natural part of how they make decisions.


In [ ]:
insert_new_labels(sample_id)

NER: {'start': 1, 'end': 3}
Answer: {'start': 1, 'end': 24}
Total Labeling Count: 72


In [ ]:
sample_id = next(data_sample_ids)
print(sample_id)
encoded_input = tokenizer(dict_data[sample_id]['desc'])
dict_data[sample_id]['desc']

1332


'Touchcare is a non-intrusive activity monitoring system for elderly care innovations via body communication. The system also for governments, businesses, and consumers to improve healthcare provision for the elderly, the less-abled, those at risk, and the hospital discharged. They capture every time the medicine box etc is touched.'

In [ ]:
" ".join([tokenizer.decode(token) + get_super(str(i)) for i, token in enumerate(encoded_input["input_ids"])])

'[CLS]⁰ Touch¹ ##care² is³ a⁴ non⁵ -⁶ in⁷ ##tr⁸ ##usive⁹ activity¹⁰ monitoring¹¹ system¹² for¹³ elderly¹⁴ care¹⁵ innovations¹⁶ via¹⁷ body¹⁸ communication¹⁹ .²⁰ The²¹ system²² also²³ for²⁴ governments²⁵ ,²⁶ businesses²⁷ ,²⁸ and²⁹ consumers³⁰ to³¹ improve³² healthcare³³ provision³⁴ for³⁵ the³⁶ elderly³⁷ ,³⁸ the³⁹ less⁴⁰ -⁴¹ able⁴² ##d⁴³ ,⁴⁴ those⁴⁵ at⁴⁶ risk⁴⁷ ,⁴⁸ and⁴⁹ the⁵⁰ hospital⁵¹ discharged⁵² .⁵³ They⁵⁴ capture⁵⁵ every⁵⁶ time⁵⁷ the⁵⁸ medicine⁵⁹ box⁶⁰ etc⁶¹ is⁶² touched⁶³ .⁶⁴ [SEP]⁶⁵'

In [ ]:
#NER
start_ner_company = 1
end_ner_company = 2 + 1

#Q1
start_answer = 1
end_answer = 20 + 1

print(tokenizer.decode(encoded_input["input_ids"][start_ner_company:end_ner_company]))
print(tokenizer.decode(encoded_input["input_ids"][start_answer:end_answer]))

Touchcare
Touchcare is a non - intrusive activity monitoring system for elderly care innovations via body communication.


In [ ]:
insert_new_labels(sample_id)

NER: {'start': 1, 'end': 3}
Answer: {'start': 1, 'end': 21}
Total Labeling Count: 73


In [ ]:
sample_id = next(data_sample_ids)
print(sample_id)
encoded_input = tokenizer(dict_data[sample_id]['desc'])
dict_data[sample_id]['desc']

2882


'Violet Defense is on a journey to find new ways to protect people from harmful germs that have grown resistant to traditional forms of cleaning and disinfecting. After years of research, Mark discovered a unique way to combine a light source with a highly transmissive lens material to produce greater photonic energy.  Because the lens can withstanddirect and immediate contact with the light source, we have the ability to miniaturize the technology deployment.  Mark secured extensive worldwide patent protection for this breakthrough discovery.In 2012, Mark founded Violet Defense to research and develop new products using this patented technology that can help protect our world from germs.  The first line, Surface Air Germ Elimination (S.A.G.E.) System hit the market in May 2017. The possibilities of where this technology goes from here are endless.Read More'

In [ ]:
" ".join([tokenizer.decode(token) + get_super(str(i)) for i, token in enumerate(encoded_input["input_ids"])])

'[CLS]⁰ Violet¹ Defense² is³ on⁴ a⁵ journey⁶ to⁷ find⁸ new⁹ ways¹⁰ to¹¹ protect¹² people¹³ from¹⁴ harmful¹⁵ g¹⁶ ##er¹⁷ ##ms¹⁸ that¹⁹ have²⁰ grown²¹ resistant²² to²³ traditional²⁴ forms²⁵ of²⁶ cleaning²⁷ and²⁸ di²⁹ ##sin³⁰ ##fect³¹ ##ing³² .³³ After³⁴ years³⁵ of³⁶ research³⁷ ,³⁸ Mark³⁹ discovered⁴⁰ a⁴¹ unique⁴² way⁴³ to⁴⁴ combine⁴⁵ a⁴⁶ light⁴⁷ source⁴⁸ with⁴⁹ a⁵⁰ highly⁵¹ trans⁵² ##mis⁵³ ##sive⁵⁴ lens⁵⁵ material⁵⁶ to⁵⁷ produce⁵⁸ greater⁵⁹ photo⁶⁰ ##nic⁶¹ energy⁶² .⁶³ Because⁶⁴ the⁶⁵ lens⁶⁶ can⁶⁷ withstand⁶⁸ ##dir⁶⁹ ##ec⁷⁰ ##t⁷¹ and⁷² immediate⁷³ contact⁷⁴ with⁷⁵ the⁷⁶ light⁷⁷ source⁷⁸ ,⁷⁹ we⁸⁰ have⁸¹ the⁸² ability⁸³ to⁸⁴ mini⁸⁵ ##at⁸⁶ ##uri⁸⁷ ##ze⁸⁸ the⁸⁹ technology⁹⁰ deployment⁹¹ .⁹² Mark⁹³ secured⁹⁴ extensive⁹⁵ worldwide⁹⁶ patent⁹⁷ protection⁹⁸ for⁹⁹ this¹⁰⁰ breakthrough¹⁰¹ discovery¹⁰² .¹⁰³ In¹⁰⁴ 2012¹⁰⁵ ,¹⁰⁶ Mark¹⁰⁷ founded¹⁰⁸ Violet¹⁰⁹ Defense¹¹⁰ to¹¹¹ research¹¹² and¹¹³ develop¹¹⁴ new¹¹⁵ products¹¹⁶ using¹¹⁷ this¹¹⁸ patented¹¹⁹ technology¹²⁰ that¹²¹ can¹²² help¹²³ protect¹²⁴ our¹²

In [ ]:
#NER
start_ner_company = 1
end_ner_company = 2 + 1

#Q1
start_answer = 1
end_answer = 33 + 1

print(tokenizer.decode(encoded_input["input_ids"][start_ner_company:end_ner_company]))
print(tokenizer.decode(encoded_input["input_ids"][start_answer:end_answer]))

Violet Defense
Violet Defense is on a journey to find new ways to protect people from harmful germs that have grown resistant to traditional forms of cleaning and disinfecting.


In [ ]:
insert_new_labels(sample_id)

NER: {'start': 1, 'end': 3}
Answer: {'start': 1, 'end': 34}
Total Labeling Count: 74


In [ ]:
sample_id = next(data_sample_ids)
print(sample_id)
encoded_input = tokenizer(dict_data[sample_id]['desc'])
dict_data[sample_id]['desc']

257


'ScriptDrop’s technology platform enables connectivity across the healthcare ecosystem connecting patients, pharmacists, providers, and pharmaceutical manufacturers. The platform provides a seamless experience for patients, offering them prescription delivery in all 50 states and allowing for important additional services to be provided for betteradherence.The company was founded in 2016 and is headquartered in Columbus, Ohio.Read More'

In [ ]:
" ".join([tokenizer.decode(token) + get_super(str(i)) for i, token in enumerate(encoded_input["input_ids"])])

'[CLS]⁰ Sc¹ ##rip² ##t³ ##D⁴ ##rop⁵ ’⁶ s⁷ technology⁸ platform⁹ enables¹⁰ connectivity¹¹ across¹² the¹³ healthcare¹⁴ ecosystem¹⁵ connecting¹⁶ patients¹⁷ ,¹⁸ p¹⁹ ##har²⁰ ##ma²¹ ##cist²² ##s²³ ,²⁴ providers²⁵ ,²⁶ and²⁷ pharmaceutical²⁸ manufacturers²⁹ .³⁰ The³¹ platform³² provides³³ a³⁴ sea³⁵ ##m³⁶ ##less³⁷ experience³⁸ for³⁹ patients⁴⁰ ,⁴¹ offering⁴² them⁴³ prescription⁴⁴ delivery⁴⁵ in⁴⁶ all⁴⁷ 50⁴⁸ states⁴⁹ and⁵⁰ allowing⁵¹ for⁵² important⁵³ additional⁵⁴ services⁵⁵ to⁵⁶ be⁵⁷ provided⁵⁸ for⁵⁹ better⁶⁰ ##ad⁶¹ ##herence⁶² .⁶³ The⁶⁴ company⁶⁵ was⁶⁶ founded⁶⁷ in⁶⁸ 2016⁶⁹ and⁷⁰ is⁷¹ headquartered⁷² in⁷³ Columbus⁷⁴ ,⁷⁵ Ohio⁷⁶ .⁷⁷ Read⁷⁸ More⁷⁹ [SEP]⁸⁰'

In [ ]:
#NER
start_ner_company = 1
end_ner_company = 5 + 1

#Q1
start_answer = 1
end_answer = 30 + 1

print(tokenizer.decode(encoded_input["input_ids"][start_ner_company:end_ner_company]))
print(tokenizer.decode(encoded_input["input_ids"][start_answer:end_answer]))

ScriptDrop
ScriptDrop ’ s technology platform enables connectivity across the healthcare ecosystem connecting patients, pharmacists, providers, and pharmaceutical manufacturers.


In [ ]:
insert_new_labels(sample_id)

NER: {'start': 1, 'end': 6}
Answer: {'start': 1, 'end': 31}
Total Labeling Count: 75


In [ ]:
sample_id = next(data_sample_ids)
print(sample_id)
encoded_input = tokenizer(dict_data[sample_id]['desc'])
dict_data[sample_id]['desc']

782


"coachbetter ist a digital marketplace that disrupts the way of how soccer coaches learn, communicate and manage for the better. It's our aim to deliver professional coaching content and digital solutions that elevates everyone's coaching game. We offer a digital, holistic and fully integrated SaaS platform for socher coaches all aroundthe world.Read More"

In [ ]:
" ".join([tokenizer.decode(token) + get_super(str(i)) for i, token in enumerate(encoded_input["input_ids"])])

"[CLS]⁰ coach¹ ##bet² ##ter³ is⁴ ##t⁵ a⁶ digital⁷ marketplace⁸ that⁹ disrupt¹⁰ ##s¹¹ the¹² way¹³ of¹⁴ how¹⁵ soccer¹⁶ coaches¹⁷ learn¹⁸ ,¹⁹ communicate²⁰ and²¹ manage²² for²³ the²⁴ better²⁵ .²⁶ It²⁷ '²⁸ s²⁹ our³⁰ aim³¹ to³² deliver³³ professional³⁴ coaching³⁵ content³⁶ and³⁷ digital³⁸ solutions³⁹ that⁴⁰ el⁴¹ ##eva⁴² ##tes⁴³ everyone⁴⁴ '⁴⁵ s⁴⁶ coaching⁴⁷ game⁴⁸ .⁴⁹ We⁵⁰ offer⁵¹ a⁵² digital⁵³ ,⁵⁴ ho⁵⁵ ##listic⁵⁶ and⁵⁷ fully⁵⁸ integrated⁵⁹ Sa⁶⁰ ##a⁶¹ ##S⁶² platform⁶³ for⁶⁴ so⁶⁵ ##cher⁶⁶ coaches⁶⁷ all⁶⁸ around⁶⁹ ##the⁷⁰ world⁷¹ .⁷² Read⁷³ More⁷⁴ [SEP]⁷⁵"

In [ ]:
#NER
start_ner_company = 1
end_ner_company = 3 + 1

#Q1
start_answer = 1
end_answer = 26 + 1

print(tokenizer.decode(encoded_input["input_ids"][start_ner_company:end_ner_company]))
print(tokenizer.decode(encoded_input["input_ids"][start_answer:end_answer]))

coachbetter
coachbetter ist a digital marketplace that disrupts the way of how soccer coaches learn, communicate and manage for the better.


In [ ]:
insert_new_labels(sample_id)

NER: {'start': 1, 'end': 4}
Answer: {'start': 1, 'end': 27}
Total Labeling Count: 76


In [ ]:
sample_id = next(data_sample_ids)
print(sample_id)
encoded_input = tokenizer(dict_data[sample_id]['desc'])
dict_data[sample_id]['desc']

2324


'Clarabridge is a Customer Experience Management platform that helps businesses win the hearts and minds of their customers. They use advanced text analytics, enterprise survey, and robust engagement that transforms all forms of customer feedback empowering confident, decisive action across the business.Since its inception, Clarabridge has helpedhundreds of the world’s leading brands understand and improve their customer experience.Read More'

In [ ]:
" ".join([tokenizer.decode(token) + get_super(str(i)) for i, token in enumerate(encoded_input["input_ids"])])

'[CLS]⁰ Clara¹ ##bridge² is³ a⁴ Custom⁵ ##er⁶ Experience⁷ Management⁸ platform⁹ that¹⁰ helps¹¹ businesses¹² win¹³ the¹⁴ hearts¹⁵ and¹⁶ minds¹⁷ of¹⁸ their¹⁹ customers²⁰ .²¹ They²² use²³ advanced²⁴ text²⁵ anal²⁶ ##ytic²⁷ ##s²⁸ ,²⁹ enterprise³⁰ survey³¹ ,³² and³³ robust³⁴ engagement³⁵ that³⁶ transforms³⁷ all³⁸ forms³⁹ of⁴⁰ customer⁴¹ feedback⁴² em⁴³ ##powering⁴⁴ confident⁴⁵ ,⁴⁶ decisive⁴⁷ action⁴⁸ across⁴⁹ the⁵⁰ business⁵¹ .⁵² Since⁵³ its⁵⁴ inception⁵⁵ ,⁵⁶ Clara⁵⁷ ##bridge⁵⁸ has⁵⁹ helped⁶⁰ ##hun⁶¹ ##dre⁶² ##ds⁶³ of⁶⁴ the⁶⁵ world⁶⁶ ’⁶⁷ s⁶⁸ leading⁶⁹ brands⁷⁰ understand⁷¹ and⁷² improve⁷³ their⁷⁴ customer⁷⁵ experience⁷⁶ .⁷⁷ Read⁷⁸ More⁷⁹ [SEP]⁸⁰'

In [ ]:
#NER
start_ner_company = 1
end_ner_company = 2 + 1

#Q1
start_answer = 1
end_answer = 21 + 1

print(tokenizer.decode(encoded_input["input_ids"][start_ner_company:end_ner_company]))
print(tokenizer.decode(encoded_input["input_ids"][start_answer:end_answer]))

Clarabridge
Clarabridge is a Customer Experience Management platform that helps businesses win the hearts and minds of their customers.


In [ ]:
insert_new_labels(sample_id)

NER: {'start': 1, 'end': 3}
Answer: {'start': 1, 'end': 22}
Total Labeling Count: 77


In [ ]:
sample_id = next(data_sample_ids)
print(sample_id)
encoded_input = tokenizer(dict_data[sample_id]['desc'])
dict_data[sample_id]['desc']

908


'RepSpring takes medical device reps out of the operating room. Using our system, reps can provide remote support to operating surgeons in real-time over a HIPAA-compliant network.\nThe RepSpring system lowers costs, increases efficiency, and removes an extra body from the operating room-- all while providing the support surgeons are accustomed to.'

In [ ]:
" ".join([tokenizer.decode(token) + get_super(str(i)) for i, token in enumerate(encoded_input["input_ids"])])

'[CLS]⁰ Rep¹ ##S² ##p³ ##ring⁴ takes⁵ medical⁶ device⁷ re⁸ ##ps⁹ out¹⁰ of¹¹ the¹² operating¹³ room¹⁴ .¹⁵ Using¹⁶ our¹⁷ system¹⁸ ,¹⁹ re²⁰ ##ps²¹ can²² provide²³ remote²⁴ support²⁵ to²⁶ operating²⁷ surgeon²⁸ ##s²⁹ in³⁰ real³¹ -³² time³³ over³⁴ a³⁵ H³⁶ ##IP³⁷ ##AA³⁸ -³⁹ com⁴⁰ ##pliant⁴¹ network⁴² .⁴³ The⁴⁴ Rep⁴⁵ ##S⁴⁶ ##p⁴⁷ ##ring⁴⁸ system⁴⁹ lower⁵⁰ ##s⁵¹ costs⁵² ,⁵³ increases⁵⁴ efficiency⁵⁵ ,⁵⁶ and⁵⁷ removes⁵⁸ an⁵⁹ extra⁶⁰ body⁶¹ from⁶² the⁶³ operating⁶⁴ room⁶⁵ -⁶⁶ -⁶⁷ all⁶⁸ while⁶⁹ providing⁷⁰ the⁷¹ support⁷² surgeon⁷³ ##s⁷⁴ are⁷⁵ accustomed⁷⁶ to⁷⁷ .⁷⁸ [SEP]⁷⁹'

In [ ]:
#NER
start_ner_company = 1
end_ner_company = 4 + 1

#Q1
start_answer = 1
end_answer = 15 + 1

print(tokenizer.decode(encoded_input["input_ids"][start_ner_company:end_ner_company]))
print(tokenizer.decode(encoded_input["input_ids"][start_answer:end_answer]))

RepSpring
RepSpring takes medical device reps out of the operating room.


In [ ]:
insert_new_labels(sample_id)

NER: {'start': 1, 'end': 5}
Answer: {'start': 1, 'end': 16}
Total Labeling Count: 78


In [ ]:
sample_id = next(data_sample_ids)
print(sample_id)
encoded_input = tokenizer(dict_data[sample_id]['desc'])
dict_data[sample_id]['desc']

977


'Lunit, abbreviated from “learning unit,” is a venture company devoted to developing advanced software for medical data analysis and interpretation via cutting-edge deep learning technology. Our main area of expertise lies in processing medical imaging data. We envision a near-future when our systems would greatly help physicians make accurate,consistent, and efficient clinical decisions, not limited to diagnoses, through our data-driven imaging biomarker technology. \nThe high level of our deep learning technology has been well demonstrated, ranked top 5 at ImageNet Challenge 2015, which is the world’s largest and most prestigious image recognition competition. What distinguishes our technology from that of other similar parties is full accommodation of weakly labeled data, challenging the boundaries of human understanding in terms of pattern recognition. \nFunded by major venture capital companies, Formation 8 and SoftBank Ventures Korea, our endeavors have been well received by the 

In [ ]:
" ".join([tokenizer.decode(token) + get_super(str(i)) for i, token in enumerate(encoded_input["input_ids"])])

'[CLS]⁰ Lu¹ ##ni² ##t³ ,⁴ abbreviated⁵ from⁶ “⁷ learning⁸ unit⁹ ,¹⁰ ”¹¹ is¹² a¹³ venture¹⁴ company¹⁵ devoted¹⁶ to¹⁷ developing¹⁸ advanced¹⁹ software²⁰ for²¹ medical²² data²³ analysis²⁴ and²⁵ interpretation²⁶ via²⁷ cutting²⁸ -²⁹ edge³⁰ deep³¹ learning³² technology³³ .³⁴ Our³⁵ main³⁶ area³⁷ of³⁸ expertise³⁹ lies⁴⁰ in⁴¹ processing⁴² medical⁴³ imaging⁴⁴ data⁴⁵ .⁴⁶ We⁴⁷ en⁴⁸ ##vision⁴⁹ a⁵⁰ near⁵¹ -⁵² future⁵³ when⁵⁴ our⁵⁵ systems⁵⁶ would⁵⁷ greatly⁵⁸ help⁵⁹ physicians⁶⁰ make⁶¹ accurate⁶² ,⁶³ consistent⁶⁴ ,⁶⁵ and⁶⁶ efficient⁶⁷ clinical⁶⁸ decisions⁶⁹ ,⁷⁰ not⁷¹ limited⁷² to⁷³ di⁷⁴ ##ag⁷⁵ ##nose⁷⁶ ##s⁷⁷ ,⁷⁸ through⁷⁹ our⁸⁰ data⁸¹ -⁸² driven⁸³ imaging⁸⁴ bio⁸⁵ ##mark⁸⁶ ##er⁸⁷ technology⁸⁸ .⁸⁹ The⁹⁰ high⁹¹ level⁹² of⁹³ our⁹⁴ deep⁹⁵ learning⁹⁶ technology⁹⁷ has⁹⁸ been⁹⁹ well¹⁰⁰ demonstrated¹⁰¹ ,¹⁰² ranked¹⁰³ top¹⁰⁴ 5¹⁰⁵ at¹⁰⁶ Image¹⁰⁷ ##Net¹⁰⁸ Challenge¹⁰⁹ 2015¹¹⁰ ,¹¹¹ which¹¹² is¹¹³ the¹¹⁴ world¹¹⁵ ’¹¹⁶ s¹¹⁷ largest¹¹⁸ and¹¹⁹ most¹²⁰ prestigious¹²¹ image¹²² recognition¹²³ competition¹²⁴ .¹²⁵ What¹²⁶

In [ ]:
#NER
start_ner_company = 1
end_ner_company = 3 + 1

#Q1
start_answer = 1
end_answer = 34 + 1

print(tokenizer.decode(encoded_input["input_ids"][start_ner_company:end_ner_company]))
print(tokenizer.decode(encoded_input["input_ids"][start_answer:end_answer]))

Lunit
Lunit, abbreviated from “ learning unit, ” is a venture company devoted to developing advanced software for medical data analysis and interpretation via cutting - edge deep learning technology.


In [ ]:
insert_new_labels(sample_id)

NER: {'start': 1, 'end': 4}
Answer: {'start': 1, 'end': 35}
Total Labeling Count: 79


In [ ]:
sample_id = next(data_sample_ids)
print(sample_id)
encoded_input = tokenizer(dict_data[sample_id]['desc'])
dict_data[sample_id]['desc']

3290


"Nama is an Automated Chat Platform using true AI to make time-consuming services simpler, faster and more convenient, saving customers time and money. The Platform includes an proprietary AI API, an Open Source framework to develop chatbots and an AI suggestions system used by customer service representatives. Combining the best of humanintelligence with machines, Nama enables companies to meet their increasing customer expectations at scale. Founded in 2014 in Brazil, Nama brings AI to people's hearts.Bad customer service costs money and customers. The cost of customer service wait times is $100 billion per year in the USA, according to clicksoftware.com. Digital’s Customer Service Benchmark at compete.com indicates that people prefer chat over email or phone to relate to businesses by a margin of 41%. Although chatbots can help users and businesses overcome these losses, there is no simple solution that combines the best aspects of live chat with humans and chatbots.Read More"

In [ ]:
" ".join([tokenizer.decode(token) + get_super(str(i)) for i, token in enumerate(encoded_input["input_ids"])])

"[CLS]⁰ Nam¹ ##a² is³ an⁴ Auto⁵ ##mated⁶ Cha⁷ ##t⁸ Platform⁹ using¹⁰ true¹¹ AI¹² to¹³ make¹⁴ time¹⁵ -¹⁶ consuming¹⁷ services¹⁸ simpler¹⁹ ,²⁰ faster²¹ and²² more²³ convenient²⁴ ,²⁵ saving²⁶ customers²⁷ time²⁸ and²⁹ money³⁰ .³¹ The³² Platform³³ includes³⁴ an³⁵ proprietary³⁶ AI³⁷ API³⁸ ,³⁹ an⁴⁰ Open⁴¹ Source⁴² framework⁴³ to⁴⁴ develop⁴⁵ chat⁴⁶ ##bot⁴⁷ ##s⁴⁸ and⁴⁹ an⁵⁰ AI⁵¹ suggestions⁵² system⁵³ used⁵⁴ by⁵⁵ customer⁵⁶ service⁵⁷ representatives⁵⁸ .⁵⁹ Co⁶⁰ ##mb⁶¹ ##ining⁶² the⁶³ best⁶⁴ of⁶⁵ human⁶⁶ ##int⁶⁷ ##elli⁶⁸ ##gence⁶⁹ with⁷⁰ machines⁷¹ ,⁷² Nam⁷³ ##a⁷⁴ enables⁷⁵ companies⁷⁶ to⁷⁷ meet⁷⁸ their⁷⁹ increasing⁸⁰ customer⁸¹ expectations⁸² at⁸³ scale⁸⁴ .⁸⁵ Founded⁸⁶ in⁸⁷ 2014⁸⁸ in⁸⁹ Brazil⁹⁰ ,⁹¹ Nam⁹² ##a⁹³ brings⁹⁴ AI⁹⁵ to⁹⁶ people⁹⁷ '⁹⁸ s⁹⁹ hearts¹⁰⁰ .¹⁰¹ Bad¹⁰² customer¹⁰³ service¹⁰⁴ costs¹⁰⁵ money¹⁰⁶ and¹⁰⁷ customers¹⁰⁸ .¹⁰⁹ The¹¹⁰ cost¹¹¹ of¹¹² customer¹¹³ service¹¹⁴ wait¹¹⁵ times¹¹⁶ is¹¹⁷ $¹¹⁸ 100¹¹⁹ billion¹²⁰ per¹²¹ year¹²² in¹²³ the¹²⁴ USA¹²⁵ ,¹²⁶ according¹²⁷ to¹²⁸ click¹²⁹ ##so¹³⁰ 

In [ ]:
#NER
start_ner_company = 1
end_ner_company = 2 + 1

#Q1
start_answer = 1
end_answer = 31 + 1

print(tokenizer.decode(encoded_input["input_ids"][start_ner_company:end_ner_company]))
print(tokenizer.decode(encoded_input["input_ids"][start_answer:end_answer]))

Nama
Nama is an Automated Chat Platform using true AI to make time - consuming services simpler, faster and more convenient, saving customers time and money.


In [ ]:
insert_new_labels(sample_id)

NER: {'start': 1, 'end': 3}
Answer: {'start': 1, 'end': 32}
Total Labeling Count: 80


In [ ]:
sample_id = next(data_sample_ids)
print(sample_id)
encoded_input = tokenizer(dict_data[sample_id]['desc'])
dict_data[sample_id]['desc']

583


'Noodle.ai offers pioneering business solutions in Enterprise Artificial Intelligence, a unique collaboration among business executives, process experts, and artificial intelligence technologies (e.g., machine learning, predictive data analytics, data science).  The company creates and implements these solutions to solve complex business challengesand drive dramatic improvements in customer, product, and enterprise operations.Read More'

In [ ]:
" ".join([tokenizer.decode(token) + get_super(str(i)) for i, token in enumerate(encoded_input["input_ids"])])

'[CLS]⁰ No¹ ##od² ##le³ .⁴ a⁵ ##i⁶ offers⁷ pioneering⁸ business⁹ solutions¹⁰ in¹¹ Enterprise¹² Art¹³ ##ific¹⁴ ##ial¹⁵ Intelligence¹⁶ ,¹⁷ a¹⁸ unique¹⁹ collaboration²⁰ among²¹ business²² executives²³ ,²⁴ process²⁵ experts²⁶ ,²⁷ and²⁸ artificial²⁹ intelligence³⁰ technologies³¹ (³² e³³ .³⁴ g³⁵ .³⁶ ,³⁷ machine³⁸ learning³⁹ ,⁴⁰ predict⁴¹ ##ive⁴² data⁴³ anal⁴⁴ ##ytic⁴⁵ ##s⁴⁶ ,⁴⁷ data⁴⁸ science⁴⁹ )⁵⁰ .⁵¹ The⁵² company⁵³ creates⁵⁴ and⁵⁵ implements⁵⁶ these⁵⁷ solutions⁵⁸ to⁵⁹ solve⁶⁰ complex⁶¹ business⁶² challenges⁶³ ##and⁶⁴ drive⁶⁵ dramatic⁶⁶ improvements⁶⁷ in⁶⁸ customer⁶⁹ ,⁷⁰ product⁷¹ ,⁷² and⁷³ enterprise⁷⁴ operations⁷⁵ .⁷⁶ Read⁷⁷ More⁷⁸ [SEP]⁷⁹'

In [ ]:
#NER
start_ner_company = 1
end_ner_company = 6 + 1

#Q1
start_answer = 1
end_answer = 51 + 1

print(tokenizer.decode(encoded_input["input_ids"][start_ner_company:end_ner_company]))
print(tokenizer.decode(encoded_input["input_ids"][start_answer:end_answer]))

Noodle. ai
Noodle. ai offers pioneering business solutions in Enterprise Artificial Intelligence, a unique collaboration among business executives, process experts, and artificial intelligence technologies ( e. g., machine learning, predictive data analytics, data science ).


In [ ]:
insert_new_labels(sample_id)

NER: {'start': 1, 'end': 7}
Answer: {'start': 1, 'end': 52}
Total Labeling Count: 81


In [ ]:
sample_id = next(data_sample_ids)
print(sample_id)
encoded_input = tokenizer(dict_data[sample_id]['desc'])
dict_data[sample_id]['desc']

3289


"Al Jameel Medical Services (AJMS) is devoted to delivering high-quality medical services in compliance with national healthcare regulations and international standards. As a result, they strive to provide safe, readily accessible, and affordable services, which are tailored to clients' and the community's needs. We offer the latesttechnologies in healthcare to provide services. Providers include professionals from a range of backgrounds, such as nurses, laboratory specialists, pharmacists, and biomedical engineers. They can offer on-site consultations and help you determine the best healthcare solutionsRead More"

In [ ]:
" ".join([tokenizer.decode(token) + get_super(str(i)) for i, token in enumerate(encoded_input["input_ids"])])

"[CLS]⁰ Al¹ Jam² ##eel³ Medical⁴ Services⁵ (⁶ AJ⁷ ##MS⁸ )⁹ is¹⁰ devoted¹¹ to¹² delivering¹³ high¹⁴ -¹⁵ quality¹⁶ medical¹⁷ services¹⁸ in¹⁹ compliance²⁰ with²¹ national²² healthcare²³ regulations²⁴ and²⁵ international²⁶ standards²⁷ .²⁸ As²⁹ a³⁰ result³¹ ,³² they³³ s³⁴ ##tri³⁵ ##ve³⁶ to³⁷ provide³⁸ safe³⁹ ,⁴⁰ readily⁴¹ accessible⁴² ,⁴³ and⁴⁴ affordable⁴⁵ services⁴⁶ ,⁴⁷ which⁴⁸ are⁴⁹ tailored⁵⁰ to⁵¹ clients⁵² '⁵³ and⁵⁴ the⁵⁵ community⁵⁶ '⁵⁷ s⁵⁸ needs⁵⁹ .⁶⁰ We⁶¹ offer⁶² the⁶³ latest⁶⁴ ##tech⁶⁵ ##no⁶⁶ ##logies⁶⁷ in⁶⁸ healthcare⁶⁹ to⁷⁰ provide⁷¹ services⁷² .⁷³ Pro⁷⁴ ##vide⁷⁵ ##rs⁷⁶ include⁷⁷ professionals⁷⁸ from⁷⁹ a⁸⁰ range⁸¹ of⁸² backgrounds⁸³ ,⁸⁴ such⁸⁵ as⁸⁶ nurses⁸⁷ ,⁸⁸ laboratory⁸⁹ specialists⁹⁰ ,⁹¹ p⁹² ##har⁹³ ##ma⁹⁴ ##cist⁹⁵ ##s⁹⁶ ,⁹⁷ and⁹⁸ bio⁹⁹ ##medical¹⁰⁰ engineers¹⁰¹ .¹⁰² They¹⁰³ can¹⁰⁴ offer¹⁰⁵ on¹⁰⁶ -¹⁰⁷ site¹⁰⁸ consultation¹⁰⁹ ##s¹¹⁰ and¹¹¹ help¹¹² you¹¹³ determine¹¹⁴ the¹¹⁵ best¹¹⁶ healthcare¹¹⁷ solutions¹¹⁸ ##R¹¹⁹ ##ead¹²⁰ More¹²¹ [SEP]¹²²"

In [ ]:
#NER
start_ner_company = 1
end_ner_company = 5 + 1

#Q1
start_answer = 1
end_answer = 28 + 1

print(tokenizer.decode(encoded_input["input_ids"][start_ner_company:end_ner_company]))
print(tokenizer.decode(encoded_input["input_ids"][start_answer:end_answer]))

Al Jameel Medical Services
Al Jameel Medical Services ( AJMS ) is devoted to delivering high - quality medical services in compliance with national healthcare regulations and international standards.


In [ ]:
insert_new_labels(sample_id)

NER: {'start': 1, 'end': 6}
Answer: {'start': 1, 'end': 29}
Total Labeling Count: 82


In [ ]:
sample_id = next(data_sample_ids)
print(sample_id)
encoded_input = tokenizer(dict_data[sample_id]['desc'])
dict_data[sample_id]['desc']

2224


'Scorechain provides business intelligence and big data for the blockchain technologies.\n\n We provide our customers risk analysis and metrics for transactions and keys.\n\n Our product is the missing link between innovative decentralised technologies and the regulated world of businesses.'

In [ ]:
" ".join([tokenizer.decode(token) + get_super(str(i)) for i, token in enumerate(encoded_input["input_ids"])])

'[CLS]⁰ Score¹ ##cha² ##in³ provides⁴ business⁵ intelligence⁶ and⁷ big⁸ data⁹ for¹⁰ the¹¹ block¹² ##cha¹³ ##in¹⁴ technologies¹⁵ .¹⁶ We¹⁷ provide¹⁸ our¹⁹ customers²⁰ risk²¹ analysis²² and²³ metric²⁴ ##s²⁵ for²⁶ transactions²⁷ and²⁸ keys²⁹ .³⁰ Our³¹ product³² is³³ the³⁴ missing³⁵ link³⁶ between³⁷ innovative³⁸ decent³⁹ ##ral⁴⁰ ##ised⁴¹ technologies⁴² and⁴³ the⁴⁴ regulated⁴⁵ world⁴⁶ of⁴⁷ businesses⁴⁸ .⁴⁹ [SEP]⁵⁰'

In [ ]:
#NER
start_ner_company = 1
end_ner_company = 3 + 1

#Q1
start_answer = 1
end_answer = 16 + 1

print(tokenizer.decode(encoded_input["input_ids"][start_ner_company:end_ner_company]))
print(tokenizer.decode(encoded_input["input_ids"][start_answer:end_answer]))

Scorechain
Scorechain provides business intelligence and big data for the blockchain technologies.


In [ ]:
insert_new_labels(sample_id)

NER: {'start': 1, 'end': 4}
Answer: {'start': 1, 'end': 17}
Total Labeling Count: 83


In [ ]:
sample_id = next(data_sample_ids)
print(sample_id)
encoded_input = tokenizer(dict_data[sample_id]['desc'])
dict_data[sample_id]['desc']

1834


'Naked Insurance is the new generation: effortless, transparent and sincere. Naked Insurance offers short-term retail insurance products.'

In [ ]:
" ".join([tokenizer.decode(token) + get_super(str(i)) for i, token in enumerate(encoded_input["input_ids"])])

'[CLS]⁰ Naked¹ Insurance² is³ the⁴ new⁵ generation⁶ :⁷ effort⁸ ##less⁹ ,¹⁰ transparent¹¹ and¹² sincere¹³ .¹⁴ Naked¹⁵ Insurance¹⁶ offers¹⁷ short¹⁸ -¹⁹ term²⁰ retail²¹ insurance²² products²³ .²⁴ [SEP]²⁵'

In [ ]:
#NER
start_ner_company = 1
end_ner_company = 2 + 1

#Q1
start_answer = 1
end_answer = 14 + 1

print(tokenizer.decode(encoded_input["input_ids"][start_ner_company:end_ner_company]))
print(tokenizer.decode(encoded_input["input_ids"][start_answer:end_answer]))

Naked Insurance
Naked Insurance is the new generation : effortless, transparent and sincere.


In [ ]:
insert_new_labels(sample_id)

NER: {'start': 1, 'end': 3}
Answer: {'start': 1, 'end': 15}
Total Labeling Count: 84


In [ ]:
sample_id = next(data_sample_ids)
print(sample_id)
encoded_input = tokenizer(dict_data[sample_id]['desc'])
dict_data[sample_id]['desc']

373


'Minimum has developed an Enterprise-grade climate impact modelling system, specializing in serving large, complex, distributed enterprises.'

In [ ]:
" ".join([tokenizer.decode(token) + get_super(str(i)) for i, token in enumerate(encoded_input["input_ids"])])

'[CLS]⁰ Mini¹ ##mum² has³ developed⁴ an⁵ Enterprise⁶ -⁷ grade⁸ climate⁹ impact¹⁰ modelling¹¹ system¹² ,¹³ specializing¹⁴ in¹⁵ serving¹⁶ large¹⁷ ,¹⁸ complex¹⁹ ,²⁰ distributed²¹ enterprises²² .²³ [SEP]²⁴'

In [ ]:
#NER
start_ner_company = 1
end_ner_company = 2 + 1

#Q1
start_answer = 1
end_answer = 23 + 1

print(tokenizer.decode(encoded_input["input_ids"][start_ner_company:end_ner_company]))
print(tokenizer.decode(encoded_input["input_ids"][start_answer:end_answer]))

Minimum
Minimum has developed an Enterprise - grade climate impact modelling system, specializing in serving large, complex, distributed enterprises.


In [ ]:
insert_new_labels(sample_id)

NER: {'start': 1, 'end': 3}
Answer: {'start': 1, 'end': 24}
Total Labeling Count: 85


In [ ]:
sample_id = next(data_sample_ids)
print(sample_id)
encoded_input = tokenizer(dict_data[sample_id]['desc'])
dict_data[sample_id]['desc']

329


'Jitterbit is an API transformation company that helps businesses to exploit data from any source. The company provides a platform that enables companies to rapidly connect SaaS on-premise and cloud applications and instantly infuse artificial intelligence into any business process. Its intuitive API creation technology allows companies to reusebusiness-critical applications and data to bring new offerings to market in days, not months.Read More'

In [ ]:
" ".join([tokenizer.decode(token) + get_super(str(i)) for i, token in enumerate(encoded_input["input_ids"])])

'[CLS]⁰ Ji¹ ##tter² ##bit³ is⁴ an⁵ API⁶ transformation⁷ company⁸ that⁹ helps¹⁰ businesses¹¹ to¹² exploit¹³ data¹⁴ from¹⁵ any¹⁶ source¹⁷ .¹⁸ The¹⁹ company²⁰ provides²¹ a²² platform²³ that²⁴ enables²⁵ companies²⁶ to²⁷ rapidly²⁸ connect²⁹ Sa³⁰ ##a³¹ ##S³² on³³ -³⁴ premise³⁵ and³⁶ cloud³⁷ applications³⁸ and³⁹ instantly⁴⁰ in⁴¹ ##fuse⁴² artificial⁴³ intelligence⁴⁴ into⁴⁵ any⁴⁶ business⁴⁷ process⁴⁸ .⁴⁹ Its⁵⁰ in⁵¹ ##tu⁵² ##itive⁵³ API⁵⁴ creation⁵⁵ technology⁵⁶ allows⁵⁷ companies⁵⁸ to⁵⁹ re⁶⁰ ##use⁶¹ ##bus⁶² ##iness⁶³ -⁶⁴ critical⁶⁵ applications⁶⁶ and⁶⁷ data⁶⁸ to⁶⁹ bring⁷⁰ new⁷¹ offerings⁷² to⁷³ market⁷⁴ in⁷⁵ days⁷⁶ ,⁷⁷ not⁷⁸ months⁷⁹ .⁸⁰ Read⁸¹ More⁸² [SEP]⁸³'

In [ ]:
#NER
start_ner_company = 1
end_ner_company = 2 + 1

#Q1
start_answer = 1
end_answer = 18 + 1

print(tokenizer.decode(encoded_input["input_ids"][start_ner_company:end_ner_company]))
print(tokenizer.decode(encoded_input["input_ids"][start_answer:end_answer]))

Jitter
Jitterbit is an API transformation company that helps businesses to exploit data from any source.


In [ ]:
insert_new_labels(sample_id)

NER: {'start': 1, 'end': 3}
Answer: {'start': 1, 'end': 19}
Total Labeling Count: 86


In [ ]:
sample_id = next(data_sample_ids)
print(sample_id)
encoded_input = tokenizer(dict_data[sample_id]['desc'])
dict_data[sample_id]['desc']

1310


'Dog Health Scout provides dog owners with actionable insights to better understand and manage the behavioral health of their dogs. By completing our behavioral assessment form, you gain access to customized training and healthcare guidance for your dog.'

In [ ]:
" ".join([tokenizer.decode(token) + get_super(str(i)) for i, token in enumerate(encoded_input["input_ids"])])

'[CLS]⁰ Dog¹ Health² Scout³ provides⁴ dog⁵ owners⁶ with⁷ action⁸ ##able⁹ insights¹⁰ to¹¹ better¹² understand¹³ and¹⁴ manage¹⁵ the¹⁶ behavioral¹⁷ health¹⁸ of¹⁹ their²⁰ dogs²¹ .²² By²³ completing²⁴ our²⁵ behavioral²⁶ assessment²⁷ form²⁸ ,²⁹ you³⁰ gain³¹ access³² to³³ custom³⁴ ##ized³⁵ training³⁶ and³⁷ healthcare³⁸ guidance³⁹ for⁴⁰ your⁴¹ dog⁴² .⁴³ [SEP]⁴⁴'

In [ ]:
#NER
start_ner_company = 1
end_ner_company = 3 + 1

#Q1
start_answer = 1
end_answer = 22 + 1

print(tokenizer.decode(encoded_input["input_ids"][start_ner_company:end_ner_company]))
print(tokenizer.decode(encoded_input["input_ids"][start_answer:end_answer]))

Dog Health Scout
Dog Health Scout provides dog owners with actionable insights to better understand and manage the behavioral health of their dogs.


In [ ]:
insert_new_labels(sample_id)

NER: {'start': 1, 'end': 4}
Answer: {'start': 1, 'end': 23}
Total Labeling Count: 87


In [ ]:
sample_id = next(data_sample_ids)
print(sample_id)
encoded_input = tokenizer(dict_data[sample_id]['desc'])
dict_data[sample_id]['desc']

2080


'PhishLabs is the leading provider of 24/7 cybersecurity services that protect against threats that exploit people. The company is trusted by top organizations worldwide, including 4 of the 5 largest U.S. financial institutions. PhishLabs combines proprietary technology, intelligence, and human expertise to rapidly detect, analyze, and stoptargeted cyberattacks before they impact organizations. Additionally, the company provides robust threat intelligence that strengthens existing cyber defenses and optimizes threat prevention. Leading organizations partner with PhishLabs to more effectively disrupt targeted cyberattacks, prevent data breaches, and reduce online fraud.Follow PhishLabs:  Twitter   |   http://www.twitter.com/phishlabsLinkedIn   |   http://www.linkedin.com/organization/phishlabsGoogle+   |   https://plus.google.com/+PhishlabsTeam/postsRead More'

In [ ]:
" ".join([tokenizer.decode(token) + get_super(str(i)) for i, token in enumerate(encoded_input["input_ids"])])

'[CLS]⁰ Phi¹ ##sh² ##L³ ##ab⁴ ##s⁵ is⁶ the⁷ leading⁸ provider⁹ of¹⁰ 24¹¹ /¹² 7¹³ c¹⁴ ##y¹⁵ ##bers¹⁶ ##ec¹⁷ ##urity¹⁸ services¹⁹ that²⁰ protect²¹ against²² threats²³ that²⁴ exploit²⁵ people²⁶ .²⁷ The²⁸ company²⁹ is³⁰ trusted³¹ by³² top³³ organizations³⁴ worldwide³⁵ ,³⁶ including³⁷ 4³⁸ of³⁹ the⁴⁰ 5⁴¹ largest⁴² U⁴³ .⁴⁴ S⁴⁵ .⁴⁶ financial⁴⁷ institutions⁴⁸ .⁴⁹ Phi⁵⁰ ##sh⁵¹ ##L⁵² ##ab⁵³ ##s⁵⁴ combines⁵⁵ proprietary⁵⁶ technology⁵⁷ ,⁵⁸ intelligence⁵⁹ ,⁶⁰ and⁶¹ human⁶² expertise⁶³ to⁶⁴ rapidly⁶⁵ detect⁶⁶ ,⁶⁷ analyze⁶⁸ ,⁶⁹ and⁷⁰ stop⁷¹ ##tar⁷² ##get⁷³ ##ed⁷⁴ c⁷⁵ ##y⁷⁶ ##ber⁷⁷ ##att⁷⁸ ##ack⁷⁹ ##s⁸⁰ before⁸¹ they⁸² impact⁸³ organizations⁸⁴ .⁸⁵ Additionally⁸⁶ ,⁸⁷ the⁸⁸ company⁸⁹ provides⁹⁰ robust⁹¹ threat⁹² intelligence⁹³ that⁹⁴ strengthen⁹⁵ ##s⁹⁶ existing⁹⁷ c⁹⁸ ##y⁹⁹ ##ber¹⁰⁰ defenses¹⁰¹ and¹⁰² op¹⁰³ ##ti¹⁰⁴ ##mize¹⁰⁵ ##s¹⁰⁶ threat¹⁰⁷ prevention¹⁰⁸ .¹⁰⁹ Leading¹¹⁰ organizations¹¹¹ partner¹¹² with¹¹³ Phi¹¹⁴ ##sh¹¹⁵ ##L¹¹⁶ ##ab¹¹⁷ ##s¹¹⁸ to¹¹⁹ more¹²⁰ effectively¹²¹ disrupt¹²² targeted¹²³ c¹²⁴ ##y¹²⁵

In [ ]:
#NER
start_ner_company = 1
end_ner_company = 5 + 1

#Q1
start_answer = 1
end_answer = 27 + 1

print(tokenizer.decode(encoded_input["input_ids"][start_ner_company:end_ner_company]))
print(tokenizer.decode(encoded_input["input_ids"][start_answer:end_answer]))

PhishLabs
PhishLabs is the leading provider of 24 / 7 cybersecurity services that protect against threats that exploit people.


In [ ]:
insert_new_labels(sample_id)

NER: {'start': 1, 'end': 6}
Answer: {'start': 1, 'end': 28}
Total Labeling Count: 88


In [ ]:
sample_id = next(data_sample_ids)
print(sample_id)
encoded_input = tokenizer(dict_data[sample_id]['desc'])
dict_data[sample_id]['desc']

2004


'Springbuk provides a health analytics software solution to prevent disease with data. Its health intelligence platform unifies medical claims, pharmacy, biometric, and activity data. The predictive modeling engine helps employers target their engagement with pinpoint accuracy, forecast overall healthcare spending, identify at-risk employees, andmeasure their wellness programs.The company was founded in 2015 and is headquartered in Indianapolis, Indiana.Read More'

In [ ]:
" ".join([tokenizer.decode(token) + get_super(str(i)) for i, token in enumerate(encoded_input["input_ids"])])

'[CLS]⁰ Spring¹ ##bu² ##k³ provides⁴ a⁵ health⁶ anal⁷ ##ytic⁸ ##s⁹ software¹⁰ solution¹¹ to¹² prevent¹³ disease¹⁴ with¹⁵ data¹⁶ .¹⁷ Its¹⁸ health¹⁹ intelligence²⁰ platform²¹ un²² ##ifies²³ medical²⁴ claims²⁵ ,²⁶ pharmacy²⁷ ,²⁸ bio²⁹ ##metric³⁰ ,³¹ and³² activity³³ data³⁴ .³⁵ The³⁶ predict³⁷ ##ive³⁸ modeling³⁹ engine⁴⁰ helps⁴¹ employers⁴² target⁴³ their⁴⁴ engagement⁴⁵ with⁴⁶ pin⁴⁷ ##point⁴⁸ accuracy⁴⁹ ,⁵⁰ forecast⁵¹ overall⁵² healthcare⁵³ spending⁵⁴ ,⁵⁵ identify⁵⁶ at⁵⁷ -⁵⁸ risk⁵⁹ employees⁶⁰ ,⁶¹ and⁶² ##me⁶³ ##as⁶⁴ ##ure⁶⁵ their⁶⁶ well⁶⁷ ##ness⁶⁸ programs⁶⁹ .⁷⁰ The⁷¹ company⁷² was⁷³ founded⁷⁴ in⁷⁵ 2015⁷⁶ and⁷⁷ is⁷⁸ headquartered⁷⁹ in⁸⁰ Indianapolis⁸¹ ,⁸² Indiana⁸³ .⁸⁴ Read⁸⁵ More⁸⁶ [SEP]⁸⁷'

In [ ]:
#NER
start_ner_company = 1
end_ner_company = 3 + 1

#Q1
start_answer = 1
end_answer = 17 + 1

print(tokenizer.decode(encoded_input["input_ids"][start_ner_company:end_ner_company]))
print(tokenizer.decode(encoded_input["input_ids"][start_answer:end_answer]))

Springbuk
Springbuk provides a health analytics software solution to prevent disease with data.


In [ ]:
insert_new_labels(sample_id)

NER: {'start': 1, 'end': 4}
Answer: {'start': 1, 'end': 18}
Total Labeling Count: 89


In [ ]:
sample_id = next(data_sample_ids)
print(sample_id)
encoded_input = tokenizer(dict_data[sample_id]['desc'])
dict_data[sample_id]['desc']

446


'Apostrophe offers an intelligent health benefits platform that delivers benefits for less money to self-insured employers of any size. It has curated their best practices into a plug-n-play health plan that any self-insured employer can seamlessly adopt.'

In [ ]:
" ".join([tokenizer.decode(token) + get_super(str(i)) for i, token in enumerate(encoded_input["input_ids"])])

'[CLS]⁰ A¹ ##post² ##rophe³ offers⁴ an⁵ intelligent⁶ health⁷ benefits⁸ platform⁹ that¹⁰ delivers¹¹ benefits¹² for¹³ less¹⁴ money¹⁵ to¹⁶ self¹⁷ -¹⁸ ins¹⁹ ##ured²⁰ employers²¹ of²² any²³ size²⁴ .²⁵ It²⁶ has²⁷ curated²⁸ their²⁹ best³⁰ practices³¹ into³² a³³ plug³⁴ -³⁵ n³⁶ -³⁷ play³⁸ health³⁹ plan⁴⁰ that⁴¹ any⁴² self⁴³ -⁴⁴ ins⁴⁵ ##ured⁴⁶ employer⁴⁷ can⁴⁸ sea⁴⁹ ##m⁵⁰ ##lessly⁵¹ adopt⁵² .⁵³ [SEP]⁵⁴'

In [ ]:
#NER
start_ner_company = 1
end_ner_company = 3 + 1

#Q1
start_answer = 1
end_answer = 25 + 1

print(tokenizer.decode(encoded_input["input_ids"][start_ner_company:end_ner_company]))
print(tokenizer.decode(encoded_input["input_ids"][start_answer:end_answer]))

Apostrophe
Apostrophe offers an intelligent health benefits platform that delivers benefits for less money to self - insured employers of any size.


In [ ]:
insert_new_labels(sample_id)

NER: {'start': 1, 'end': 4}
Answer: {'start': 1, 'end': 26}
Total Labeling Count: 90


In [ ]:
sample_id = next(data_sample_ids)
print(sample_id)
encoded_input = tokenizer(dict_data[sample_id]['desc'])
dict_data[sample_id]['desc']

1234


'Ascend Money is a fintech startup company that delivers e-payment and micro-lending products.'

In [ ]:
" ".join([tokenizer.decode(token) + get_super(str(i)) for i, token in enumerate(encoded_input["input_ids"])])

'[CLS]⁰ As¹ ##cend² Money³ is⁴ a⁵ fin⁶ ##tech⁷ start⁸ ##up⁹ company¹⁰ that¹¹ delivers¹² e¹³ -¹⁴ payment¹⁵ and¹⁶ micro¹⁷ -¹⁸ lending¹⁹ products²⁰ .²¹ [SEP]²²'

In [ ]:
#NER
start_ner_company = 1
end_ner_company = 3 + 1

#Q1
start_answer = 1
end_answer = 21 + 1

print(tokenizer.decode(encoded_input["input_ids"][start_ner_company:end_ner_company]))
print(tokenizer.decode(encoded_input["input_ids"][start_answer:end_answer]))

Ascend Money
Ascend Money is a fintech startup company that delivers e - payment and micro - lending products.


In [ ]:
insert_new_labels(sample_id)

NER: {'start': 1, 'end': 4}
Answer: {'start': 1, 'end': 22}
Total Labeling Count: 91


In [ ]:
sample_id = next(data_sample_ids)
print(sample_id)
encoded_input = tokenizer(dict_data[sample_id]['desc'])
dict_data[sample_id]['desc']

2257


'UneeQ is a platform that enables chatbots to become an engaging, visual customer experience that creates emotional connection. Using UneeQ, businesses and creators can design digital humans to embody their brand and bring the human touch (emotional quotient or ‘EQ’) back to a digital world - something a form, website or even a chatbot can’t do.UneeQ digital humans are the future brand ambassadors of organizations. UneeQ digital humans have EQ - they can understand and respond to users with empathy and emotion. They function beyond ‘knowledge intelligence’ (IQ) only to build a personal connection with the customers they interact with. Powered by AI, UneeQ’s intelligent digital human platform supercharges a brand’s ability to create, real-time interaction based on tailored content and memorable personas who build an emotional connection with customers by using the power of the human face. UneeQ is a global enterprise with customers across Asia-Pacific, Europe and North America.Read More

In [ ]:
" ".join([tokenizer.decode(token) + get_super(str(i)) for i, token in enumerate(encoded_input["input_ids"])])

'[CLS]⁰ Un¹ ##ee² ##Q³ is⁴ a⁵ platform⁶ that⁷ enables⁸ chat⁹ ##bot¹⁰ ##s¹¹ to¹² become¹³ an¹⁴ engaging¹⁵ ,¹⁶ visual¹⁷ customer¹⁸ experience¹⁹ that²⁰ creates²¹ emotional²² connection²³ .²⁴ Using²⁵ Un²⁶ ##ee²⁷ ##Q²⁸ ,²⁹ businesses³⁰ and³¹ creators³² can³³ design³⁴ digital³⁵ humans³⁶ to³⁷ em³⁸ ##body³⁹ their⁴⁰ brand⁴¹ and⁴² bring⁴³ the⁴⁴ human⁴⁵ touch⁴⁶ (⁴⁷ emotional⁴⁸ q⁴⁹ ##uo⁵⁰ ##tien⁵¹ ##t⁵² or⁵³ ‘⁵⁴ E⁵⁵ ##Q⁵⁶ ’⁵⁷ )⁵⁸ back⁵⁹ to⁶⁰ a⁶¹ digital⁶² world⁶³ -⁶⁴ something⁶⁵ a⁶⁶ form⁶⁷ ,⁶⁸ website⁶⁹ or⁷⁰ even⁷¹ a⁷² chat⁷³ ##bot⁷⁴ can⁷⁵ ’⁷⁶ t⁷⁷ do⁷⁸ .⁷⁹ Un⁸⁰ ##ee⁸¹ ##Q⁸² digital⁸³ humans⁸⁴ are⁸⁵ the⁸⁶ future⁸⁷ brand⁸⁸ ambassador⁸⁹ ##s⁹⁰ of⁹¹ organizations⁹² .⁹³ Un⁹⁴ ##ee⁹⁵ ##Q⁹⁶ digital⁹⁷ humans⁹⁸ have⁹⁹ E¹⁰⁰ ##Q¹⁰¹ -¹⁰² they¹⁰³ can¹⁰⁴ understand¹⁰⁵ and¹⁰⁶ respond¹⁰⁷ to¹⁰⁸ users¹⁰⁹ with¹¹⁰ em¹¹¹ ##pathy¹¹² and¹¹³ emotion¹¹⁴ .¹¹⁵ They¹¹⁶ function¹¹⁷ beyond¹¹⁸ ‘¹¹⁹ knowledge¹²⁰ intelligence¹²¹ ’¹²² (¹²³ I¹²⁴ ##Q¹²⁵ )¹²⁶ only¹²⁷ to¹²⁸ build¹²⁹ a¹³⁰ personal¹³¹ connection¹³² with¹³³ the¹³⁴ customer

In [ ]:
#NER
start_ner_company = 1
end_ner_company = 3 + 1

#Q1
start_answer = 1
end_answer = 24 + 1

print(tokenizer.decode(encoded_input["input_ids"][start_ner_company:end_ner_company]))
print(tokenizer.decode(encoded_input["input_ids"][start_answer:end_answer]))

UneeQ
UneeQ is a platform that enables chatbots to become an engaging, visual customer experience that creates emotional connection.


In [ ]:
insert_new_labels(sample_id)

NER: {'start': 1, 'end': 4}
Answer: {'start': 1, 'end': 25}
Total Labeling Count: 92


In [ ]:
sample_id = next(data_sample_ids)
print(sample_id)
encoded_input = tokenizer(dict_data[sample_id]['desc'])
dict_data[sample_id]['desc']

1192


'Seldon\xa0(London / AI, DevOps)\xa0enables ML engineering teams to accelerate from R&D to production with proven 84% efficiency gains. Customers include large enterprises across sectors and geographies, including leaders in technology, pharma, automotive, finance and retail. Their open-source projects are considered best-of-breed in modelserving, explanations and monitoring with over 2.5m installs 250k models deployed. Leading ML platforms from Google and Red Hat integrate Seldon. Previously raised $3.4m in Jan 2019 in a Seed round led by Amadeus Capital Partners alongside Global Brain and Techstars Ventures. Recently named a Gartner "Cool Vendor" 2020 in AI Core technologies.Read More'

In [ ]:
" ".join([tokenizer.decode(token) + get_super(str(i)) for i, token in enumerate(encoded_input["input_ids"])])

'[CLS]⁰ Se¹ ##ldon² (³ London⁴ /⁵ AI⁶ ,⁷ Dev⁸ ##O⁹ ##ps¹⁰ )¹¹ enables¹² M¹³ ##L¹⁴ engineering¹⁵ teams¹⁶ to¹⁷ accelerate¹⁸ from¹⁹ R²⁰ &²¹ D²² to²³ production²⁴ with²⁵ proven²⁶ 84²⁷ %²⁸ efficiency²⁹ gains³⁰ .³¹ Custom³² ##ers³³ include³⁴ large³⁵ enterprises³⁶ across³⁷ sectors³⁸ and³⁹ g⁴⁰ ##eo⁴¹ ##graph⁴² ##ies⁴³ ,⁴⁴ including⁴⁵ leaders⁴⁶ in⁴⁷ technology⁴⁸ ,⁴⁹ p⁵⁰ ##har⁵¹ ##ma⁵² ,⁵³ automotive⁵⁴ ,⁵⁵ finance⁵⁶ and⁵⁷ retail⁵⁸ .⁵⁹ Their⁶⁰ open⁶¹ -⁶² source⁶³ projects⁶⁴ are⁶⁵ considered⁶⁶ best⁶⁷ -⁶⁸ of⁶⁹ -⁷⁰ breed⁷¹ in⁷² models⁷³ ##er⁷⁴ ##ving⁷⁵ ,⁷⁶ explanations⁷⁷ and⁷⁸ monitoring⁷⁹ with⁸⁰ over⁸¹ 2⁸² .⁸³ 5⁸⁴ ##m⁸⁵ install⁸⁶ ##s⁸⁷ 250⁸⁸ ##k⁸⁹ models⁹⁰ deployed⁹¹ .⁹² Leading⁹³ M⁹⁴ ##L⁹⁵ platforms⁹⁶ from⁹⁷ Google⁹⁸ and⁹⁹ Red¹⁰⁰ Hat¹⁰¹ integrate¹⁰² Se¹⁰³ ##ldon¹⁰⁴ .¹⁰⁵ Previously¹⁰⁶ raised¹⁰⁷ $¹⁰⁸ 3¹⁰⁹ .¹¹⁰ 4¹¹¹ ##m¹¹² in¹¹³ Jan¹¹⁴ 2019¹¹⁵ in¹¹⁶ a¹¹⁷ See¹¹⁸ ##d¹¹⁹ round¹²⁰ led¹²¹ by¹²² Am¹²³ ##ade¹²⁴ ##us¹²⁵ Capital¹²⁶ Partners¹²⁷ alongside¹²⁸ Global¹²⁹ Brain¹³⁰ and¹³¹ Tech¹³² ##stars¹³³ Ventures

In [ ]:
#NER
start_ner_company = 1
end_ner_company = 2 + 1

#Q1
start_answer = 1
end_answer = 31 + 1

print(tokenizer.decode(encoded_input["input_ids"][start_ner_company:end_ner_company]))
print(tokenizer.decode(encoded_input["input_ids"][start_answer:end_answer]))

Seldon
Seldon ( London / AI, DevOps ) enables ML engineering teams to accelerate from R & D to production with proven 84 % efficiency gains.


In [ ]:
insert_new_labels(sample_id)

NER: {'start': 1, 'end': 3}
Answer: {'start': 1, 'end': 32}
Total Labeling Count: 93


In [ ]:
sample_id = next(data_sample_ids)
print(sample_id)
encoded_input = tokenizer(dict_data[sample_id]['desc'])
dict_data[sample_id]['desc']

2894


'Scale accelerates the development of AI applications by helping machine learning teams generate high-quality ground truth data. Our advanced LiDAR, video, and image annotation APIs allow self-driving, drone, and robotics teams at companies like Lyft, OpenAI, Zoox, Pinterest, and Airbnb focus on building differentiated models vs. labeling data.'

In [ ]:
" ".join([tokenizer.decode(token) + get_super(str(i)) for i, token in enumerate(encoded_input["input_ids"])])

'[CLS]⁰ Scale¹ accelerate² ##s³ the⁴ development⁵ of⁶ AI⁷ applications⁸ by⁹ helping¹⁰ machine¹¹ learning¹² teams¹³ generate¹⁴ high¹⁵ -¹⁶ quality¹⁷ ground¹⁸ truth¹⁹ data²⁰ .²¹ Our²² advanced²³ Li²⁴ ##DA²⁵ ##R²⁶ ,²⁷ video²⁸ ,²⁹ and³⁰ image³¹ an³² ##not³³ ##ation³⁴ API³⁵ ##s³⁶ allow³⁷ self³⁸ -³⁹ driving⁴⁰ ,⁴¹ drone⁴² ,⁴³ and⁴⁴ robotic⁴⁵ ##s⁴⁶ teams⁴⁷ at⁴⁸ companies⁴⁹ like⁵⁰ L⁵¹ ##y⁵² ##ft⁵³ ,⁵⁴ Open⁵⁵ ##A⁵⁶ ##I⁵⁷ ,⁵⁸ Zoo⁵⁹ ##x⁶⁰ ,⁶¹ Pi⁶² ##nter⁶³ ##est⁶⁴ ,⁶⁵ and⁶⁶ Air⁶⁷ ##b⁶⁸ ##n⁶⁹ ##b⁷⁰ focus⁷¹ on⁷² building⁷³ differentiated⁷⁴ models⁷⁵ vs⁷⁶ .⁷⁷ label⁷⁸ ##ing⁷⁹ data⁸⁰ .⁸¹ [SEP]⁸²'

In [ ]:
#NER
start_ner_company = 1
end_ner_company = 1 + 1

#Q1
start_answer = 1
end_answer = 21 + 1

print(tokenizer.decode(encoded_input["input_ids"][start_ner_company:end_ner_company]))
print(tokenizer.decode(encoded_input["input_ids"][start_answer:end_answer]))

Scale
Scale accelerates the development of AI applications by helping machine learning teams generate high - quality ground truth data.


In [ ]:
insert_new_labels(sample_id)

NER: {'start': 1, 'end': 2}
Answer: {'start': 1, 'end': 22}
Total Labeling Count: 94


In [ ]:
sample_id = next(data_sample_ids)
print(sample_id)
encoded_input = tokenizer(dict_data[sample_id]['desc'])
dict_data[sample_id]['desc']

511


'Whisper develops AI-enabled, updateable hearing aids for people with hearing loss. Whisper’s Hearing System is linked to a phone, allowing updates to features such as its Sound Separation Engine, which use AI algorithms to optimize sound in different environments.The Whisper Hearing System has earpieces like a traditional hearing aid, and it hasa pocket-sized power and intelligence of the Whisper Brain, which the company says, works wirelessly with the earpieces to enable a proprietary AI-based sound separation engine.Read More'

In [ ]:
" ".join([tokenizer.decode(token) + get_super(str(i)) for i, token in enumerate(encoded_input["input_ids"])])

'[CLS]⁰ W¹ ##his² ##per³ develops⁴ AI⁵ -⁶ enabled⁷ ,⁸ update⁹ ##able¹⁰ hearing¹¹ aids¹² for¹³ people¹⁴ with¹⁵ hearing¹⁶ loss¹⁷ .¹⁸ W¹⁹ ##his²⁰ ##per²¹ ’²² s²³ Hearing²⁴ System²⁵ is²⁶ linked²⁷ to²⁸ a²⁹ phone³⁰ ,³¹ allowing³² updates³³ to³⁴ features³⁵ such³⁶ as³⁷ its³⁸ Sound³⁹ Sep⁴⁰ ##ara⁴¹ ##tion⁴² Engine⁴³ ,⁴⁴ which⁴⁵ use⁴⁶ AI⁴⁷ algorithms⁴⁸ to⁴⁹ op⁵⁰ ##ti⁵¹ ##mize⁵² sound⁵³ in⁵⁴ different⁵⁵ environments⁵⁶ .⁵⁷ The⁵⁸ W⁵⁹ ##his⁶⁰ ##per⁶¹ Hearing⁶² System⁶³ has⁶⁴ ear⁶⁵ ##piece⁶⁶ ##s⁶⁷ like⁶⁸ a⁶⁹ traditional⁷⁰ hearing⁷¹ aid⁷² ,⁷³ and⁷⁴ it⁷⁵ has⁷⁶ ##a⁷⁷ pocket⁷⁸ -⁷⁹ sized⁸⁰ power⁸¹ and⁸² intelligence⁸³ of⁸⁴ the⁸⁵ W⁸⁶ ##his⁸⁷ ##per⁸⁸ Brain⁸⁹ ,⁹⁰ which⁹¹ the⁹² company⁹³ says⁹⁴ ,⁹⁵ works⁹⁶ wireless⁹⁷ ##ly⁹⁸ with⁹⁹ the¹⁰⁰ ear¹⁰¹ ##piece¹⁰² ##s¹⁰³ to¹⁰⁴ enable¹⁰⁵ a¹⁰⁶ proprietary¹⁰⁷ AI¹⁰⁸ -¹⁰⁹ based¹¹⁰ sound¹¹¹ separation¹¹² engine¹¹³ .¹¹⁴ Read¹¹⁵ More¹¹⁶ [SEP]¹¹⁷'

In [ ]:
#NER
start_ner_company = 1
end_ner_company = 3 + 1

#Q1
start_answer = 1
end_answer = 18 + 1

print(tokenizer.decode(encoded_input["input_ids"][start_ner_company:end_ner_company]))
print(tokenizer.decode(encoded_input["input_ids"][start_answer:end_answer]))

Whisper
Whisper develops AI - enabled, updateable hearing aids for people with hearing loss.


In [ ]:
insert_new_labels(sample_id)

NER: {'start': 1, 'end': 4}
Answer: {'start': 1, 'end': 19}
Total Labeling Count: 95


In [ ]:
sample_id = next(data_sample_ids)
print(sample_id)
encoded_input = tokenizer(dict_data[sample_id]['desc'])
dict_data[sample_id]['desc']

2242


'Union Personal offers comprehensive medical coverage solutions across the country. Founded in 1995 and is based in Buenos Aires, Distrito Federal, Argentina.'

In [ ]:
" ".join([tokenizer.decode(token) + get_super(str(i)) for i, token in enumerate(encoded_input["input_ids"])])

'[CLS]⁰ Union¹ Personal² offers³ comprehensive⁴ medical⁵ coverage⁶ solutions⁷ across⁸ the⁹ country¹⁰ .¹¹ Founded¹² in¹³ 1995¹⁴ and¹⁵ is¹⁶ based¹⁷ in¹⁸ Buenos¹⁹ Aires²⁰ ,²¹ Di²² ##st²³ ##rito²⁴ Federal²⁵ ,²⁶ Argentina²⁷ .²⁸ [SEP]²⁹'

In [ ]:
#NER
start_ner_company = 1
end_ner_company = 2 + 1

#Q1
start_answer = 1
end_answer = 11 + 1

print(tokenizer.decode(encoded_input["input_ids"][start_ner_company:end_ner_company]))
print(tokenizer.decode(encoded_input["input_ids"][start_answer:end_answer]))

Union Personal
Union Personal offers comprehensive medical coverage solutions across the country.


In [ ]:
insert_new_labels(sample_id)

NER: {'start': 1, 'end': 3}
Answer: {'start': 1, 'end': 12}
Total Labeling Count: 96


In [ ]:
sample_id = next(data_sample_ids)
print(sample_id)
encoded_input = tokenizer(dict_data[sample_id]['desc'])
dict_data[sample_id]['desc']

1362


'Patriot Benefit Solutions specializes in group health programs, cobra administration, brokerage, consulting, and employee benefits services.'

In [ ]:
" ".join([tokenizer.decode(token) + get_super(str(i)) for i, token in enumerate(encoded_input["input_ids"])])

'[CLS]⁰ Patriot¹ Ben² ##ef³ ##it⁴ Solutions⁵ specializes⁶ in⁷ group⁸ health⁹ programs¹⁰ ,¹¹ co¹² ##bra¹³ administration¹⁴ ,¹⁵ broker¹⁶ ##age¹⁷ ,¹⁸ consulting¹⁹ ,²⁰ and²¹ employee²² benefits²³ services²⁴ .²⁵ [SEP]²⁶'

In [ ]:
#NER
start_ner_company = 1
end_ner_company = 5 + 1

#Q1
start_answer = 1
end_answer = 25 + 1

print(tokenizer.decode(encoded_input["input_ids"][start_ner_company:end_ner_company]))
print(tokenizer.decode(encoded_input["input_ids"][start_answer:end_answer]))

Patriot Benefit Solutions
Patriot Benefit Solutions specializes in group health programs, cobra administration, brokerage, consulting, and employee benefits services.


In [ ]:
insert_new_labels(sample_id)

NER: {'start': 1, 'end': 6}
Answer: {'start': 1, 'end': 26}
Total Labeling Count: 97


In [ ]:
sample_id = next(data_sample_ids)
print(sample_id)
encoded_input = tokenizer(dict_data[sample_id]['desc'])
dict_data[sample_id]['desc']

3336


'ShipBob is a technology company that fulfills e-commerce orders for direct-to-consumer brands. As an end-to-end fulfillment solution, ShipBob provides warehousing and package delivery in 1-2 business days, with a network of fulfillment centers across the US. Its software combines order and inventory management, customer communication, predictiveinsights, and optimized shipping for online merchants. ShipBob’s software is also fully integrated with e-commerce platforms, including eBay, Shopify, Amazon, Magento, and many more. Based in Chicago, Illinois, ShipBob was founded in 2014 by Divey Gulati and Dhruv Saxena.Read More'

In [ ]:
" ".join([tokenizer.decode(token) + get_super(str(i)) for i, token in enumerate(encoded_input["input_ids"])])

'[CLS]⁰ Ship¹ ##B² ##ob³ is⁴ a⁵ technology⁶ company⁷ that⁸ fulfill⁹ ##s¹⁰ e¹¹ -¹² commerce¹³ orders¹⁴ for¹⁵ direct¹⁶ -¹⁷ to¹⁸ -¹⁹ consumer²⁰ brands²¹ .²² As²³ an²⁴ end²⁵ -²⁶ to²⁷ -²⁸ end²⁹ fulfill³⁰ ##ment³¹ solution³² ,³³ Ship³⁴ ##B³⁵ ##ob³⁶ provides³⁷ war³⁸ ##eh³⁹ ##ous⁴⁰ ##ing⁴¹ and⁴² package⁴³ delivery⁴⁴ in⁴⁵ 1⁴⁶ -⁴⁷ 2⁴⁸ business⁴⁹ days⁵⁰ ,⁵¹ with⁵² a⁵³ network⁵⁴ of⁵⁵ fulfill⁵⁶ ##ment⁵⁷ centers⁵⁸ across⁵⁹ the⁶⁰ US⁶¹ .⁶² Its⁶³ software⁶⁴ combines⁶⁵ order⁶⁶ and⁶⁷ inventory⁶⁸ management⁶⁹ ,⁷⁰ customer⁷¹ communication⁷² ,⁷³ predict⁷⁴ ##ive⁷⁵ ##ins⁷⁶ ##ight⁷⁷ ##s⁷⁸ ,⁷⁹ and⁸⁰ op⁸¹ ##ti⁸² ##mized⁸³ shipping⁸⁴ for⁸⁵ online⁸⁶ merchants⁸⁷ .⁸⁸ Ship⁸⁹ ##B⁹⁰ ##ob⁹¹ ’⁹² s⁹³ software⁹⁴ is⁹⁵ also⁹⁶ fully⁹⁷ integrated⁹⁸ with⁹⁹ e¹⁰⁰ -¹⁰¹ commerce¹⁰² platforms¹⁰³ ,¹⁰⁴ including¹⁰⁵ e¹⁰⁶ ##B¹⁰⁷ ##ay¹⁰⁸ ,¹⁰⁹ Shop¹¹⁰ ##ify¹¹¹ ,¹¹² Amazon¹¹³ ,¹¹⁴ Ma¹¹⁵ ##gent¹¹⁶ ##o¹¹⁷ ,¹¹⁸ and¹¹⁹ many¹²⁰ more¹²¹ .¹²² Based¹²³ in¹²⁴ Chicago¹²⁵ ,¹²⁶ Illinois¹²⁷ ,¹²⁸ Ship¹²⁹ ##B¹³⁰ ##ob¹³¹ was¹³² founded¹³³ in¹³⁴ 2014¹³⁵ by

In [ ]:
#NER
start_ner_company = 1
end_ner_company = 3 + 1

#Q1
start_answer = 1
end_answer = 22 + 1

print(tokenizer.decode(encoded_input["input_ids"][start_ner_company:end_ner_company]))
print(tokenizer.decode(encoded_input["input_ids"][start_answer:end_answer]))

ShipBob
ShipBob is a technology company that fulfills e - commerce orders for direct - to - consumer brands.


In [ ]:
insert_new_labels(sample_id)

NER: {'start': 1, 'end': 4}
Answer: {'start': 1, 'end': 23}
Total Labeling Count: 98


In [ ]:
sample_id = next(data_sample_ids)
print(sample_id)
encoded_input = tokenizer(dict_data[sample_id]['desc'])
dict_data[sample_id]['desc']

2213


"At Careseekers Pty Ltd they connect people looking for carers with carers who are qualified and ready to work. They're committed to finding carers no matter what your situation is. Whether you need assistance for yourself or someone recovering from surgery, help with someone with a disability, or simply a carer for someone who wants to remainat home in their senior years, they'll seek out the best and then you choose the one you want. Their online service is simply offering a better way to find carers whether it's for ongoing care or for one hour. In fact it's never been easier, or more affordable, to get in-home care. Trust is a priority for us because they know its essential for you. They know that putting a carer in your home, or a loved one's home, is a big step and that you need that carer to be trustworthy, someone who is of good character and experienced. That why they have a carer on boarding process. All carers on their site have National Police Check Certificates, 100 points

In [ ]:
" ".join([tokenizer.decode(token) + get_super(str(i)) for i, token in enumerate(encoded_input["input_ids"])])

"[CLS]⁰ At¹ Care² ##see³ ##kers⁴ P⁵ ##ty⁶ Ltd⁷ they⁸ connect⁹ people¹⁰ looking¹¹ for¹² care¹³ ##rs¹⁴ with¹⁵ care¹⁶ ##rs¹⁷ who¹⁸ are¹⁹ qualified²⁰ and²¹ ready²² to²³ work²⁴ .²⁵ They²⁶ '²⁷ re²⁸ committed²⁹ to³⁰ finding³¹ care³² ##rs³³ no³⁴ matter³⁵ what³⁶ your³⁷ situation³⁸ is³⁹ .⁴⁰ Whether⁴¹ you⁴² need⁴³ assistance⁴⁴ for⁴⁵ yourself⁴⁶ or⁴⁷ someone⁴⁸ recovering⁴⁹ from⁵⁰ surgery⁵¹ ,⁵² help⁵³ with⁵⁴ someone⁵⁵ with⁵⁶ a⁵⁷ disability⁵⁸ ,⁵⁹ or⁶⁰ simply⁶¹ a⁶² care⁶³ ##r⁶⁴ for⁶⁵ someone⁶⁶ who⁶⁷ wants⁶⁸ to⁶⁹ remain⁷⁰ ##at⁷¹ home⁷² in⁷³ their⁷⁴ senior⁷⁵ years⁷⁶ ,⁷⁷ they⁷⁸ '⁷⁹ ll⁸⁰ seek⁸¹ out⁸² the⁸³ best⁸⁴ and⁸⁵ then⁸⁶ you⁸⁷ choose⁸⁸ the⁸⁹ one⁹⁰ you⁹¹ want⁹² .⁹³ Their⁹⁴ online⁹⁵ service⁹⁶ is⁹⁷ simply⁹⁸ offering⁹⁹ a¹⁰⁰ better¹⁰¹ way¹⁰² to¹⁰³ find¹⁰⁴ care¹⁰⁵ ##rs¹⁰⁶ whether¹⁰⁷ it¹⁰⁸ '¹⁰⁹ s¹¹⁰ for¹¹¹ ongoing¹¹² care¹¹³ or¹¹⁴ for¹¹⁵ one¹¹⁶ hour¹¹⁷ .¹¹⁸ In¹¹⁹ fact¹²⁰ it¹²¹ '¹²² s¹²³ never¹²⁴ been¹²⁵ easier¹²⁶ ,¹²⁷ or¹²⁸ more¹²⁹ affordable¹³⁰ ,¹³¹ to¹³² get¹³³ in¹³⁴ -¹³⁵ home¹³⁶ care¹³⁷ .¹³⁸ Trust¹³⁹ is¹

In [ ]:
#NER
start_ner_company = 2
end_ner_company = 7 + 1

#Q1
start_answer = 1
end_answer = 25 + 1

print(tokenizer.decode(encoded_input["input_ids"][start_ner_company:end_ner_company]))
print(tokenizer.decode(encoded_input["input_ids"][start_answer:end_answer]))

Careseekers Pty Ltd
At Careseekers Pty Ltd they connect people looking for carers with carers who are qualified and ready to work.


In [ ]:
insert_new_labels(sample_id)

NER: {'start': 2, 'end': 8}
Answer: {'start': 1, 'end': 26}
Total Labeling Count: 99


In [ ]:
sample_id = next(data_sample_ids)
print(sample_id)
encoded_input = tokenizer(dict_data[sample_id]['desc'])
dict_data[sample_id]['desc']

832


'ObEN is an artificial intelligence (AI) company that is building a decentralized AI platform for intelligent avatars, enabling never before possible social and virtual interactions. The company’s technology allows users to create intelligent 3D avatars that look, sound, and behave like them. Deployed on the blockchain, ObEN’s Personal AI (PAI)technology enables users to create, use, and manage their own PAI on a secure, decentralized platform. Founded in 2014, ObEN is a Tencent, Softbank Ventures Korea and HTC Vive X portfolio company and is located at Idealab in Pasadena, California. To learn more about ObEN, please visit oben.com.Read More'

In [ ]:
" ".join([tokenizer.decode(token) + get_super(str(i)) for i, token in enumerate(encoded_input["input_ids"])])

'[CLS]⁰ O¹ ##b² ##EN³ is⁴ an⁵ artificial⁶ intelligence⁷ (⁸ AI⁹ )¹⁰ company¹¹ that¹² is¹³ building¹⁴ a¹⁵ decent¹⁶ ##ral¹⁷ ##ized¹⁸ AI¹⁹ platform²⁰ for²¹ intelligent²² a²³ ##vat²⁴ ##ars²⁵ ,²⁶ enabling²⁷ never²⁸ before²⁹ possible³⁰ social³¹ and³² virtual³³ interactions³⁴ .³⁵ The³⁶ company³⁷ ’³⁸ s³⁹ technology⁴⁰ allows⁴¹ users⁴² to⁴³ create⁴⁴ intelligent⁴⁵ 3D⁴⁶ a⁴⁷ ##vat⁴⁸ ##ars⁴⁹ that⁵⁰ look⁵¹ ,⁵² sound⁵³ ,⁵⁴ and⁵⁵ behave⁵⁶ like⁵⁷ them⁵⁸ .⁵⁹ De⁶⁰ ##p⁶¹ ##loy⁶² ##ed⁶³ on⁶⁴ the⁶⁵ block⁶⁶ ##cha⁶⁷ ##in⁶⁸ ,⁶⁹ O⁷⁰ ##b⁷¹ ##EN⁷² ’⁷³ s⁷⁴ Personal⁷⁵ AI⁷⁶ (⁷⁷ PA⁷⁸ ##I⁷⁹ )⁸⁰ technology⁸¹ enables⁸² users⁸³ to⁸⁴ create⁸⁵ ,⁸⁶ use⁸⁷ ,⁸⁸ and⁸⁹ manage⁹⁰ their⁹¹ own⁹² PA⁹³ ##I⁹⁴ on⁹⁵ a⁹⁶ secure⁹⁷ ,⁹⁸ decent⁹⁹ ##ral¹⁰⁰ ##ized¹⁰¹ platform¹⁰² .¹⁰³ Founded¹⁰⁴ in¹⁰⁵ 2014¹⁰⁶ ,¹⁰⁷ O¹⁰⁸ ##b¹⁰⁹ ##EN¹¹⁰ is¹¹¹ a¹¹² Ten¹¹³ ##cent¹¹⁴ ,¹¹⁵ Soft¹¹⁶ ##bank¹¹⁷ Ventures¹¹⁸ Korea¹¹⁹ and¹²⁰ H¹²¹ ##TC¹²² V¹²³ ##ive¹²⁴ X¹²⁵ portfolio¹²⁶ company¹²⁷ and¹²⁸ is¹²⁹ located¹³⁰ at¹³¹ I¹³² ##dea¹³³ ##la¹³⁴ ##b¹³⁵ in¹³⁶ Pasadena¹³⁷ ,¹³⁸ 

In [ ]:
#NER
start_ner_company = 1
end_ner_company = 3 + 1

#Q1
start_answer = 1
end_answer = 35 + 1

print(tokenizer.decode(encoded_input["input_ids"][start_ner_company:end_ner_company]))
print(tokenizer.decode(encoded_input["input_ids"][start_answer:end_answer]))

ObEN
ObEN is an artificial intelligence ( AI ) company that is building a decentralized AI platform for intelligent avatars, enabling never before possible social and virtual interactions.


In [ ]:
insert_new_labels(sample_id)

NER: {'start': 1, 'end': 4}
Answer: {'start': 1, 'end': 36}
Total Labeling Count: 100


In [ ]:
import json
# with open('/content/drive/My Drive/Teste_A3Data/ner_company_dict_list.json', 'w') as f:
#   json.dump(ner_company_dict_list, f)
# with open('/content/drive/My Drive/Teste_A3Data/question_answer_dict_list.json', 'w') as f:
#   json.dump(question_answer_dict_list, f)
with open('/content/drive/My Drive/Teste_A3Data/labels_dict_list.json', 'w') as f:
  json.dump(labels_dict_list, f)